# Setup 

### IMPORTANT: Set your OpenAI key in the "Specify your OpenAI key" cell or this wont run!!!

Note 1: If a cell fails to run, try rerunning it. That usually fixes it. (Usually this is because the agent is getting stuck in a loop of trying to find evidence to support a difficult to support claim). Put it in a try catch loop and it should work eventually

Note 2: Brew some coffee - The full notebook takes about 1 hour to run for an average round. You can see the debate happen in real time by scrolling down.

Note 3: This is running with gpt-4.1-mini. It costs ~1-3 USD to simulate a full debate (run the full notebook) per run. Don't say I didn't warn you! This is a DEEP Debater!

### Install Dependencies

In [ ]:
!pip3 install ducksearch openai ag2 tqdm agentops

### Download the OpenDebateEvidence bm25 index database

In [ ]:
import os
import requests
from tqdm.notebook import tqdm
from IPython.display import display, HTML
import json
import ast


def download_file_from_hf(url, dest_path, chunk_size=1024*1024):
    """
    Download a file from a URL with progress bar.
    """
    if os.path.exists(dest_path):
        print(f"{dest_path} already exists. Skipping download.")
        return
    response = requests.get(url, stream=True)
    total = int(response.headers.get('content-length', 0))
    with open(dest_path, 'wb') as file, tqdm(
        desc=f"Downloading {os.path.basename(dest_path)}",
        total=total,
        unit='B',
        unit_scale=True,
        unit_divisor=1024,
    ) as bar:
        for data in response.iter_content(chunk_size=chunk_size):
            size = file.write(data)
            bar.update(size)
    print(f"Downloaded to {dest_path}")

db_url = "https://huggingface.co/datasets/Hellisotherpeople/OpenDebateEvidenceBM25DuckDB/resolve/main/opendebateevidence_good.duckdb"
db_path = "opendebateevidence_good.duckdb"

download_file_from_hf(db_url, db_path)



### Function for retrieving the actual evidence given ID

In [ ]:
import duckdb


def get_document_by_id(doc_id):
    """
    Query the bm25_tables.documents table for a document by its id.
    Returns the document as a dictionary, or None if not found.
    """
    con = duckdb.connect("opendebateevidence_good.duckdb")
    query = """
        SELECT * FROM bm25_tables.documents WHERE id = ?
    """
    result = con.execute(query, [doc_id]).fetchone()
    #print(result)
    if result is None:
        return None
    # Get column names for dict conversion
    colnames = [desc[0] for desc in con.description]
    con.close()
    return dict(zip(colnames, result))

# Example usage:
doc = get_document_by_id(1655472)
from pprint import pprint


### Function for searching with BM25 and ducksearch

In [ ]:
from ducksearch import search
from IPython.display import display, HTML


def search_debate_cards(query: str) -> list:
    """
    Search the debate card database using BM25 keyword search.

    For best results with BM25 search:
    - Use specific keywords rather than natural sentences
    - Include key technical terms and proper nouns
    - Avoid common words and stop words
    - Order keywords from most to least important
    - Do not search for specific dates (e.g., "2020", "February 15, 2020", etc.)
    - Avoid searching for debate terms like "advantage", "uniqueness", "cp", "solvency", "link", "impact", etc. Instead, focus on the actual topic or content you want evidence about.
    
    Example good query: "Biden foreign policy achievements NATO Ukraine"
    Example bad query: "What are some good things Biden has done as president?"
    Bad query: "uniqueness impact link advantage Biden"
    Bad query: "immigration court reform 2020"

    Args:
        query (str): The search query using keywords and operators.
                     Example: "Biden NATO expansion military aid"

    Returns:
        list: List of matching debate cards (as dicts), ranked by BM25 relevance score.
    """
    top_k = 10
    display_markdown = True
    
    result = search.documents(
        database="opendebateevidence_good.duckdb",
        queries=[query],
        top_k=top_k,
        # filters=filters,
        order_by="score DESC",
    )
    import random
    return_list = [{"id": r.get("id"), "tag": r.get("tag"), "fullcite": r.get("fullcite"), "markup": r.get("markup")} for r in result[0]]
    random.shuffle(return_list)
    
    # Display the markdown of each result using HTML
    if display_markdown:
        # Create a collapsible HTML section for the cards
        collapsible_html = """
        <details style="margin-bottom:1em;">
          <summary style="font-size:1.1em; font-weight:bold; cursor:pointer;">Show Search Results</summary>
          <div id="debate-cards-collapsible">
        """
        for r in return_list:
            if r.get("markup"):
                collapsible_html += r["markup"] + "<hr style='margin:1em 0;'>"
        collapsible_html += "</div></details>"
        display(HTML(collapsible_html))

    return return_list

# Example usage:
results = search_debate_cards("Biden is a good president")[0]
print(results)

### Specify your OpenAI key

In [ ]:
from autogen import ConversableAgent, LLMConfig, register_function
from autogen import GroupChatManager
from autogen import ConversableAgent, Agent
from pydantic import BaseModel, Field
from typing import List, Literal
import agentops
from autogen import GroupChat
from pprint import pprint
import json


ENABLE_AGENTOPS = False  # Set to True to enable agentops.init

if ENABLE_AGENTOPS:
    agentops.init(api_key="")

# Set your OpenAI API key here so you don't have to keep regenerating it
OPENAI_API_KEY = ""


## Specify Debate Topic

In [ ]:
debate_topic = "Resolved: The United States, Canada, and Mexico should form a North American Union similar to the European Union."

# Affirmative Workflows

## Plantext Generation

In [ ]:
def generate_plantext_for_topic(debate_topic: str) -> str:
    """
    Encapsulates the plantext generation workflow and returns the plantext string for the given debate topic.
    """
    # Structured output for plantext generation and review
    class PlantextReview(BaseModel):
        plantext: str
        rationale: str
        advice_for_next_search: str

    llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        parallel_tool_calls=None,
        temperature=2,
        top_p=0.9,
    )
    required_llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        tool_choice="required",
        parallel_tool_calls=None,
        temperature=2,
        top_p=0.9,
    )
    plan_eval_llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        response_format=PlantextReview,
        parallel_tool_calls=None,
        temperature=2,
        top_p=0.9,
    )

    # Agent that devises and iteratively refines a plantext based on evidence
    plantext_generator = ConversableAgent(
        name="plantext_generator",
        system_message=(
            "You are a policy debate expert tasked with generating a well-supported plantext for a given debate topic. "
            "You must extensively review the debate evidence dataset, iteratively searching for evidence and revising your plantext. "
            "Your plantext should be generic enough that it is likely to be well-supported by available evidence in inherency, links, impacts, and solvency, but as specific as possible otherwise.  "
        ),
        llm_config=llm_config,
    )

    # Agent that searches the debate evidence dataset
    debate_search_agent = ConversableAgent(
        name="debate_search_agent",
        system_message="You are a helpful assistant that can search the debate evidence dataset for a given tag or query. Your query will retrieve a list of debate cards.",
        llm_config=required_llm_config,
    )

    # Executor agent for running search tools
    executor_agent = ConversableAgent(
        name="executor_agent",
        human_input_mode="NEVER",
        llm_config=llm_config,
    )

    # Agent that reviews the plantext and determines if it is ready
    plantext_reviewer = ConversableAgent(
        name="plantext_reviewer",
        system_message=(
            "You are a highly rigorous debate coach. Your job is to review the current plantext, the rationale, and the evidence gathered so far. "
            "For each plantext, assess whether it is likely to be well-supported in inherency, links, impacts, and solvency, based on the evidence, for a wide variety of advantages and stock issues. "
            "The plantext should always be written in the form: 'Plan: The <plan actor> should <do actions>'."
            "The plantext should try to use a specific actor (i.e. a specific branch of the federal government) and a specific action (i.e. a specific policy or program). That said, it should remain as generic as possible otherwise so that it's easier to gather evidence in support of it. "
            "Give advice for the next search iteration for how to search for evidence to improve the plantext given the current plantext, evidence, and rationale. "
            "After each search, update your plantext and provide rationale and advice for the next search. "
        ),
        llm_config=plan_eval_llm_config,
    )

    # Register the search function (do not modify this part)
    register_function(
        search_debate_cards,
        caller=debate_search_agent,
        executor=executor_agent,
        description="Search the debate evidence dataset using natural language queries. Return a list of debate cards.",
    )

    iterations = 0
    MAX_ITERATIONS = 4

    def custom_speaker_selection_func(last_speaker: Agent, groupchat: GroupChat):
        """
        Custom speaker selection for plantext generation and review.
        Forces 5 search-review iterations before ending, or ends early if plan_ready is True.
        """
        nonlocal iterations
        messages = groupchat.messages

        if len(messages) == 0:
            return plantext_generator

        # If last was plantext_generator, search next
        if last_speaker is plantext_generator:
            return debate_search_agent

        # If last was search, execute search
        if last_speaker is debate_search_agent:
            return executor_agent

        # If last was executor, review plantext and evidence
        if last_speaker is executor_agent:
            return plantext_reviewer

        # If last was reviewer, check if plan is ready or continue
        if last_speaker is plantext_reviewer:
            iterations += 1
            # print(f"Iteration: {iterations}")
            # Check if plan_ready is True in the last reviewer message
            try:
                content = messages[-1]["content"]
                if isinstance(content, dict):
                    plan_ready = content.get("plan_ready", "False")
                else:
                    import json
                    plan_ready = json.loads(content).get("plan_ready", "False")
            except Exception:
                plan_ready = "False"
            if iterations >= MAX_ITERATIONS:
                return None
            else:
                return debate_search_agent

        # Default fallback
        return "round_robin"

    group_chat = GroupChat(
        agents=[plantext_generator, debate_search_agent, executor_agent, plantext_reviewer],
        messages=[],
        max_round=50,
        speaker_selection_method=custom_speaker_selection_func
    )

    group_chat_manager = GroupChatManager(
        groupchat=group_chat,
        llm_config=llm_config,
    )

    # Example: Generate a plantext for a debate topic
    
    chat_result = plantext_generator.initiate_chat(
        group_chat_manager,
        message=f"Devise a generic plantext for the following debate topic, using iterative literature review: {debate_topic}",
        silent=True,
    )

    plantext_output = json.loads(chat_result.chat_history[-1]["content"])["plantext"]
    return plantext_output



In [ ]:
#plantext_output = generate_plantext_for_topic(debate_topic)

#print(plantext_output)

## Harms Workflow

In [ ]:
def generate_plan_with_harm_evidence(plan_text: str, debate_topic: str) -> str:
    class DebateCard(BaseModel):
        id: int
        cite: str
        include_in_case: Literal["include_it", "False"]
        reason_to_include: str
        retagged_argument_as_read_outloud_in_the_debate_round: str  # Only the argument/tag, not the verbatim card

    class DebateCardSearchResult(BaseModel):
        cards: List[DebateCard] = Field(..., min_items=1, max_items=1)

    llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        temperature=2.0,
        top_p=0.9,
        parallel_tool_calls=None
    )
    required_llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        tool_choice="required",
        temperature=2.0,
        top_p=0.9,
        parallel_tool_calls=None
    )
    debate_eval_llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        response_format=DebateCardSearchResult,
        temperature=2.0,
        top_p=0.9,
        parallel_tool_calls=None
    )

    argument_evaluator = ConversableAgent(
        name="argument_evaluator", 
        system_message=(
            "You are an expert policy debater focused on finding the best possible evidence of harms to support a specific plan. "
            "Your job is to:\n"
            "1. Break down the plan into the exact harms or impacts that must be proven for the case to win.\n"
            "2. Guide evidence collection by:\n"
            "   - Formulating extremely precise search queries that target only evidence which directly and specifically supports the plan's harms or impact claims.\n"
            "   - Using BM25 search to find relevant cards from a debate evidence database (cutoff year 2022).\n"
            "   - Suggesting query refinements to maximize the chance of finding evidence that is both recent and directly supports the plan's harms.\n"
            "3. Evaluate evidence quality for:\n"
            "   - Recency and timeliness (must be as recent as possible, ideally within the last 6 years).\n"
            "   - Direct, explicit support for the plan's harm or impact claim (evidence must not merely be tangentially related).\n"
            "   - Specificity: The evidence must establish that the harm is significant and relevant to the plan, not just a generic or background problem.\n"
            "   - Empirical support and authoritativeness.\n"
            "Reject any evidence that does not fully and directly support the plan's harms or that could be interpreted as generic or non-specific. "
            "Your goal is to find the strictest, most plan-relevant harms evidence possible."
        ),
        llm_config=llm_config,
    )

    debate_eval_agent = ConversableAgent(
        name="debate_eval_agent",
        system_message=(
            "You are an extremely selective and rigorous debate coach and argument analyst. "
            "Your job is to strictly evaluate whether evidence meets the highest standards for inclusion as harms evidence supporting a specific plan in policy debate. "
            "For each piece of evidence, meticulously scrutinize its:\n"
            "- Recency (must be from within the last 6 years unless historically significant)\n"
            "- Author qualifications (must be from recognized experts or authoritative sources)\n"
            "- Empirical basis (must be supported by concrete data and research)\n"
            "- Direct relevance (must precisely and explicitly support the plan's harm or impact claim)\n"
            "- Strategic value (must provide unique and compelling support for the plan's harms, not just generic support)\n"
            "- Specificity (must not duplicate or closely overlap with other selected evidence, and must establish that the harm is significant and directly relevant to the plan)\n"
            "- Wording precision (must use exact terminology needed to establish the impact for the plan)\n\n"
            "After evaluating the evidence, you must:\n"
            "1. IMMEDIATELY REJECT (mark as 'False') any evidence that has already been marked as 'include_it' in previous iterations\n"
            "2. Reject any evidence that duplicates or closely mirrors already selected cards\n"
            "3. Ensure terminology precisely matches what's needed for the plan's impact link chains\n"
            "4. Only approve evidence that meets ALL evaluation criteria and is strictly plan-relevant for harms\n\n"
            "If you decide the card should be included, you MUST:\n"
            "- Retag the evidence with a new, precise long tag in the 'new_long_tag' variable. The new tag must explicitly pertain to a harm that the plan is trying to solve, making clear the specific impact or problem addressed by the plan.\n"
            "- Cut the evidence as a policy debater would: reproduce the EXACT source material (any failure here is strictly grounds for disqualification), but with newly modified highlighting and underlining to emphasize the most important parts for the plan's harms.\n"
            "- Under NO circumstances should you fail to fully and exactly transcribe the selected evidence in the 'new_markup_underlined_highlighted' variable. This must be a faithful, complete reproduction of the chosen text, with your new markup applied.\n"
            "- You must also generate a fully detailed, well-structured, and *long* argument in support of the card, explaining how and why this evidence proves the plan's harm. This argument should be written as though it will be orally presented to an audience in a debate round as the first card after the plantext—make it persuasive, logically sound, and tailored to the plan and topic. The argument you generate should be included in the 'retagged_argument_as_read_outloud_in_the_debate_round' field of the DebateCard. The argument must be long, thorough, and detailed, not a short tag or summary. "
            "IMPORTANT: The 'retagged_argument_as_read_outloud_in_the_debate_round' field must ONLY contain the new argument/tag to be read out loud, NOT the verbatim card or evidence text. Do NOT include the full card or any evidence markup in this field—only the argument/tag, and ensure it is a long, detailed, and fully developed argument."
            "\n\nYour goal is to ensure we have the highest quality, plan-specific harms evidence, with absolutely no duplicate or generic cards, and that all included evidence is retagged and recut with precise, policy debate-style markup."
        ),
        llm_config=debate_eval_llm_config,
    )

    debate_search_agent = ConversableAgent(
        name="debate_search_agent",
        system_message="You are a helpful assistant that can search the debate evidence dataset for a given tag. Your query will retrieve a list of debate cards.",
        llm_config=required_llm_config,
    )

    executor_agent = ConversableAgent(
        name="executor_agent",
        human_input_mode="NEVER",
        llm_config=llm_config,
    )

    # 4. Registers the tool with the agents, the description will be used by the LLM
    register_function(
        search_debate_cards,
        caller=debate_search_agent,
        executor=executor_agent,
        description="Search the debate evidence dataset using natural language queries. Return a list of debate cards.",
    )

    from autogen import GroupChat

    global iterations
    iterations = 0

    allowed_transitions = {
        argument_evaluator: [debate_search_agent],
        debate_search_agent: [executor_agent],
        executor_agent: [argument_evaluator]
    } # Not being used but a good example

    def custom_speaker_selection_func(last_speaker: Agent, groupchat: GroupChat):
        """Define a customized speaker selection function.
        A recommended way is to define a transition for each speaker in the groupchat.

        Returns:
            Return an `Agent` class or a string from ['auto', 'manual', 'random', 'round_robin'] to select a default method to use.
        """
        global iterations
        messages = groupchat.messages

        # We'll start with a transition to the planner
        if len(messages) <= 1:
            return argument_evaluator

        if last_speaker is debate_search_agent:
            return executor_agent

        if last_speaker is executor_agent:
            return debate_eval_agent
        
        if last_speaker is debate_eval_agent:
            if "include_it" in messages[-1]["content"]:
                iterations += 1
                print(f"iterations: {iterations}")
                if iterations > 2:
                    return None
                else:
                    return debate_search_agent
            else:
                return debate_search_agent

        if last_speaker is argument_evaluator:
            return debate_search_agent
        else:
            return "round_robin"

    group_chat = GroupChat(
        agents=[argument_evaluator, debate_search_agent, executor_agent, debate_eval_agent],
        messages=[],
        max_round=40,
        speaker_selection_method=custom_speaker_selection_func
    )

    group_chat_manager = GroupChatManager(
        groupchat=group_chat,
        llm_config=llm_config,
    )

    # Example: Replace with a plan-specific harms prompt

    # Include the debate topic as a separate string for context

    chat_result = argument_evaluator.initiate_chat(
        group_chat_manager,
        message=(
            f"Debate Topic: \"{debate_topic}\"\n"
            f"Plan: \"{plan_text}\"\n\n"
            "Assume that the current year is 2022 for the purposes of the 6-year evidence recency requirement.\n"
            "Find the best, most recent, and most plan-specific evidence of harms supporting the plan above. "
            "Only consider evidence that directly and specifically supports the claim that the plan addresses a significant harm or impact. "
            "Reject any evidence that is generic, tangential, or not strictly relevant to the plan's harms. "
            "When generating the retagged_argument_as_read_outloud_in_the_debate_round, ensure that the argument is long, detailed, and fully developed—at least several sentences or a full paragraph, not a short tag or summary."
        ),
    )

    harm_output_raw_string = chat_result.chat_history[-1]["content"]
    harm_card = json.loads(harm_output_raw_string)["cards"][0]
    harm_output_id = harm_card["id"]
    # Only the argument/tag, not the verbatim card
    harm_argument = harm_card.get("retagged_argument_as_read_outloud_in_the_debate_round", "")
    # reason_to_include is not included in the output string per instructions

    doc = get_document_by_id(harm_output_id)
    output_string = (
        f"<div>"
        f"<h2>Debate Topic</h2>"
        f"<p>{debate_topic}</p>"
        f"<h2>Plan</h2>"
        f"<p>{plan_text}</p>"
        f"<h2>Harm Argument</h2>"
        f"<p>{harm_argument}</p>"
        f"<h2>Harm Evidence</h2>"
        f"{doc['markup']}"
        f"</div>"
    )
    return output_string

In [ ]:
#debate_case = generate_plan_with_harm_evidence(plantext_output, debate_topic)

In [ ]:
#
#display(HTML(debate_case))

## Inherency Workflow

In [ ]:
def append_inherency_argument_and_evidence(plan_string):

    class DebateCard(BaseModel):
        id: int
        cite: str
        include_in_case: Literal["include_it", "False"]
        reason_to_include: str
        retagged_argument_as_read_outloud_in_the_debate_round: str  # Argument to be presented as the first card after the plantext in a debate round

    class DebateCardSearchResult(BaseModel):
        cards: List[DebateCard] = Field(..., min_items=1, max_items=1)

    llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        temperature=2.0,
        top_p=0.9,
        parallel_tool_calls=None
    )
    required_llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        tool_choice="required",
        temperature=2.0,
        top_p=0.9,
        parallel_tool_calls=None
    )
    debate_eval_llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        response_format=DebateCardSearchResult,
        temperature=2.0,
        top_p=0.9,
        parallel_tool_calls=None
    )

    # Set up agents and group chat as before
    argument_evaluator = ConversableAgent(
        name="argument_evaluator", 
        system_message=(
            "You are an expert policy debater focused on finding the best possible evidence of inherency to support a specific plan and its articulated harm. "
            "Your job is to:\n"
            "1. Break down the plan and the chosen harm into the exact barriers, structural obstacles, or status quo failures that prevent the plan's harm from being solved under current policy.\n"
            "2. Guide evidence collection by:\n"
            "   - Formulating extremely precise search queries that target only evidence which directly and specifically demonstrates that the harm persists in the status quo and will not be solved absent the plan (i.e., inherency).\n"
            "   - Using BM25 search to find relevant cards from a debate evidence database (cutoff year 2022).\n"
            "   - If you are being called after previous searches, you must significantly modify and refine your BM25 search queries to maximize the chance of finding new, more relevant, or more specific evidence. Do not simply repeat or slightly alter previous queries—make substantial changes to your search approach, keywords, or focus.\n"
            "   - Suggest query refinements to maximize the chance of finding evidence that is both recent and directly supports the inherency claim for the plan and harm.\n"
            "   - Do NOT select or search for any evidence that has already appeared in the debate case so far (including any evidence already included for harms or in previous sections). Exclude any such evidence from your search and selection process.\n"
            "3. Evaluate evidence quality for:\n"
            "   - Recency and timeliness (must be as recent as possible, ideally within the last 6 years).\n"
            "   - Direct, explicit support for the inherency claim (evidence must not merely be tangentially related or generic background).\n"
            "   - Specificity: The evidence must establish that the harm is ongoing and not being solved by current policy, and that the plan is necessary to address it.\n"
            "   - Empirical support and authoritativeness.\n"
            "Reject any evidence that does not fully and directly support the inherency of the harm or that could be interpreted as generic or non-specific. "
            "Your goal is to find the strictest, most plan- and harm-relevant inherency evidence possible, ensuring that each selected card is unique and not a duplicate of any previously included evidence or any evidence already present in the debate case."
        ),
        llm_config=llm_config,
    )

    debate_eval_agent = ConversableAgent(
        name="debate_eval_agent",
        system_message=(
            "You are an extremely selective and rigorous debate coach and argument analyst. "
            "Your job is to strictly evaluate whether evidence meets the highest standards for inclusion as inherency evidence supporting a specific plan and harm in policy debate. "
            "For each piece of evidence, meticulously scrutinize its:\n"
            "- Recency (must be from within the last 6 years unless historically significant)\n"
            "- Author qualifications (must be from recognized experts or authoritative sources)\n"
            "- Empirical basis (must be supported by concrete data and research)\n"
            "- Direct relevance (must precisely and explicitly support the plan's inherency claim for the articulated harm)\n"
            "- Strategic value (must provide unique and compelling support for the inherency of the harm, not just generic background)\n"
            "- Specificity (must not duplicate or closely overlap with other selected evidence, and must establish that the harm is ongoing and not being solved by current policy)\n"
            "- Wording precision (must use exact terminology needed to establish the inherency link for the plan and harm)\n\n"
            "After evaluating the evidence, you must:\n"
            "1. IMMEDIATELY REJECT (mark as 'False' and/or ignore) any evidence that has already been marked as 'include_it' in previous iterations\n"
            "2. Reject any evidence that duplicates already selected cards\n"
            "3. Reject any evidence that has already appeared in the debate case so far (including any evidence used for harms or in previous sections). Do not allow any evidence to be included more than once in the debate case.\n"
            "4. Ensure terminology precisely matches what's needed for the plan's inherency link chains\n"
            "5. Only approve evidence that meets ALL evaluation criteria and is strictly plan- and harm-relevant for inherency\n\n"
            "Your goal is to ensure we have the highest quality, plan- and harm-specific inherency evidence, with absolutely no duplicate or generic cards, and that all included evidence is retagged and recut with precise, policy debate-style markup."
        ),
        llm_config=debate_eval_llm_config,
    )

    debate_search_agent = ConversableAgent(
        name="debate_search_agent",
        system_message=(
            "You are a helpful assistant that can search the debate evidence dataset for a given tag. Your query will retrieve a list of debate cards. "
            "Do NOT search for or return any evidence that has already appeared in the debate case so far (including any evidence already included for harms or in previous sections). Exclude any such evidence from your search results."
        ),
        llm_config=required_llm_config,
    )

    executor_agent = ConversableAgent(
        name="executor_agent",
        human_input_mode="NEVER",
        llm_config=llm_config,
    )

    register_function(
        search_debate_cards,
        caller=debate_search_agent,
        executor=executor_agent,
        description="Search the debate evidence dataset using natural language queries. Return a list of debate cards.",
    )

    from autogen import GroupChat

    global iterations
    iterations = 0

    def custom_speaker_selection_func(last_speaker: Agent, groupchat: GroupChat):
        global iterations
        messages = groupchat.messages

        if len(messages) <= 1:
            return argument_evaluator

        if last_speaker is debate_search_agent:
            return executor_agent

        if last_speaker is executor_agent:
            return debate_eval_agent
        
        if last_speaker is debate_eval_agent:
            if "include_it" in messages[-1]["content"]:
                iterations += 1
                if iterations > 0:
                    return None
                else:
                    return debate_search_agent
            else:
                return debate_search_agent

        if last_speaker is argument_evaluator:
            return debate_search_agent
        else:
            return "round_robin"

    group_chat = GroupChat(
        agents=[argument_evaluator, debate_search_agent, executor_agent, debate_eval_agent],
        messages=[],
        max_round=20,
        speaker_selection_method=custom_speaker_selection_func
    )

    group_chat_manager = GroupChatManager(
        groupchat=group_chat,
        llm_config=llm_config,
    )

    # Run the inherency workflow
    chat_result = argument_evaluator.initiate_chat(
        group_chat_manager,
        message=(
            f"{plan_string}\n\n"
            "Assume that the current year is 2022.\n"
            "Find the best, most recent, and most plan- and harm-specific evidence of inherency supporting the plan and the articulated harm above. "
            "Only consider evidence that directly and specifically supports the claim that the harm persists in the status quo and will not be solved absent the plan (i.e., inherency). "
            "Reject any evidence that is generic, tangential, not relevant to the plan's inherency for the articulated harm. Reject using the harm evidence as the inherency evidence. "
            "Do NOT select or include any evidence that has already appeared in the debate case so far (including any evidence already included for harms or in previous sections)."
        ),
    )


    inherency_output_raw_string = chat_result.chat_history[-1]["content"]
    inherency_card = json.loads(inherency_output_raw_string)["cards"][0]
    inherency_output_id = inherency_card["id"]
    inherency_argument = inherency_card.get("retagged_argument_as_read_outloud_in_the_debate_round", "")

    doc = get_document_by_id(inherency_output_id)

    # Append inherency argument and evidence to plan_string with HTML tags
    inherency_html = (
        f"<div>"
        f"<h2>Inherency Argument</h2>"
        f"<p>{inherency_argument}</p>"
        f"<h2>Inherency Evidence</h2>"
        f"{str(doc['markup'])}"
        f"</div>"
    )
    return plan_string + "\n" + inherency_html

In [ ]:
#debate_case = append_inherency_argument_and_evidence(debate_case)


In [ ]:
#display(HTML(debate_case))

## Advantages

In [ ]:
def generate_advantages(debate_case: str) -> list[str]:

    class Advantage(BaseModel):
        title: str
        core_argument: str

    class Advantages(BaseModel):
        advantages: List[Advantage] = Field(..., min_items=3, max_items=3)
        rationale: str
        advice_for_next_search: str

    llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        parallel_tool_calls=None,
        temperature=2.0,
        top_p=0.9
    )
    required_llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        tool_choice="required",
        parallel_tool_calls=None,
        temperature=2.0,
        top_p=0.9
    )
    adv_eval_llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        response_format=Advantages,
        parallel_tool_calls=None,
        temperature=2.0,
        top_p=0.9
    )

    advantage_generator = ConversableAgent(
        name="advantage_generator",
        system_message=(
            "You are a policy debate expert tasked with generating 1-3 well-supported advantages for a given plantext, using the provided harms and inherency evidence. "
            "You must extensively review the debate evidence dataset, iteratively searching for evidence and revising your advantages. "
            "Each advantage should be clearly articulated, specific to the plantext, and directly supported by the harms and inherency evidence. "
            "Advantages should be distinct, non-redundant, and phrased in a way that makes them easy to support with available evidence. "
            "Importantly, you must follow policy debate tradition: each advantage should be highly unique and diverse, ideally representing a different traditional policy debate domain (e.g., one about the economy, another about the environment, another about geopolitics, public health, social justice, etc.). "
            "Do not generate multiple advantages that are similar or overlap in impact area. Strive for maximum diversity and uniqueness in the types of advantages you generate, drawing on classic policy debate categories and creative, well-supported impacts."
        ),
        llm_config=llm_config,
    )

    debate_search_agent = ConversableAgent(
        name="debate_search_agent",
        system_message="You are a helpful assistant that can search the debate evidence dataset for a given tag or query. Your query will retrieve a list of debate cards.",
        llm_config=required_llm_config,
    )

    executor_agent = ConversableAgent(
        name="executor_agent",
        human_input_mode="NEVER",
        llm_config=llm_config,
    )

    advantage_reviewer = ConversableAgent(
        name="advantage_reviewer",
        system_message=(
            "You are a highly rigorous debate coach. Your job is to review the current set of advantages, the rationale, and the evidence gathered so far. "
            "For each advantage, assess whether it is likely to be well-supported by the plantext, harms, and inherency evidence, and whether it is distinct and non-redundant. "
            "Give advice for the next search iteration for how to search for evidence to improve the advantages given the current advantages, evidence, and rationale. "
            "After each search, update your advantages and provide rationale and advice for the next search. "
            "You may suggest rewording, combining, or splitting advantages as needed to maximize clarity and support. "
            "Encourage the generation of highly unique and diverse advantages, each representing a different traditional policy debate domain (e.g., economy, environment, geopolitics, public health, social justice, etc.), and discourage overlap or redundancy in impact areas."
        ),
        llm_config=adv_eval_llm_config,
    )

    register_function(
        search_debate_cards,
        caller=debate_search_agent,
        executor=executor_agent,
        description="Search the debate evidence dataset using natural language queries. Return a list of debate cards.",
    )

    from autogen import GroupChat

    iterations = 0
    MAX_ITERATIONS = 1

    def custom_speaker_selection_func(last_speaker: Agent, groupchat: GroupChat):
        nonlocal iterations
        messages = groupchat.messages

        if len(messages) == 0:
            return advantage_generator

        if last_speaker is advantage_generator:
            return debate_search_agent

        if last_speaker is debate_search_agent:
            return executor_agent

        if last_speaker is executor_agent:
            return advantage_reviewer

        if last_speaker is advantage_reviewer:
            iterations += 1
            try:
                content = messages[-1]["content"]
                if isinstance(content, dict):
                    plan_ready = content.get("plan_ready", "False")
                else:
                    import json
                    plan_ready = json.loads(content).get("plan_ready", "False")
            except Exception:
                plan_ready = "False"
            if iterations >= MAX_ITERATIONS:
                return None
            else:
                return debate_search_agent

        return "round_robin"

    group_chat = GroupChat(
        agents=[advantage_generator, debate_search_agent, executor_agent, advantage_reviewer],
        messages=[],
        max_round=50,
        speaker_selection_method=custom_speaker_selection_func
    )

    group_chat_manager = GroupChatManager(
        groupchat=group_chat,
        llm_config=llm_config,
    )

    # Parse the debate_case string to extract plantext, harms, and inherency evidence
    # For this rewrite, we assume debate_case contains all necessary info in a formatted string
    # and we simply pass it as context to the prompt.

    advantage_prompt = (
        f"Given the following debate case, generate 3 distinct, well-supported advantages. "
        f"Do not generate multiple advantages that are similar or overlap in impact area. "
        f"{debate_case}\n"
        "Use iterative literature review to ensure each advantage is specific, distinct, and well-supported. "
        "Strive for maximum diversity and uniqueness in the types of advantages you generate, drawing on both classic policy debate categories and creative, well-supported impacts. "
        "You are encouraged to generate highly progressive or novel arguments, including 'kritikal' (critical theory-based) advantages, but you should be just as likely to generate traditional, hardcore policy arguments. "
    )

    chat_result = advantage_generator.initiate_chat(
        group_chat_manager,
        message=advantage_prompt,
    )


    advantage_output_raw_string = chat_result.chat_history[-1]["content"]
    advantages = json.loads(advantage_output_raw_string)["advantages"]
    # Return a list of string representations for each advantage
    return [str(adv) for adv in advantages]


In [ ]:
#advantages = generate_advantages(debate_case)

In [ ]:
# advantage_1 = json.loads(json.dumps(ast.literal_eval(advantages[0])))
# advantage_1_title = advantage_1["title"]
# advantage_1_core_argument = advantage_1["core_argument"]

# # Append advantage 1 title and core argument to debate_case using h2, div, and p tags
# debate_case += (
#     f"\n<h2>Advantage 1: {advantage_1_title}</h2>"
#     f"\n<div><p>{advantage_1_core_argument}</p></div>"
# )

In [ ]:
#display(HTML(debate_case))

### Advantage Uniqueness

In [ ]:
def add_advantage_uniqueness_to_case(debate_case, advantage_number: str):
    """
    Given a debate_case string and an advantage_number (as a string, e.g., "1", "2", etc.),
    finds the best uniqueness evidence for the specified advantage and appends it to the debate_case using h2, div, and p tags.
    The output includes the retagged argument as read out loud in the debate round and the card itself, but not the reason.
    Returns the modified debate_case string.
    """
    class DebateCard(BaseModel):
        id: int
        cite: str
        include_in_case: Literal["include_it", "False"]
        reason_to_include: str
        retagged_argument_as_read_outloud_in_the_debate_round: str  # Argument to be presented as the first card after the plantext in a debate round

    class DebateCardSearchResult(BaseModel):
        cards: List[DebateCard] = Field(..., min_items=1, max_items=1)

    llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        temperature=2.0,
        top_p=0.9,
        parallel_tool_calls=None
    )
    required_llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        tool_choice="required",
        temperature=2.0,
        top_p=0.9,
        parallel_tool_calls=None
    )
    debate_eval_llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        response_format=DebateCardSearchResult,
        temperature=2.0,
        top_p=0.9,
        parallel_tool_calls=None
    )

    # Agent setup
    argument_evaluator = ConversableAgent(
        name="argument_evaluator", 
        system_message=(
            f"You are an expert policy debater focused on finding the best possible evidence of uniqueness for a specific advantage in a policy debate case. "
            f"Your job is to:\n"
            f"1. Break down the plan, inherency, harm, and advantage {advantage_number} into their key components and causal relationships.\n"
            f"2. Guide evidence collection by:\n"
            f"   - Formulating extremely precise search queries that target only evidence which directly and specifically demonstrates the current status quo regarding the impact area of advantage {advantage_number} (i.e., uniqueness). "
            f"   - Using BM25 search to find relevant cards from a debate evidence database (cutoff year 2022).\n"
            f"   - If you are being called after previous searches, you must significantly modify and refine your BM25 search queries to maximize the chance of finding new, more relevant, or more specific uniqueness evidence. Do not simply repeat or slightly alter previous queries—make substantial changes to your search approach, keywords, or focus.\n"
            f"   - Suggest query refinements to maximize the chance of finding evidence that directly supports the uniqueness claim for advantage {advantage_number}.\n"
            f"3. Evaluate evidence quality for:\n"
            f"   - Direct, explicit support for the uniqueness claim (evidence must not merely be tangentially related or generic background).\n"
            f"   - Specificity: The evidence must establish the current state of affairs in the impact area of advantage {advantage_number}, and explain why the impact is not already occurring or is not inevitable absent the plan.\n"
            f"   - Empirical support and authoritativeness.\n"
            f"Reject any evidence that does not fully and directly support the uniqueness of advantage {advantage_number} or that could be interpreted as generic or non-specific. "
            f"Your goal is to find the strictest, most advantage- and plan-relevant uniqueness evidence possible, ensuring that each selected card is unique and not a duplicate of any previously included evidence."
            f"\n\nIMPORTANT: Do NOT select or use any evidence (by cite, author, or content) that has already appeared in the current debate_case string below. You must only select new, previously unused evidence. Carefully check the debate_case string for all previously included evidence and avoid any repetition."
        ),
        llm_config=llm_config,
    )

    debate_eval_agent = ConversableAgent(
        name="debate_eval_agent",
        system_message=(
            f"You are an extremely selective and rigorous debate coach and argument analyst. "
            f"Your job is to strictly evaluate whether evidence meets the highest standards for inclusion as uniqueness evidence supporting a specific advantage in policy debate. "
            f"For each piece of evidence, meticulously scrutinize its:\n"
            f"- Author qualifications (must be from recognized experts or authoritative sources)\n"
            f"- Empirical basis (must be supported by concrete data and research)\n"
            f"- Direct relevance (must precisely and explicitly support the uniqueness claim for advantage {advantage_number})\n"
            f"- Strategic value (must provide unique and compelling support for the advantage's uniqueness, not just generic background)\n"
            f"- Specificity (must not duplicate or closely overlap with other selected evidence, and must establish the current state of affairs in the impact area of advantage {advantage_number})\n"
            f"- Wording precision (must use exact terminology needed to establish the uniqueness link for advantage {advantage_number})\n\n"
            f"After evaluating the evidence, you must:\n"
            f"1. IMMEDIATELY REJECT (mark as 'False' and/or ignore) any evidence that has already been marked as 'include_it' in previous iterations\n"
            f"2. Reject any evidence that duplicates already selected cards\n"
            f"3. Ensure terminology precisely matches what's needed for the advantage's uniqueness link chains\n"
            f"4. Only approve evidence that meets ALL evaluation criteria and is strictly advantage- and plan-relevant for uniqueness\n"
            f"5. IMPORTANT: Do NOT approve or select any evidence (by cite, author, or content) that has already appeared in the current debate_case string below. You must only approve new, previously unused evidence. Carefully check the debate_case string for all previously included evidence and avoid any repetition.\n\n"
            f"Your goal is to ensure we have the highest quality, advantage- and plan-specific uniqueness evidence, with absolutely no duplicate or generic cards, and that all included evidence is retagged and recut with precise, policy debate-style markup."
        ),
        llm_config=debate_eval_llm_config,
    )

    debate_search_agent = ConversableAgent(
        name="debate_search_agent",
        system_message="You are a helpful assistant that can search the debate evidence dataset for a given tag. Your query will retrieve a list of debate cards.",
        llm_config=required_llm_config,
    )

    executor_agent = ConversableAgent(
        name="executor_agent",
        human_input_mode="NEVER",
        llm_config=llm_config,
    )

    register_function(
        search_debate_cards,
        caller=debate_search_agent,
        executor=executor_agent,
        description="Search the debate evidence dataset using natural language queries. Return a list of debate cards.",
    )

    from autogen import GroupChat

    iterations = 0

    def custom_speaker_selection_func(last_speaker: Agent, groupchat: GroupChat):
        nonlocal iterations
        messages = groupchat.messages

        if len(messages) <= 1:
            return argument_evaluator

        if last_speaker is debate_search_agent:
            return executor_agent

        if last_speaker is executor_agent:
            return debate_eval_agent
        
        if last_speaker is debate_eval_agent:
            if "include_it" in messages[-1]["content"]:
                iterations += 1
                if iterations >= 3:
                    return None
                else:
                    return debate_search_agent
            else:
                return debate_search_agent

        if last_speaker is argument_evaluator:
            return debate_search_agent
        else:
            return "round_robin"

    group_chat = GroupChat(
        agents=[argument_evaluator, debate_search_agent, executor_agent, debate_eval_agent],
        messages=[],
        max_round=40,
        speaker_selection_method=custom_speaker_selection_func
    )

    group_chat_manager = GroupChatManager(
        groupchat=group_chat,
        llm_config=llm_config,
    )

    # Use the plan_string, which includes the debate topic, the plan, inherency, harm, and the specified advantage, for context
    chat_result = argument_evaluator.initiate_chat(
        group_chat_manager,
        message=(
            f"{debate_case}\n\n"
            "Assume that the current year is 2022.\n"
            f"Find the best, most plan- and advantage-specific evidence of uniqueness supporting advantage {advantage_number} articulated above. "
            f"Only consider evidence that directly and specifically supports the claim that the impact area of advantage {advantage_number} is not already occurring, is not inevitable, or is not being solved in the status quo. "
            f"Reject any evidence that is generic, tangential, or not relevant to the uniqueness of advantage {advantage_number}. "
            f"\n\nIMPORTANT: Do NOT select or use any evidence (by cite, author, or content) that has already appeared in the current debate_case string above. You must only select new, previously unused evidence. Carefully check the debate_case string for all previously included evidence and avoid any repetition."
        ),
    )

    # Parse the result
    advantage_uniqueness_raw_string = chat_result.chat_history[-1]["content"]
    card_json = json.loads(advantage_uniqueness_raw_string)["cards"][0]
    advantage_uniqueness_id = card_json["id"]
    retagged_argument = card_json.get("retagged_argument_as_read_outloud_in_the_debate_round", "")
    # Get the card document
    advantage_uniqueness_doc = get_document_by_id(advantage_uniqueness_id)
    card_markup = str(advantage_uniqueness_doc['markup'])

    # Append to debate_case using h2, div, and p tags
    debate_case += (
        f"\n<h2>Advantage {advantage_number} Uniqueness</h2>"
        f"\n<div><p>{retagged_argument}</p></div>"
        f"\n<div><p>{card_markup}</p></div>"
    )
    return debate_case


In [ ]:
#debate_case = add_advantage_uniqueness_to_case(debate_case, "1")

In [ ]:
#display(HTML(debate_case))

### Advantage Link

In [ ]:
def add_advantage_link_to_case(debate_case, advantage_number: str):
    class DebateCard(BaseModel):
        id: int
        cite: str
        include_in_case: Literal["include_it", "False"]
        reason_to_include: str
        retagged_argument_as_read_outloud_in_the_debate_round: str  # Argument to be presented as the first card after the plantext in a debate round

    class DebateCardSearchResult(BaseModel):
        cards: List[DebateCard] = Field(..., min_items=1, max_items=1)

    llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        temperature=2.0,
        top_p=0.9,
        parallel_tool_calls=None
    )
    required_llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        tool_choice="required",
        temperature=2.0,
        top_p=0.9,
        parallel_tool_calls=None
    )
    debate_eval_llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        response_format=DebateCardSearchResult,
        temperature=2.0,
        top_p=0.9,
        parallel_tool_calls=None
    )

    argument_evaluator = ConversableAgent(
        name="argument_evaluator", 
        system_message=(
            f"You are an expert policy debater focused on finding the best possible evidence of a causal link for advantage {advantage_number} in a policy debate case. "
            f"The plantext, inherency, harm, advantage {advantage_number}, and the uniqueness evidence for advantage {advantage_number} have already been provided. "
            "Your job is to:\n"
            f"1. Break down the plan, inherency, harm, advantage {advantage_number}, and the uniqueness evidence into their key components and causal relationships.\n"
            "2. Guide evidence collection by:\n"
            f"   - Formulating extremely precise search queries that target only evidence which directly and specifically demonstrates that the plan, if enacted, would cause the impact described in advantage {advantage_number} (i.e., link evidence). "
            "   - Using BM25 search to find relevant cards from a debate evidence database.\n"
            "   - If you are being called after previous searches, you must significantly modify and refine your BM25 search queries to maximize the chance of finding new, more relevant, or more specific link evidence. Do not simply repeat or slightly alter previous queries—make substantial changes to your search approach, keywords, or focus.\n"
            "   - Suggest query refinements to maximize the chance of finding evidence that directly supports the link claim for the advantage.\n"
            "3. Evaluate evidence quality for:\n"
            "   - Direct, explicit support for the link claim (evidence must not merely be tangentially related or generic background).\n"
            f"   - Specificity: The evidence must establish that the plan, as proposed, will cause the impact described in advantage {advantage_number}, and explain the mechanism by which this occurs.\n"
            "   - Empirical support and authoritativeness.\n"
            f"Reject any evidence that does not fully and directly support the link between the plan and the impact of advantage {advantage_number}, or that could be interpreted as generic or non-specific. "
            "IMPORTANT: Do NOT select or use any evidence (by cite, author, or content) that has already appeared in the current debate_case string above. You must only select new, previously unused evidence. Carefully check the debate_case string for all previously included evidence and avoid any repetition."
            " Your goal is to find the strictest, most advantage- and plan-relevant link evidence possible, ensuring that each selected card is unique and not a duplicate of any previously included evidence."
        ),
        llm_config=llm_config,
    )

    debate_eval_agent = ConversableAgent(
        name="debate_eval_agent",
        system_message=(
            "You are an extremely selective and rigorous debate coach and argument analyst. "
            f"Your job is to strictly evaluate whether evidence meets the highest standards for inclusion as link evidence supporting advantage {advantage_number} in policy debate. "
            "For each piece of evidence, meticulously scrutinize its:\n"
            "- Author qualifications (must be from recognized experts or authoritative sources)\n"
            "- Empirical basis (must be supported by concrete data and research)\n"
            f"- Direct relevance (must precisely and explicitly support the link claim for advantage {advantage_number})\n"
            "- Strategic value (must provide unique and compelling support for the advantage's link, not just generic background)\n"
            f"- Specificity (must not duplicate or closely overlap with other selected evidence, and must establish that the plan, as proposed, will cause the impact described in advantage {advantage_number})\n"
            "- Wording precision (must use exact terminology needed to establish the link for the advantage)\n\n"
            "After evaluating the evidence, you must:\n"
            "1. IMMEDIATELY REJECT (mark as 'False' and/or ignore) any evidence that has already been marked as 'include_it' in previous iterations\n"
            "2. Reject any evidence that duplicates already selected cards\n"
            "3. Ensure terminology precisely matches what's needed for the advantage's link chains\n"
            "4. Only approve evidence that meets ALL evaluation criteria and is strictly advantage- and plan-relevant for the link\n"
            "5. IMPORTANT: Do NOT select or approve any evidence (by cite, author, or content) that has already appeared in the current debate_case string above. You must only select new, previously unused evidence. Carefully check the debate_case string for all previously included evidence and avoid any repetition.\n\n"
            f"Your goal is to ensure we have the highest quality, advantage- and plan-specific link evidence for advantage {advantage_number}, with absolutely no duplicate or generic cards, and that all included evidence is retagged and recut with precise, policy debate-style markup."
        ),
        llm_config=debate_eval_llm_config,
    )

    debate_search_agent = ConversableAgent(
        name="debate_search_agent",
        system_message=(
            "You are a helpful assistant that can search the debate evidence dataset for a given tag. Your query will retrieve a list of debate cards. "
            "IMPORTANT: Do NOT search for or select any evidence (by cite, author, or content) that has already appeared in the current debate_case string above. You must only search for and select new, previously unused evidence. Carefully check the debate_case string for all previously included evidence and avoid any repetition."
        ),
        llm_config=required_llm_config,
    )

    executor_agent = ConversableAgent(
        name="executor_agent",
        human_input_mode="NEVER",
        llm_config=llm_config,
    )

    register_function(
        search_debate_cards,
        caller=debate_search_agent,
        executor=executor_agent,
        description="Search the debate evidence dataset using natural language queries. Return a list of debate cards.",
    )

    from autogen import GroupChat

    iterations = 0

    def custom_speaker_selection_func(last_speaker: Agent, groupchat: GroupChat):
        nonlocal iterations
        messages = groupchat.messages

        if len(messages) <= 1:
            return argument_evaluator

        if last_speaker is debate_search_agent:
            return executor_agent

        if last_speaker is executor_agent:
            return debate_eval_agent
        
        if last_speaker is debate_eval_agent:
            if "include_it" in messages[-1]["content"]:
                iterations += 1
                print(f"iterations: {iterations}")
                if iterations >= 3:
                    return None
                else:
                    return debate_search_agent
            else:
                return debate_search_agent

        if last_speaker is argument_evaluator:
            return debate_search_agent
        else:
            return "round_robin"

    group_chat = GroupChat(
        agents=[argument_evaluator, debate_search_agent, executor_agent, debate_eval_agent],
        messages=[],
        max_round=40,
        speaker_selection_method=custom_speaker_selection_func
    )

    group_chat_manager = GroupChatManager(
        groupchat=group_chat,
        llm_config=llm_config,
    )

    # Use the debate_case, which includes the debate topic, the plan, inherency, harm, advantage N, and advantage N uniqueness evidence for context

    chat_result = argument_evaluator.initiate_chat(
        group_chat_manager,
        message=(
            f"{debate_case}\n\n"
            "Assume that the current year is 2022.\n"
            f"The inherency, harm, advantage {advantage_number}, and advantage {advantage_number} uniqueness evidence have already been established above.\n"
            f"Find the best and most plan- and advantage-specific evidence of a causal link supporting advantage {advantage_number} articulated above. "
            f"Only consider evidence that directly and specifically supports the claim that the plan, if enacted, would cause the impact described in advantage {advantage_number}. "
            f"Reject any evidence that is generic, tangential, or not relevant to the link between the plan and advantage {advantage_number}. "
            f"\n\nIMPORTANT: Do NOT select or use any evidence (by cite, author, or content) that has already appeared in the current debate_case string above. You must only select new, previously unused evidence. Carefully check the debate_case string for all previously included evidence and avoid any repetition."
            # No recency or cutoff requirements; old evidence is acceptable if it is high quality.
        ),
    )

    advantage_link_raw_string = chat_result.chat_history[-1]["content"]
    advantage_link_json = json.loads(advantage_link_raw_string)
    card_json = advantage_link_json["cards"][0]
    advantage_link_id = card_json["id"]
    retagged_argument = card_json.get("retagged_argument_as_read_outloud_in_the_debate_round", "")

    advantage_link_doc = get_document_by_id(advantage_link_id)
    card_markup = str(advantage_link_doc['markup'])

    # Append to debate_case using h2, div, and p tags
    debate_case += (
        f"\n<h2>Advantage {advantage_number} Link</h2>"
        f"\n<div><p>{retagged_argument}</p></div>"
        f"\n<div><p>{card_markup}</p></div>"
    )
    return debate_case

In [ ]:
#debate_case = add_advantage_link_to_case(debate_case, "1")

In [ ]:
#display(HTML(debate_case))

### Advantage Internal Link

In [ ]:
def add_advantage_internal_link_to_case(debate_case, advantage_number: str):
    class DebateCard(BaseModel):
        id: int
        cite: str
        include_in_case: Literal["include_it", "False"]
        reason_to_include: str
        retagged_argument_as_read_outloud_in_the_debate_round: str  # Argument to be presented as the first card after the plantext in a debate round

    class DebateCardSearchResult(BaseModel):
        cards: List[DebateCard] = Field(..., min_items=1, max_items=1)

    llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        temperature=2.0,
        top_p=0.9,
        parallel_tool_calls=None
    )
    required_llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        tool_choice="required",
        temperature=2.0,
        top_p=0.9,
        parallel_tool_calls=None
    )
    debate_eval_llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        response_format=DebateCardSearchResult,
        temperature=2.0,
        top_p=0.9,
        parallel_tool_calls=None
    )

    argument_evaluator = ConversableAgent(
        name="argument_evaluator", 
        system_message=(
            f"You are an expert policy debater focused on finding the best possible evidence of an internal link for advantage {advantage_number} in a policy debate case. "
            f"The plantext, inherency, harm, advantage {advantage_number}, the uniqueness evidence for advantage {advantage_number}, and the link evidence for advantage {advantage_number} have already been provided. "
            "Your job is to:\n"
            f"1. Break down the plan, inherency, harm, advantage {advantage_number}, uniqueness evidence, and the link evidence into their key components and causal relationships, tracing the internal link chain to the impact.\n"
            "2. Guide evidence collection by:\n"
            f"   - Formulating extremely precise search queries that target only evidence which directly and specifically demonstrates the internal link(s) between the link and the impact described in advantage {advantage_number} (i.e., internal link evidence). "
            "   - Using BM25 search to find relevant cards from a debate evidence database.\n"
            "   - If you are being called after previous searches, you must significantly modify and refine your BM25 search queries to maximize the chance of finding new, more relevant, or more specific internal link evidence. Do not simply repeat or slightly alter previous queries—make substantial changes to your search approach, keywords, or focus.\n"
            "   - Suggest query refinements to maximize the chance of finding evidence that directly supports the internal link claim for the advantage.\n"
            "3. Evaluate evidence quality for:\n"
            "   - Direct, explicit support for the internal link claim (evidence must not merely be tangentially related or generic background).\n"
            f"   - Specificity: The evidence must establish that the internal link(s) in the causal chain for advantage {advantage_number} are true, and explain the mechanism by which the link leads to the impact.\n"
            "   - Empirical support and authoritativeness.\n"
            f"Reject any evidence that does not fully and directly support the internal link(s) between the link and the impact of advantage {advantage_number}, or that could be interpreted as generic or non-specific. "
            "Your goal is to find the strictest, most advantage- and plan-relevant internal link evidence possible, ensuring that each selected card is unique and not a duplicate of any previously included evidence in the case. "
            "IMPORTANT: Do NOT select or search for any card (by id, cite, or content) that already appears in the current debate_case string above. You must only select new, previously unused evidence. Carefully check the debate_case string for all previously included evidence and avoid any repetition. If a card has already appeared in the debate_case, do not include it and do not search for it again."
        ),
        llm_config=llm_config,
    )

    debate_eval_agent = ConversableAgent(
        name="debate_eval_agent",
        system_message=(
            "You are an extremely selective and rigorous debate coach and argument analyst. "
            f"Your job is to strictly evaluate whether evidence meets the highest standards for inclusion as internal link evidence supporting advantage {advantage_number} in policy debate. "
            "For each piece of evidence, meticulously scrutinize its:\n"
            "- Author qualifications (must be from recognized experts or authoritative sources)\n"
            "- Empirical basis (must be supported by concrete data and research)\n"
            f"- Direct relevance (must precisely and explicitly support the internal link claim for advantage {advantage_number})\n"
            "- Strategic value (must provide unique and compelling support for the advantage's internal link, not just generic background)\n"
            f"- Specificity (must not duplicate or closely overlap with other selected evidence, and must establish that the internal link(s) in the causal chain for advantage {advantage_number} are true and relevant)\n"
            "- Wording precision (must use exact terminology needed to establish the internal link for the advantage)\n\n"
            "After evaluating the evidence, you must:\n"
            "1. IMMEDIATELY REJECT (mark as 'False' and/or ignore) any evidence that has already been marked as 'include_it' in previous iterations\n"
            "2. Reject any evidence that duplicates already selected cards or any card that already appears in the current debate_case (by id, cite, or content). If a card has already appeared in the debate_case, do not include it and do not search for it again.\n"
            "3. Ensure terminology precisely matches what's needed for the advantage's internal link chains\n"
            "4. Only approve evidence that meets ALL evaluation criteria and is strictly advantage- and plan-relevant for the internal link\n\n"
            f"Your goal is to ensure we have the highest quality, advantage- and plan-specific internal link evidence for advantage {advantage_number}, with absolutely no duplicate or generic cards, and that all included evidence is retagged and recut with precise, policy debate-style markup."
        ),
        llm_config=debate_eval_llm_config,
    )

    debate_search_agent = ConversableAgent(
        name="debate_search_agent",
        system_message=(
            "You are a helpful assistant that can search the debate evidence dataset for a given tag. Your query will retrieve a list of debate cards. "
            "IMPORTANT: Do NOT search for or return any evidence (by id, cite, or content) that already appears in the current debate_case string above. Only search for and return new, previously unused evidence. Carefully check the debate_case string for all previously included evidence and avoid any repetition. If a card has already appeared in the debate_case, do not include it and do not search for it again."
        ),
        llm_config=required_llm_config,
    )

    executor_agent = ConversableAgent(
        name="executor_agent",
        human_input_mode="NEVER",
        llm_config=llm_config,
    )

    register_function(
        search_debate_cards,
        caller=debate_search_agent,
        executor=executor_agent,
        description="Search the debate evidence dataset using natural language queries. Return a list of debate cards.",
    )

    from autogen import GroupChat

    iterations = 0

    def custom_speaker_selection_func(last_speaker: Agent, groupchat: GroupChat):
        nonlocal iterations
        messages = groupchat.messages

        if len(messages) <= 1:
            return argument_evaluator

        if last_speaker is debate_search_agent:
            return executor_agent

        if last_speaker is executor_agent:
            return debate_eval_agent
        
        if last_speaker is debate_eval_agent:
            if "include_it" in messages[-1]["content"]:
                iterations += 1
                print(f"iterations: {iterations}")
                if iterations >= 3:
                    return None
                else:
                    return debate_search_agent
            else:
                return debate_search_agent

        if last_speaker is argument_evaluator:
            return debate_search_agent
        else:
            return "round_robin"

    group_chat = GroupChat(
        agents=[argument_evaluator, debate_search_agent, executor_agent, debate_eval_agent],
        messages=[],
        max_round=40,
        speaker_selection_method=custom_speaker_selection_func
    )

    group_chat_manager = GroupChatManager(
        groupchat=group_chat,
        llm_config=llm_config,
    )

    # Use the debate_case, which includes the debate topic, the plan, inherency, harm, advantage N, advantage N uniqueness evidence, and link evidence for context

    chat_result = argument_evaluator.initiate_chat(
        group_chat_manager,
        message=(
            f"{debate_case}\n\n"
            "Assume that the current year is 2022.\n"
            f"The inherency, harm, advantage {advantage_number}, advantage {advantage_number} uniqueness evidence, and link evidence have already been established above.\n"
            f"Find the best and most plan- and advantage-specific evidence of an internal link supporting advantage {advantage_number} articulated above. "
            f"Only consider evidence that directly and specifically supports the claim that the internal link(s) in the causal chain for advantage {advantage_number} are true and connect the link to the impact. "
            f"Reject any evidence that is generic, tangential, not relevant to the internal link(s) between the link and the impact for advantage {advantage_number}, or that already appears in the current debate_case (by id, cite, or content). "
            f"IMPORTANT: Do NOT select or search for any evidence (by id, cite, or content) that has already appeared in the debate_case string above. You must only select new, previously unused evidence. Carefully check the debate_case string for all previously included evidence and avoid any repetition. If a card has already appeared in the debate_case, do not include it and do not search for it again."
            # No recency or cutoff requirements; old evidence is acceptable if it is high quality.
        ),
    )

    advantage_internal_link_raw_string = chat_result.chat_history[-1]["content"]
    advantage_internal_link_json = json.loads(advantage_internal_link_raw_string)
    card_json = advantage_internal_link_json["cards"][0]
    advantage_internal_link_id = card_json["id"]
    retagged_argument = card_json.get("retagged_argument_as_read_outloud_in_the_debate_round", "")

    advantage_internal_link_doc = get_document_by_id(advantage_internal_link_id)
    card_markup = str(advantage_internal_link_doc['markup'])

    # Append to debate_case using h2, div, and p tags
    debate_case += (
        f"\n<h2>Advantage {advantage_number} Internal Link</h2>"
        f"\n<div><p>{retagged_argument}</p></div>"
        f"\n<div><p>{card_markup}</p></div>"
    )
    return debate_case

### Advantage Impact

In [ ]:
    def add_advantage_impact_to_case(debate_case, advantage_number: str):
        class DebateCard(BaseModel):
            id: int
            cite: str
            include_in_case: Literal["include_it", "False"]
            reason_to_include: str
            retagged_argument_as_read_outloud_in_the_debate_round: str  # Argument to be presented as the first card after the plantext in a debate round

        class DebateCardSearchResult(BaseModel):
            cards: List[DebateCard] = Field(..., min_items=2, max_items=1)

        llm_config = LLMConfig(
            api_type="openai",
            model="gpt-4.1-mini",
            api_key=OPENAI_API_KEY,
            temperature=2.0,
            top_p=0.9,
            parallel_tool_calls=None
        )
        required_llm_config = LLMConfig(
            api_type="openai",
            model="gpt-4.1-mini",
            api_key=OPENAI_API_KEY,
            tool_choice="required",
            temperature=2.0,
            top_p=0.9,
            parallel_tool_calls=None
        )
        debate_eval_llm_config = LLMConfig(
            api_type="openai",
            model="gpt-4.1-mini",
            api_key=OPENAI_API_KEY,
            response_format=DebateCardSearchResult,
            temperature=2.0,
            top_p=0.9,
            parallel_tool_calls=None
        )

        argument_evaluator = ConversableAgent(
            name="argument_evaluator", 
            system_message=(
                f"You are an expert policy debater focused on finding the best possible evidence of the impact for advantage {advantage_number} in a policy debate case. "
                f"The plantext, inherency, harm, advantage {advantage_number}, the uniqueness evidence for advantage {advantage_number}, and the link evidence for advantage {advantage_number} have already been provided. "
                "Your job is to:\n"
                f"1. Break down the plan, inherency, harm, advantage {advantage_number}, uniqueness evidence, and the link evidence into their key components and causal relationships, tracing the full internal link chain to the impact.\n"
                "2. Guide evidence collection by:\n"
                f"   - Formulating extremely precise search queries that target evidence which demonstrates the most significant, maximized impact that could result from the link chain established for advantage {advantage_number} (i.e., impact evidence). "
                "   - Strongly prefer evidence that supports the largest, most policy-relevant, and extreme impacts possible (such as nuclear war, extinction, existential risk, or catastrophic loss of value to life), even if the connection to the plan and link evidence is indirect or only loosely established. "
                "   - It is acceptable to include evidence of extreme impacts (especially extinction-style impacts) even if the causal connection to the plan is indirect, as long as there is a plausible link chain. "
                "   - Using BM25 search to find relevant cards from a debate evidence database.\n"
                "   - If you are being called after previous searches, you must significantly modify and refine your BM25 search queries to maximize the chance of finding new, more relevant, or more specific impact evidence. Do not simply repeat or slightly alter previous queries—make substantial changes to your search approach, keywords, or focus.\n"
                "   - Suggest query refinements to maximize the chance of finding evidence that is high quality and supports the maximized impact claim for the advantage.\n"
                "3. Evaluate evidence quality for:\n"
                "   - Explicit support for the impact claim (evidence should not merely be generic background, but the connection to the plan can be indirect if the impact is extreme).\n"
                f"   - Specificity: The evidence should ideally establish that the plan, as proposed, will cause the maximized impact described in advantage {advantage_number}, but for extinction or similar extreme impacts, an indirect or plausible connection is sufficient.\n"
                "   - Empirical support and authoritativeness.\n"
                "   - Magnitude: Strongly prefer evidence that supports the largest, most significant impacts (e.g., extinction, nuclear war, existential risk, catastrophic loss of value to life), even if the impact is only indirectly connected to the advantage's link chain.\n"
                "Reject any evidence that is generic, non-specific, or completely disconnected from the established link chain, but be willing to accept indirect or plausible connections for extreme impacts. "
                "IMPORTANT: Do NOT select or search for any evidence (by id, cite, or content) that has already appeared in the debate_case string above. You must only select new, previously unused evidence. Carefully check the debate_case string for all previously included evidence and avoid any repetition. If a card has already appeared in the debate_case, do not include it and do not search for it again. "
                "Your goal is to find the strictest, most advantage- and plan-relevant impact evidence possible, maximizing the magnitude of the impact, and it is acceptable to include evidence of extinction or similar extreme impacts even if the connection to the plan is indirect, as long as it is not wholly implausible or generic. Ensure each selected card is unique and not a duplicate of any previously included evidence."
            ),
            llm_config=llm_config,
        )

        debate_eval_agent = ConversableAgent(
            name="debate_eval_agent",
            system_message=(
                "You are an extremely selective and rigorous debate coach and argument analyst. "
                "Your job is to strictly evaluate whether evidence meets the highest standards for inclusion as impact evidence supporting a specific advantage in policy debate. "
                "For each piece of evidence, meticulously scrutinize its:\n"
                "- Author qualifications (must be from recognized experts or authoritative sources)\n"
                "- Empirical basis (must be supported by concrete data and research)\n"
                f"- Relevance (must support the maximized impact claim for advantage {advantage_number}, and be at least plausibly connected to the established link chain; for extinction or similar extreme impacts, an indirect or plausible connection is sufficient)\n"
                "- Strategic value (must provide unique and compelling support for the advantage's impact, not just generic background)\n"
                f"- Specificity (must not duplicate or closely overlap with other selected evidence, and should ideally establish that the plan, as proposed, will cause the maximized impact described in advantage {advantage_number}; for extreme impacts, indirect or plausible connections are acceptable)\n"
                "- Magnitude (strongly prefer evidence that supports the largest, most significant impacts—such as extinction, nuclear war, existential risk, or catastrophic loss of value to life—even if those impacts are only indirectly or plausibly connected to the advantage's link chain)\n"
                "- Wording precision (must use exact terminology needed to establish the impact for the advantage)\n\n"
                "After evaluating the evidence, you must:\n"
                "1. IMMEDIATELY REJECT (mark as 'False' and/or ignore) any evidence that has already been marked as 'include_it' in previous iterations\n"
                "2. Reject any evidence that duplicates already selected cards\n"
                "3. Ensure terminology precisely matches what's needed for the advantage's impact chain\n"
                "4. Only approve evidence that meets ALL evaluation criteria and is strictly advantage- and plan-relevant for the maximized impact, but for extinction or similar extreme impacts, an indirect or plausible connection is sufficient\n\n"
                "IMPORTANT: Do NOT select or approve any evidence (by id, cite, or content) that has already appeared in the debate_case string above. You must only approve new, previously unused evidence. Carefully check the debate_case string for all previously included evidence and avoid any repetition. If a card has already appeared in the debate_case, do not include it and do not approve it again.\n"
                "Your goal is to ensure we have the highest quality, advantage- and plan-specific impact evidence, maximizing the magnitude of the impact while ensuring absolutely no duplicate or generic cards, and that all included evidence is retagged and recut with precise, policy debate-style markup. For extinction or similar extreme impacts, err on the side of inclusion even if the connection to the plan is indirect, as long as it is not wholly implausible or generic."
            ),
            llm_config=debate_eval_llm_config,
        )

        debate_search_agent = ConversableAgent(
            name="debate_search_agent",
            system_message=(
                "You are a helpful assistant that can search the debate evidence dataset for a given tag. Your query will retrieve a list of debate cards. "
                "IMPORTANT: Do NOT search for or return any evidence (by id, cite, or content) that has already appeared in the debate_case string above. You must only search for and return new, previously unused evidence. Carefully check the debate_case string for all previously included evidence and avoid any repetition. If a card has already appeared in the debate_case, do not search for it again and do not return it."
            ),
            llm_config=required_llm_config,
        )

        executor_agent = ConversableAgent(
            name="executor_agent",
            human_input_mode="NEVER",
            llm_config=llm_config,
        )

        register_function(
            search_debate_cards,
            caller=debate_search_agent,
            executor=executor_agent,
            description="Search the debate evidence dataset using natural language queries. Return a list of debate cards.",
        )

        from autogen import GroupChat

        iterations = 0

        allowed_transitions = {
            argument_evaluator: [debate_search_agent],
            debate_search_agent: [executor_agent],
            executor_agent: [argument_evaluator]
        } # Not being used but a good example

        def custom_speaker_selection_func(last_speaker: Agent, groupchat: GroupChat):
            nonlocal iterations
            messages = groupchat.messages

            if len(messages) <= 1:
                return argument_evaluator

            if last_speaker is debate_search_agent:
                return executor_agent

            if last_speaker is executor_agent:
                return debate_eval_agent
            
            if last_speaker is debate_eval_agent:
                if "include_it" in messages[-1]["content"]:
                    iterations += 1
                    print(f"iterations: {iterations}")
                    if iterations >= 1:
                        return None
                    else:
                        return debate_search_agent
                else:
                    return debate_search_agent

            if last_speaker is argument_evaluator:
                return debate_search_agent
            else:
                return "round_robin"

        group_chat = GroupChat(
            agents=[argument_evaluator, debate_search_agent, executor_agent, debate_eval_agent],
            messages=[],
            max_round=40,
            speaker_selection_method=custom_speaker_selection_func
        )

        group_chat_manager = GroupChatManager(
            groupchat=group_chat,
            llm_config=llm_config,
        )

        # Use the debate_case, which includes the debate topic, the plan, inherency, harm, advantage N, advantage N uniqueness evidence, and advantage N link evidence for context

        chat_result = argument_evaluator.initiate_chat(
            group_chat_manager,
            message=(
                f"{debate_case}\n\n"
                "Assume that the current year is 2022.\n"
                f"The inherency, harm, advantage {advantage_number}, advantage {advantage_number} uniqueness evidence, and advantage {advantage_number} link evidence have already been established above.\n"
                f"Find the best, most plan- and advantage-specific evidence of the maximized impact supporting advantage {advantage_number} articulated above. "
                "Strongly prefer evidence that supports the largest, most significant, and extreme impacts (such as nuclear war, extinction, existential risk, or catastrophic loss of value to life), even if those impacts are only indirectly or plausibly connected to the advantage's link chain. "
                "It is acceptable to include evidence of extreme impacts even if the connection to the plan is indirect, as long as it is not wholly implausible or generic. "
                f"Only consider evidence that supports the claim that the plan, if enacted, would cause the maximized impact described in advantage {advantage_number}, but for extinction or similar extreme impacts, an indirect or plausible connection is sufficient. "
                f"Reject any evidence that is generic, completely tangential, or wholly disconnected from the established link chain for advantage {advantage_number}. "
                "IMPORTANT: Do NOT select or search for any evidence (by id, cite, or content) that has already appeared in the debate_case string above. You must only select new, previously unused evidence. Carefully check the debate_case string for all previously included evidence and avoid any repetition. If a card has already appeared in the debate_case, do not include it and do not search for it again."
            ),
        )

        advantage_impact_raw_string = chat_result.chat_history[-1]["content"]
        advantage_impact_json = json.loads(advantage_impact_raw_string)
        card_json = advantage_impact_json["cards"][0]
        advantage_impact_id = card_json["id"]
        retagged_argument = card_json.get("retagged_argument_as_read_outloud_in_the_debate_round", "")

        advantage_impact_doc = get_document_by_id(advantage_impact_id)
        card_markup = str(advantage_impact_doc['markup'])

        # Append to debate_case using h2, div, and p tags
        debate_case += (
            f"\n<h2>Advantage {advantage_number} Impact</h2>"
            f"\n<div><p>{retagged_argument}</p></div>"
            f"\n<div><p>{card_markup}</p></div>"
        )
        return debate_case

In [ ]:
#debate_case = add_advantage_impact_to_case(debate_case, "1")

In [ ]:
#display(HTML(debate_case))

In [ ]:
# import json
# import ast

# advantage_2 = json.loads(json.dumps(ast.literal_eval(advantages[1])))
# advantage_2_title = advantage_2["title"]
# advantage_2_core_argument = advantage_2["core_argument"]

# # Append advantage 2 title and core argument to debate_case using h2, div, and p tags
# debate_case += (
#     f"\n<h2>Advantage 2: {advantage_2_title}</h2>"
#     f"\n<div><p>{advantage_2_core_argument}</p></div>"
# )

In [ ]:
# debate_case = add_advantage_uniqueness_to_case(debate_case, "2")
# debate_case = add_advantage_link_to_case(debate_case, "2")
# debate_case = add_advantage_impact_to_case(debate_case, "2")

In [ ]:
#display(HTML(debate_case))

In [ ]:
# import json
# import ast

# advantage_3 = json.loads(json.dumps(ast.literal_eval(advantages[2])))
# advantage_3_title = advantage_3["title"]
# advantage_3_core_argument = advantage_3["core_argument"]

# # Append advantage 3 title and core argument to debate_case using h2, div, and p tags
# debate_case += (
#     f"\n<h2>Advantage 3: {advantage_3_title}</h2>"
#     f"\n<div><p>{advantage_3_core_argument}</p></div>"
# )

In [ ]:
# debate_case = add_advantage_uniqueness_to_case(debate_case, "3")
# debate_case = add_advantage_link_to_case(debate_case, "3")
# debate_case = add_advantage_impact_to_case(debate_case, "3")

In [ ]:
#display(HTML(debate_case))

## Solvency Workflow

In [ ]:
def add_solvency_card_to_case(debate_case):
    """
    Runs the agent group chat to find the best solvency card, and appends the retagged argument and card markup to the debate_case string.
    Returns the updated debate_case.
    """
    # --- Agent and config setup (relaxed recency requirements) ---
    class DebateCard(BaseModel):
        id: int
        cite: str
        include_in_case: Literal["include_it", "False"]
        reason_to_include: str
        retagged_argument_as_read_outloud_in_the_debate_round: str

    class DebateCardSearchResult(BaseModel):
        cards: List[DebateCard] = Field(..., min_items=1, max_items=1)

    llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        parallel_tool_calls=None
    )
    required_llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        tool_choice="required",
        temperature=1.0,
        top_p=0.9,
        parallel_tool_calls=None
    )
    debate_eval_llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        response_format=DebateCardSearchResult,
        parallel_tool_calls=None
    )

    # Relaxed recency requirements in system messages
    argument_evaluator = ConversableAgent(
        name="argument_evaluator", 
        system_message=(
            "You are an expert policy debater focused on finding the best possible evidence of plan solvency to support a specific plan and its articulated harm. "
            "Your job is to:\n"
            "1. Break down the plan and the chosen harm into the exact mechanisms, actions, and causal processes by which the plan would solve or significantly reduce the harm.\n"
            "2. Guide evidence collection by:\n"
            "   - Formulating extremely precise search queries that target only evidence which directly and specifically demonstrates that the plan, if implemented, would solve or substantially mitigate the articulated harm (i.e., solvency).\n"
            "   - Using BM25 search to find relevant cards from a debate evidence database.\n"
            "   - If you are being called after previous searches, you must significantly modify and refine your BM25 search queries to maximize the chance of finding new, more relevant, or more specific evidence. Do not simply repeat or slightly alter previous queries—make substantial changes to your search approach, keywords, or focus.\n"
            "   - Suggest query refinements to maximize the chance of finding evidence that directly supports the solvency claim for the plan and harm.\n"
            "3. Evaluate evidence quality for:\n"
            "   - Direct, explicit support for the solvency claim (evidence must not merely be tangentially related or generic background).\n"
            "   - Specificity: The evidence must establish that the plan, as proposed, will solve or significantly reduce the harm, and explain the mechanism by which this occurs.\n"
            "   - Empirical support and authoritativeness.\n"
            "Reject any evidence that does not fully and directly support the solvency of the plan for the articulated harm or that could be interpreted as generic or non-specific. "
            "Your goal is to find the strictest, most plan- and harm-relevant solvency evidence possible, ensuring that each selected card is unique and not a duplicate of any previously included evidence."
            "\n\nIMPORTANT: Do NOT select or search for any evidence that has already appeared in the current debate case. If a card or evidence is already present in the debate_case, it must not be included or searched for again."
        ),
        llm_config=llm_config,
    )

    debate_eval_agent = ConversableAgent(
        name="debate_eval_agent",
        system_message=(
            "You are an extremely selective and rigorous debate coach and argument analyst. "
            "Your job is to strictly evaluate whether evidence meets the highest standards for inclusion as solvency evidence supporting a specific plan and harm in policy debate. "
            "For each piece of evidence, meticulously scrutinize its:\n"
            "- Author qualifications (must be from recognized experts or authoritative sources)\n"
            "- Empirical basis (must be supported by concrete data and research)\n"
            "- Direct relevance (must precisely and explicitly support the plan's solvency claim for the articulated harm)\n"
            "- Strategic value (must provide unique and compelling support for the plan's ability to solve the harm, not just generic background)\n"
            "- Specificity (must not duplicate or closely overlap with other selected evidence, and must establish that the plan, as proposed, will solve or significantly reduce the harm)\n"
            "- Wording precision (must use exact terminology needed to establish the solvency link for the plan and harm)\n\n"
            "After evaluating the evidence, you must:\n"
            "1. IMMEDIATELY REJECT (mark as 'False' and/or ignore) any evidence that has already been marked as 'include_it' in previous iterations\n"
            "2. Reject any evidence that duplicates already selected cards\n"
            "3. Ensure terminology precisely matches what's needed for the plan's solvency link chains\n"
            "4. Only approve evidence that meets ALL evaluation criteria and is strictly plan- and harm-relevant for solvency\n"
            "5. IMPORTANT: Do NOT approve or allow any evidence that has already appeared in the current debate case. If a card or evidence is already present in the debate_case, it must not be included or searched for again.\n\n"
            "Your goal is to ensure we have the highest quality, plan- and harm-specific solvency evidence, with absolutely no duplicate or generic cards, and that all included evidence is retagged and recut with precise, policy debate-style markup."
        ),
        llm_config=debate_eval_llm_config,
    )

    debate_search_agent = ConversableAgent(
        name="debate_search_agent",
        system_message=(
            "You are a helpful assistant that can search the debate evidence dataset for a given tag. Your query will retrieve a list of debate cards. "
            "IMPORTANT: Do NOT search for or return any evidence that has already appeared in the current debate case. If a card or evidence is already present in the debate_case, it must not be included or searched for again."
        ),
        llm_config=required_llm_config,
    )

    executor_agent = ConversableAgent(
        name="executor_agent",
        human_input_mode="NEVER",
        llm_config=llm_config,
    )

    register_function(
        search_debate_cards,
        caller=debate_search_agent,
        executor=executor_agent,
        description="Search the debate evidence dataset using natural language queries. Return a list of debate cards.",
    )

    from autogen import GroupChat

    iterations = 0

    def custom_speaker_selection_func(last_speaker: Agent, groupchat: GroupChat):
        global iterations
        messages = groupchat.messages

        if len(messages) <= 1:
            return argument_evaluator

        if last_speaker is debate_search_agent:
            return executor_agent

        if last_speaker is executor_agent:
            return debate_eval_agent
        
        if last_speaker is debate_eval_agent:
            if "include_it" in messages[-1]["content"]:
                iterations += 1
                if iterations >= 3:
                    return None
                else:
                    return debate_search_agent
            else:
                return debate_search_agent

        if last_speaker is argument_evaluator:
            return debate_search_agent
        else:
            return "round_robin"

    group_chat = GroupChat(
        agents=[argument_evaluator, debate_search_agent, executor_agent, debate_eval_agent],
        messages=[],
        max_round=40,
        speaker_selection_method=custom_speaker_selection_func
    )

    group_chat_manager = GroupChatManager(
        groupchat=group_chat,
        llm_config=llm_config,
    )

    # Prompt: recency requirement removed, and now also instructs not to select or search for evidence already in the debate_case
    chat_result = argument_evaluator.initiate_chat(
        group_chat_manager,
        message=(
            f"{debate_case}\n\n"
            "Find the best, most plan- and harm-specific evidence of solvency supporting the plan and the articulated harm above. "
            "Only consider evidence that directly and specifically supports the claim that the plan, if implemented, will solve or substantially reduce the harm (i.e., solvency). "
            "Reject any evidence that is generic, tangential, not relevant to the plan's solvency for the articulated harm. "
            "IMPORTANT: Do NOT select, search for, or include any evidence that has already appeared in the current debate case. If a card or evidence is already present in the debate_case, it must not be included or searched for again."
        ),
    )

    import json
    solvency_output_raw_string = chat_result.chat_history[-1]["content"]
    card_data = json.loads(solvency_output_raw_string)["cards"][0]
    solvency_output_id = card_data["id"]
    retagged_argument = card_data["retagged_argument_as_read_outloud_in_the_debate_round"]

    doc = get_document_by_id(solvency_output_id)

    # Append to debate_case using h2, div, and <p> tags
    debate_case += (
        f"\n<h2>Solvency Argument</h2>"
        f"\n<div><p>{retagged_argument}</p></div>"
        f"\n<h2>Solvency Card</h2>"
        f"\n<div>{doc['markup']}</div>"
    )

    return debate_case


In [ ]:
#debate_case = add_solvency_card_to_case(debate_case)

In [ ]:
#display(HTML(debate_case))

# 1AC

In [ ]:
import ast

def try_n_times(func, *args, n=3, **kwargs):
    last_exception = None
    for attempt in range(n):
        try:
            return func(*args, **kwargs)
        except Exception as e:
            last_exception = e
            if attempt == n - 1:
                raise
    if last_exception:
        raise last_exception

plantext_output = try_n_times(generate_plantext_for_topic, debate_topic)
debate_case = try_n_times(generate_plan_with_harm_evidence, plantext_output, debate_topic)
debate_case = try_n_times(append_inherency_argument_and_evidence, debate_case)
advantages = try_n_times(generate_advantages, debate_case)

for i in range(3):
    # Try to parse the advantage up to 3 times
    for attempt in range(3):
        try:
            advantage = json.loads(json.dumps(ast.literal_eval(advantages[i])))
            break
        except Exception as e:
            if attempt == 2:
                raise
    advantage_title = advantage["title"]
    advantage_core_argument = advantage["core_argument"]

    # Append advantage title and core argument to debate_case using h2, div, and p tags
    debate_case += (
        f"\n<h2>Advantage {i+1}: {advantage_title}</h2>"
        f"\n<div><p>{advantage_core_argument}</p></div>"
    )

    debate_case = try_n_times(add_advantage_uniqueness_to_case, debate_case, str(i+1))
    debate_case = try_n_times(add_advantage_link_to_case, debate_case, str(i+1))
    debate_case = try_n_times(add_advantage_internal_link_to_case, debate_case, str(i+1))
    debate_case = try_n_times(add_advantage_impact_to_case, debate_case, str(i+1))

debate_case = try_n_times(add_solvency_card_to_case, debate_case)

In [ ]:
display(HTML(debate_case))

In [ ]:
debate_case

## Cross Examination (of the 1AC)

In [ ]:
def simulate_1nc_cross_examination(debate_case: str) -> str:
    from typing import List
    from pydantic import BaseModel, Field

    # Define the structure for a cross-examination question and answer
    class CrossExQuestion(BaseModel):
        question: str

    class CrossExAnswer(BaseModel):
        answer: str

    class CrossExPair(BaseModel):
        negative_question: str
        affirmative_response: str

    class CrossExamination(BaseModel):
        cross_ex: List[CrossExPair] = Field(..., min_items=4, max_items=4)

    # LLM config for all agents
    llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        parallel_tool_calls=None
    )

    cross_ex_llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        response_format=CrossExamination,
        parallel_tool_calls=None
    )

    # Agent 1: Negative (1NC) asks questions
    negative_cross_ex_agent = ConversableAgent(
        name="negative_cross_ex_agent",
        system_message=(
            "You are the 1NC (negative) debater in a policy debate cross-examination. "
            "Your job is to ask sharp, strategic, and challenging questions about the 1AC (affirmative case) just presented. "
            "Focus on exposing weaknesses, ambiguities, or assumptions in the plan, inherency, harms, advantages, and solvency. "
            "Ask one question at a time, and wait for the affirmative to answer before asking the next. "
            "Do not answer your own questions. "
            "Be concise and direct. "
            "Do not repeat questions. "
            "You will ask a total of 3 to 7 questions."
        ),
        llm_config=llm_config,
    )

    # Agent 2: Affirmative answers
    affirmative_cross_ex_agent = ConversableAgent(
        name="affirmative_cross_ex_agent",
        system_message=(
            "You are the 1AC (affirmative) debater being cross-examined by the 1NC (negative) in a policy debate. "
            "Your job is to answer each question as clearly, persuasively, and strategically as possible, defending the affirmative case. "
            "Respond directly to the negative's question, but do not volunteer extra information. "
            "Be concise and avoid rambling. "
            "Do not ask questions yourself."
        ),
        llm_config=llm_config,
    )

    # Agent 3: Cross-ex summary agent (outputs the structured Q&A)
    cross_ex_summary_agent = ConversableAgent(
        name="cross_ex_summary_agent",
        system_message=(
            "You are a debate judge summarizing the 1NC cross-examination of the 1AC. "
            "Your job is to produce a structured list of question/answer pairs, each with a 'negative_question' and an 'affirmative_response', "
            "covering the full cross-examination as it occurred. "
            "Return the result as a list of 3 to 7 question/answer pairs, each clearly labeled."
        ),
        llm_config=cross_ex_llm_config,
    )

    from autogen import GroupChat

    cross_ex_iterations = 0  # Track the number of Q&A iterations (outside the function)

    def cross_ex_speaker_selection(last_speaker, groupchat):
        nonlocal cross_ex_iterations
        # Alternate between negative and affirmative, then finish with summary agent
        if cross_ex_iterations == 0 and last_speaker is None:
            return negative_cross_ex_agent
        if last_speaker is negative_cross_ex_agent:
            return affirmative_cross_ex_agent
        if last_speaker is affirmative_cross_ex_agent:
            cross_ex_iterations += 1
            if cross_ex_iterations >= 4:
                return cross_ex_summary_agent
            else:
                return negative_cross_ex_agent
        if last_speaker is cross_ex_summary_agent:
            return None
        return "round_robin"

    group_chat = GroupChat(
        agents=[negative_cross_ex_agent, affirmative_cross_ex_agent, cross_ex_summary_agent],
        messages=[],
        max_round=40,
        speaker_selection_method=cross_ex_speaker_selection
    )

    group_chat_manager = GroupChatManager(
        groupchat=group_chat,
        llm_config=llm_config,
    )

    # The context for the cross-examination is the debate_case (the 1AC)
    chat_result = affirmative_cross_ex_agent.initiate_chat(
        group_chat_manager,
        message=(
            f"You are about to begin the 1NC cross-examination of the 1AC. The 1AC is as follows:\n\n"
            f"{debate_case}\n\n"
            "Begin by asking your first question."
        ),
    )

    # The summary agent's output is the last message in the chat history
    cross_ex_json = chat_result.chat_history[-1]["content"]
    cross_ex_data = json.loads(cross_ex_json)
    cross_ex_pairs = cross_ex_data["cross_ex"]

    # Format as HTML for display
    html = "<h2>1NC Cross-Examination of the 1AC</h2>\n"
    for i, pair in enumerate(cross_ex_pairs, 1):
        html += f"<div><b>Negative Question {i}:</b> {pair['negative_question']}</div>\n"
        html += f"<div><b>Affirmative Response {i}:</b> {pair['affirmative_response']}</div>\n"
        html += "<br/>\n"

    return html

# For compatibility with the rest of the code, assign to 1ac_crossex_html
# Usage: 1ac_crossex_html = simulate_1nc_cross_examination(debate_case)

In [ ]:
max_attempts = 3
for attempt in range(1, max_attempts + 1):
    try:
        first_cx_transcript = simulate_1nc_cross_examination(debate_case)
        break
    except Exception as e:
        if attempt == max_attempts:
            raise

In [ ]:
debate_case = debate_case + first_cx_transcript

In [ ]:
display(HTML(debate_case))

#  Negative Workflows

In [ ]:
def generate_negative_offcase(affirmative_case: str):

    from typing import List, Optional
    from pydantic import BaseModel, Field

    class Topicality(BaseModel):
        title: str
        core_argument_summary_as_spoken_outloud_in_debate_round: str

    class Theory(BaseModel):
        """
        Debate theory argument (e.g., conditionality, severance, intrinsicness, etc.),
        not a general philosophical theory. This should be a procedural or theoretical
        objection to the structure or practices of the affirmative or negative, such as
        conditionality, multiple conditional counterplans, plan-inclusive counterplans, etc.
        All theory arguments must be designed to help the negative win the debate round and
        should directly oppose or undermine the plan or the affirmative's advocacy.
        """
        title: str
        core_argument_summary_as_spoken_outloud_in_debate_round: str

    class Disadvantage(BaseModel):
        title: str
        core_argument_summary_as_spoken_outloud_in_debate_round: str

    class Counterplan(BaseModel):
        title: str
        core_argument_summary_as_spoken_outloud_in_debate_round: str
        counterplan_text: str

    class Kritik(BaseModel):
        """
        Debate kritik: a negative position that is often philosophical or ethical in nature,
        challenging the underlying assumptions, frameworks, or ideologies of the affirmative case.
        Kritiks should include a clear alternative text, which is a formalized position (like a counterplan text)
        that usually negates or rejects the kinds of thinking or assumptions made by the affirmative.
        The alternative text should be a specific, formal statement of what the negative advocates instead.
        Kritiks must be distinct from disadvantages and counterplans.
        All kritiks must be designed to negate or oppose the plan and help the negative win the round.
        """
        title: str
        core_argument_summary_as_spoken_outloud_in_debate_round: str
        alternative_text: str

    class NegativePositions(BaseModel):
        topicality: Topicality
        theory: Theory
        # topicality: Optional[Topicality]
        # theory: Optional[Theory]
        disadvantages: List[Disadvantage] = Field(..., min_items=1, max_items=1)
        counterplans: List[Counterplan] = Field(..., min_items=1, max_items=1)
        kritiks: List[Kritik] = Field(..., min_items=1, max_items=1)
        rationale: str
        advice_for_next_search: str

    llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        parallel_tool_calls=None,
        temperature=2.0,
        top_p=0.8
    )
    required_llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        tool_choice="required",
        parallel_tool_calls=None,
        temperature=2.0,
        top_p=0.8
    )
    neg_eval_llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        response_format=NegativePositions,
        parallel_tool_calls=None,
        temperature=2.0,
        top_p=0.8
    )

    neg_generator = ConversableAgent(
        name="neg_generator",
        system_message=(
            "You are a policy debate expert tasked with generating a set of off-case negative positions for a given affirmative case. "
            "All positions you generate must be designed to negate and oppose the plan, and to help the negative win the debate round. "
            "You must review the affirmative case and generate: "
            "up to 1 topicality violation (only if it truly makes sense to do so), up to 1 theory argument (only if it truly makes sense to do so; theory means debate theory such as conditionality, severance, intrinsicness, etc.), at least 1 disadvantage, at least 1 counterplan (with counterplan text), and at least 1 kritik (with alternative text). "
            "Kritiks are debate kritiks—positions that are often philosophical or ethical in nature, challenging the underlying assumptions, frameworks, or ideologies of the affirmative case. Each kritik must include a clear alternative text, which is a formalized position (like a counterplan text) that usually negates or rejects the kinds of thinking or assumptions made by the affirmative. "
            "Only include a topicality or theory argument if there is a clear, specific, and strategic reason to do so based on the content of the affirmative case, and only if it helps the negative win the round. "
            "Each position should be clearly articulated, specific to the affirmative case, and represent a classic or creative negative strategy. "
            "Disadvantages, counterplans, and kritiks should be distinct and non-redundant. "
            "The counterplan must include a counterplan text. The kritik must include an alternative text. "
            "Strive for maximum diversity and uniqueness in the types of positions you generate, drawing on both classic policy debate categories and creative, well-supported arguments. "
            "You may include progressive or novel arguments, but should also include traditional policy arguments. "
            "All arguments, including theory and topicality, must be written to help the negative win the debate round and must directly oppose or undermine the plan or the affirmative's advocacy. "
            "IMPORTANT: For topicality and theory, the 'core_argument_summary_as_spoken_outloud_in_debate_round' should be written as a natural language explanation of the argument, as you would say it out loud in a debate round, not as a list of 'interpretation, violation, reasons to prefer' or similar debate jargon. Instead, explain the argument in a way that would make sense to a layperson, focusing on the substance and reasoning behind the objection, as if you were speaking it in a debate round."
        ),
        llm_config=llm_config,
    )

    debate_search_agent = ConversableAgent(
        name="debate_search_agent",
        system_message="You are a helpful assistant that can search the debate evidence dataset for a given tag or query. Your query will retrieve a list of debate cards.",
        llm_config=required_llm_config,
    )

    executor_agent = ConversableAgent(
        name="executor_agent",
        human_input_mode="NEVER",
        llm_config=llm_config,
    )

    neg_reviewer = ConversableAgent(
        name="neg_reviewer",
        system_message=(
            "You are a highly rigorous debate coach. Your job is to review the current set of negative positions, the rationale, and the evidence gathered so far. "
            "For each position, assess whether it is likely to be well-supported, strategic, and distinct. "
            "Give advice for the next search iteration for how to search for evidence to improve the negative positions given the current positions, evidence, and rationale. "
            "After each search, update your positions and provide rationale and advice for the next search. "
            "You may suggest rewording, combining, or splitting positions as needed to maximize clarity and support. "
            "Encourage the generation of highly unique and diverse negative positions, including topicality, theory (debate theory such as conditionality, severance, etc.), disadvantages, counterplans, and kritiks (which are debate kritiks—often philosophical or ethical arguments challenging the assumptions or frameworks of the affirmative). "
            "For kritiks, ensure the alternative is provided as an 'alternative text'—a formalized position (like a counterplan text) that negates or rejects the assumptions or frameworks of the affirmative. "
            "All positions and arguments must be written to help the negative win the debate round and must directly oppose or undermine the plan or the affirmative's advocacy. "
            "IMPORTANT: For topicality and theory, the 'core_argument_summary_as_spoken_outloud_in_debate_round' should be a natural language explanation of the argument, as you would say it out loud in a debate round, not a list of 'interpretation, violation, reasons to prefer' or similar debate jargon. The explanation should focus on the substance and reasoning behind the objection, as if explaining to a layperson and as if you were speaking it in a debate round."
        ),
        llm_config=neg_eval_llm_config,
    )

    register_function(
        search_debate_cards,
        caller=debate_search_agent,
        executor=executor_agent,
        description="Search the debate evidence dataset using natural language queries. Return a list of debate cards.",
    )

    from autogen import GroupChat

    iterations = 0
    MAX_ITERATIONS = 3

    def custom_speaker_selection_func(last_speaker: Agent, groupchat: GroupChat):
        nonlocal iterations
        messages = groupchat.messages

        if len(messages) == 0:
            return neg_generator

        if last_speaker is neg_generator:
            return debate_search_agent

        if last_speaker is debate_search_agent:
            return executor_agent

        if last_speaker is executor_agent:
            return neg_reviewer

        if last_speaker is neg_reviewer:
            iterations += 1
            try:
                content = messages[-1]["content"]
                if isinstance(content, dict):
                    plan_ready = content.get("plan_ready", "False")
                else:
                    import json
                    plan_ready = json.loads(content).get("plan_ready", "False")
            except Exception:
                plan_ready = "False"
            if iterations >= MAX_ITERATIONS:
                return None
            else:
                return debate_search_agent

        return "round_robin"

    group_chat = GroupChat(
        agents=[neg_generator, debate_search_agent, executor_agent, neg_reviewer],
        messages=[],
        max_round=50,
        speaker_selection_method=custom_speaker_selection_func
    )

    group_chat_manager = GroupChatManager(
        groupchat=group_chat,
        llm_config=llm_config,
    )

    neg_prompt = (
        f"Given the following affirmative debate case, generate a set of negative off-case positions. "
        f"All positions must be designed to negate and oppose the plan, and to help the negative win the debate round. "
        f"Only include a topicality violation or a theory argument if it truly makes sense to do so based on the content of the affirmative case, and only if it helps the negative win the round. "
        f"(Theory means debate theory such as conditionality, severance, intrinsicness, etc. — not general philosophical theory.) "
        f"At least one of each: disadvantage, counterplan (with counterplan text), and kritik (with alternative text) must be included. "
        f"Kritiks are debate kritiks—positions that are often philosophical or ethical in nature, challenging the underlying assumptions, frameworks, or ideologies of the affirmative case. Each kritik must include a clear alternative text, which is a formalized position (like a counterplan text) that usually negates or rejects the kinds of thinking or assumptions made by the affirmative. "
        f"Each position should be clearly articulated, specific to the affirmative case, and represent a classic or creative negative strategy. "
        f"{affirmative_case}\n"
        "Strive for maximum diversity and uniqueness in the types of positions you generate, drawing on both classic policy debate categories and creative, well-supported arguments. "
        "You may include progressive or novel arguments, but should also include traditional policy arguments. "
        "All arguments, including theory and topicality, must be written to help the negative win the debate round and must directly oppose or undermine the plan or the affirmative's advocacy. "
        "IMPORTANT: For topicality and theory, the 'core_argument_summary_as_spoken_outloud_in_debate_round' should be a natural language explanation of the argument, as you would say it out loud in a debate round, not a list of 'interpretation, violation, reasons to prefer' or similar debate jargon. Instead, explain the argument in a way that would make sense to a layperson, focusing on the substance and reasoning behind the objection, as if you were speaking it in a debate round."
    )

    chat_result = neg_generator.initiate_chat(
        group_chat_manager,
        message=neg_prompt,
    )


    neg_output_raw_string = chat_result.chat_history[-1]["content"]
    neg_positions = json.loads(neg_output_raw_string)

    return neg_positions


In [ ]:
#negative_case = generate_negative_offcase(debate_case)

In [ ]:
# topicality_title = negative_case['topicality']['title']
# topicality_core_argument = negative_case['topicality']['core_argument']
# negative_case_html = f"<h2>{topicality_title}</h2>\n<p>{topicality_core_argument}</p>"
# negative_case_html


## Topicality

### Topicality Interpretation

In [ ]:
def add_topicality_interpretation_and_evidence(debate_case, negative_case_html):
    """
    Given an affirmative debate_case string and a negative_case_html string,
    finds the best formalized topicality interpretation and corresponding evidence to support it,
    and appends both to the negative_case_html using h2, div, and p tags.
    Returns the updated negative_case_html string.
    """
    class DebateCard(BaseModel):
        id: int
        cite: str
        include_in_case: Literal["include_it", "False"]
        reason_to_include: str
        formalized_topicality_interpretation: str  # Formalized topicality interpretation text

    class DebateCardSearchResult(BaseModel):
        cards: List[DebateCard] = Field(..., min_items=1, max_items=1)

    llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        parallel_tool_calls=None
    )
    required_llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        tool_choice="required",
        temperature=1.0,
        top_p=0.9,
        parallel_tool_calls=None
    )
    debate_eval_llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        response_format=DebateCardSearchResult,
        parallel_tool_calls=None
    )

    # Agent setup
    topicality_interpretation_agent = ConversableAgent(
        name="topicality_interpretation_agent", 
        system_message=(
            f"You are an expert policy debater and debate coach. "
            f"Your job is to:\n"
            f"1. Read the provided affirmative debate case and negative case HTML.\n"
            f"2. Formulate a formalized topicality interpretation (definition of a key word or phrase in the resolution or plan) that is strategic, precise, and would help the negative win the round. "
            f"3. Write the interpretation in a formal debate style, including the word/phrase being defined, the definition, and a brief standards/violation explanation. "
            f"4. Suggest the best possible search query to find evidence supporting this interpretation (e.g., legal, academic, or authoritative definitions or standards)."
        ),
        llm_config=llm_config,
    )

    topicality_eval_agent = ConversableAgent(
        name="topicality_eval_agent",
        system_message=(
            f"You are a highly rigorous debate coach and argument analyst. "
            f"Your job is to strictly evaluate whether evidence meets the highest standards for inclusion as support for a topicality interpretation in policy debate. "
            f"For each piece of evidence, meticulously scrutinize its:\n"
            f"- Author qualifications (must be from recognized experts or authoritative sources)\n"
            f"- Empirical or doctrinal basis (must be supported by concrete data, legal precedent, or academic consensus)\n"
            f"- Direct relevance (must precisely and explicitly support the topicality interpretation)\n"
            f"- Strategic value (must provide unique and compelling support for the interpretation, not just generic background)\n"
            f"- Specificity (must not duplicate or closely overlap with other selected evidence, and must establish the definition or standard as interpreted)\n"
            f"- Wording precision (must use exact terminology needed to establish the interpretation)\n\n"
            f"After evaluating the evidence, you must:\n"
            f"1. IMMEDIATELY REJECT (mark as 'False' and/or ignore) any evidence that has already been marked as 'include_it' in previous iterations\n"
            f"2. Reject any evidence that duplicates already selected cards\n"
            f"3. Ensure terminology precisely matches what's needed for the topicality interpretation\n"
            f"4. Only approve evidence that meets ALL evaluation criteria and is strictly interpretation-relevant\n\n"
            f"Your goal is to ensure we have the highest quality, interpretation-specific evidence, with absolutely no duplicate or generic cards, and that all included evidence is retagged and recut with precise, policy debate-style markup."
        ),
        llm_config=debate_eval_llm_config,
    )

    topicality_search_agent = ConversableAgent(
        name="topicality_search_agent",
        system_message="You are a helpful assistant that can search the debate evidence dataset for a given tag. Your query will retrieve a list of debate cards.",
        llm_config=required_llm_config,
    )

    executor_agent = ConversableAgent(
        name="executor_agent",
        human_input_mode="NEVER",
        llm_config=llm_config,
    )

    register_function(
        search_debate_cards,
        caller=topicality_search_agent,
        executor=executor_agent,
        description="Search the debate evidence dataset using natural language queries. Return a list of debate cards.",
    )

    from autogen import GroupChat

    iterations = 0

    def custom_speaker_selection_func(last_speaker: Agent, groupchat: GroupChat):
        nonlocal iterations
        messages = groupchat.messages

        if len(messages) <= 1:
            return topicality_interpretation_agent

        if last_speaker is topicality_search_agent:
            return executor_agent

        if last_speaker is executor_agent:
            return topicality_eval_agent
        
        if last_speaker is topicality_eval_agent:
            if "include_it" in messages[-1]["content"]:
                iterations += 1
                if iterations >= 3:
                    return None
                else:
                    return topicality_search_agent
            else:
                return topicality_search_agent

        if last_speaker is topicality_interpretation_agent:
            return topicality_search_agent
        else:
            return "round_robin"

    group_chat = GroupChat(
        agents=[topicality_interpretation_agent, topicality_search_agent, executor_agent, topicality_eval_agent],
        messages=[],
        max_round=40,
        speaker_selection_method=custom_speaker_selection_func
    )

    group_chat_manager = GroupChatManager(
        groupchat=group_chat,
        llm_config=llm_config,
    )

    # Use the debate_case and negative_case_html for context
    chat_result = topicality_interpretation_agent.initiate_chat(
        group_chat_manager,
        message=(
            f"{debate_case}\n\n"
            f"{negative_case_html}\n\n"
            "Assume that the current year is 2022.\n"
            "Formulate a formalized topicality interpretation (definition of a key word or phrase in the resolution or plan) that is strategic, precise, and would help the negative win the round. "
            "Write the interpretation in a formal debate style, including the word/phrase being defined, the definition, and a brief standards/violation explanation. "
            "Then, find the best, most interpretation-specific evidence supporting this topicality interpretation. "
            "Only consider evidence that directly and specifically supports the interpretation. "
            "Reject any evidence that is generic, tangential, or not relevant to the interpretation."
        ),
    )

    # Parse the result
    topicality_evidence_raw_string = chat_result.chat_history[-1]["content"]
    card_json = json.loads(topicality_evidence_raw_string)["cards"][0]
    topicality_evidence_id = card_json["id"]
    formalized_interpretation = card_json.get("formalized_topicality_interpretation", "")
    # Get the card document
    topicality_evidence_doc = get_document_by_id(topicality_evidence_id)
    card_markup = str(topicality_evidence_doc['markup'])

    # Append to negative_case_html using h2, div, and p tags
    negative_case_html += (
        f"\n<h2>Topicality Interpretation and Evidence</h2>"
        f"\n<div><p>{formalized_interpretation}</p></div>"
        f"\n<div><p>{card_markup}</p></div>"
    )
    return negative_case_html


In [ ]:
#negative_case_html = add_topicality_interpretation_and_evidence(debate_case, negative_case_html)

In [ ]:
#display(HTML(negative_case_html))

### Topicality Violation

In [ ]:
def add_topicality_violation(debate_case, negative_case_html):
    """
    Given an affirmative debate_case string and a negative_case_html string (which already includes the topicality interpretation),
    finds the best formalized topicality violation (explaining how the affirmative violates the interpretation) and appends it
    to the negative_case_html using h2, div, and p tags.
    Returns the updated negative_case_html string.
    """
    class TopicalityViolationCard(BaseModel):
        id: int
        cite: str
        include_in_case: Literal["include_it", "False"]
        reason_to_include: str
        formalized_topicality_violation: str  # Formalized topicality violation text

    class TopicalityViolationSearchResult(BaseModel):
        cards: List[TopicalityViolationCard] = Field(..., min_items=1, max_items=1)

    llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        parallel_tool_calls=None
    )
    required_llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        tool_choice="required",
        temperature=1.0,
        top_p=0.9,
        parallel_tool_calls=None
    )
    violation_eval_llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        response_format=TopicalityViolationSearchResult,
        parallel_tool_calls=None
    )

    # Agent setup
    topicality_violation_agent = ConversableAgent(
        name="topicality_violation_agent", 
        system_message=(
            f"You are an expert policy debater and debate coach. "
            f"Your job is to:\n"
            f"1. Read the provided affirmative debate case and negative case HTML (which already contains the topicality interpretation).\n"
            f"2. Formulate a formalized topicality violation, explaining exactly how the affirmative's plan or advocacy violates the negative's interpretation. "
            f"3. Write the violation in a formal debate style, including a clear explanation of the violation and a brief reference to the standards/impacts. "
            f"4. Suggest the best possible search query to find evidence or examples supporting this violation (e.g., plan text, advocacy statements, or authoritative analysis showing the violation)."
        ),
        llm_config=llm_config,
    )

    topicality_violation_eval_agent = ConversableAgent(
        name="topicality_violation_eval_agent",
        system_message=(
            f"You are a highly rigorous debate coach and argument analyst. "
            f"Your job is to strictly evaluate whether the violation explanation and any supporting evidence meet the highest standards for inclusion as support for a topicality violation in policy debate. "
            f"For each piece of evidence or explanation, meticulously scrutinize its:\n"
            f"- Direct relevance (must precisely and explicitly show how the affirmative violates the interpretation)\n"
            f"- Strategic value (must provide unique and compelling support for the violation, not just generic background)\n"
            f"- Specificity (must not duplicate or closely overlap with other selected evidence, and must establish the violation as interpreted)\n"
            f"- Wording precision (must use exact terminology needed to establish the violation)\n\n"
            f"After evaluating, you must:\n"
            f"1. IMMEDIATELY REJECT (mark as 'False' and/or ignore) any evidence that has already been marked as 'include_it' in previous iterations\n"
            f"2. Reject any evidence that duplicates already selected cards\n"
            f"3. Ensure terminology precisely matches what's needed for the topicality violation\n"
            f"4. Only approve evidence/explanations that meet ALL evaluation criteria and are strictly violation-relevant\n\n"
            f"Your goal is to ensure we have the highest quality, interpretation-specific violation explanation, with absolutely no duplicate or generic cards, and that all included evidence is retagged and recut with precise, policy debate-style markup."
        ),
        llm_config=violation_eval_llm_config,
    )

    topicality_violation_search_agent = ConversableAgent(
        name="topicality_violation_search_agent",
        system_message="You are a helpful assistant that can search the debate evidence dataset for a given tag. Your query will retrieve a list of debate cards.",
        llm_config=required_llm_config,
    )

    executor_agent = ConversableAgent(
        name="executor_agent",
        human_input_mode="NEVER",
        llm_config=llm_config,
    )

    register_function(
        search_debate_cards,
        caller=topicality_violation_search_agent,
        executor=executor_agent,
        description="Search the debate evidence dataset using natural language queries. Return a list of debate cards.",
    )

    from autogen import GroupChat

    iterations = 0

    def custom_speaker_selection_func(last_speaker: Agent, groupchat: GroupChat):
        nonlocal iterations
        messages = groupchat.messages

        if len(messages) <= 1:
            return topicality_violation_agent

        if last_speaker is topicality_violation_search_agent:
            return executor_agent

        if last_speaker is executor_agent:
            return topicality_violation_eval_agent
        
        if last_speaker is topicality_violation_eval_agent:
            if "include_it" in messages[-1]["content"]:
                iterations += 1
                if iterations >= 3:
                    return None
                else:
                    return topicality_violation_search_agent
            else:
                return topicality_violation_search_agent

        if last_speaker is topicality_violation_agent:
            return topicality_violation_search_agent
        else:
            return "round_robin"

    group_chat = GroupChat(
        agents=[topicality_violation_agent, topicality_violation_search_agent, executor_agent, topicality_violation_eval_agent],
        messages=[],
        max_round=40,
        speaker_selection_method=custom_speaker_selection_func
    )

    group_chat_manager = GroupChatManager(
        groupchat=group_chat,
        llm_config=llm_config,
    )

    # Use the debate_case and negative_case_html for context
    chat_result = topicality_violation_agent.initiate_chat(
        group_chat_manager,
        message=(
            f"{debate_case}\n\n"
            f"{negative_case_html}\n\n"
            "Assume that the current year is 2022.\n"
            "Formulate a formalized topicality violation, explaining exactly how the affirmative's plan or advocacy violates the negative's interpretation. "
            "Write the violation in a formal debate style, including a clear explanation of the violation and a brief reference to the standards/impacts. "
            "Then, find the best, most interpretation-specific evidence or example supporting this topicality violation. "
            "Only consider evidence or examples that directly and specifically show the violation. "
            "Reject any evidence that is generic, tangential, or not relevant to the violation."
        ),
    )

    # Parse the result
    topicality_violation_raw_string = chat_result.chat_history[-1]["content"]
    card_json = json.loads(topicality_violation_raw_string)["cards"][0]
    topicality_violation_id = card_json["id"]
    formalized_violation = card_json.get("formalized_topicality_violation", "")
    # Get the card document
    topicality_violation_doc = get_document_by_id(topicality_violation_id)
    card_markup = str(topicality_violation_doc['markup'])

    # Append to negative_case_html using h2 and div, p tags (but do NOT include the evidence/card_markup)
    negative_case_html += (
        f"\n<h2>Topicality Violation</h2>"
        f"\n<div><p>{formalized_violation}</p></div>"
    )
    return negative_case_html


In [ ]:
#negative_case_html = add_topicality_violation(debate_case, negative_case_html)

In [ ]:
#display(HTML(negative_case_html))

### Topicality Reasons to Prefer

In [ ]:
def add_topicality_reasons_to_prefer_and_evidence(debate_case, negative_case_html):
    """
    Given an affirmative debate_case string and a negative_case_html string (which already contains the formal interpretation and violation),
    generates three long, detailed, formalized "reasons to prefer" the negative's interpretation (using classic policy debate topicality terminology such as 'ground', 'competing interpretations', 'education', etc),
    and finds a piece of evidence supporting these reasons. Appends only the reasons to the negative_case_html using h2, div, and p tags.
    Returns the updated negative_case_html string.
    The reasons to prefer should NOT reference or mention the evidence that is gathered.
    """
    from typing import List, Literal
    from pydantic import BaseModel, Field

    class TopicalityReasonCard(BaseModel):
        id: int
        cite: str
        include_in_case: Literal["include_it", "False"]
        reason_to_include: str
        detailed_reasons_to_prefer_arguments_as_delivered_in_debate: List[str] = Field(
            ..., min_items=3, max_items=3
        )  # 3 long, detailed, debate-style reasons using classic topicality terms

    class TopicalityReasonCardSearchResult(BaseModel):
        cards: List[TopicalityReasonCard] = Field(..., min_items=1, max_items=1)

    llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        parallel_tool_calls=None
    )
    required_llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        tool_choice="required",
        temperature=1.0,
        top_p=0.9,
        parallel_tool_calls=None
    )
    debate_eval_llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        response_format=TopicalityReasonCardSearchResult,
        parallel_tool_calls=None
    )

    # Agent setup
    topicality_reasons_agent = ConversableAgent(
        name="topicality_reasons_agent", 
        system_message=(
            f"You are an expert policy debater and debate coach. "
            f"Your job is to:\n"
            f"1. Read the provided affirmative debate case and negative case HTML (which already contains the formal interpretation and violation).\n"
            f"2. Generate exactly three distinct, formalized 'reasons to prefer' the negative's interpretation, using classic policy debate topicality terminology such as 'competing interpretations', 'education', 'ground', 'predictability', 'limits', 'fairness', 'reasonability', etc. "
            f"Each reason should be long, detailed, strategic, and written in a formal debate style, thoroughly explaining the rationale, impact, and strategic value of the reason in the context of policy debate theory. Each reason should be at least 5-7 sentences and include specific examples, impacts, and theoretical warrants. These should be the kinds of arguments a debater would actually deliver out loud in a round, using the above topicality terms as the core of each reason.\n"
            f"Do NOT reference or mention any evidence or cards in the reasons to prefer. The reasons should stand alone as arguments, without referring to any evidence that may be gathered later.\n"
            f"3. Suggest the best possible search query to find evidence supporting these reasons to prefer, using debate topicality terminology (e.g., 'competing interpretations standard', 'education standard topicality', 'ground limits topicality', etc)."
        ),
        llm_config=llm_config,
    )

    topicality_eval_agent = ConversableAgent(
        name="topicality_eval_agent",
        system_message=(
            f"You are a highly rigorous debate coach and argument analyst. "
            f"Your job is to strictly evaluate whether evidence meets the highest standards for inclusion as support for reasons to prefer a topicality interpretation in policy debate. "
            f"For each piece of evidence, meticulously scrutinize its:\n"
            f"- Author qualifications (must be from recognized experts or authoritative sources)\n"
            f"- Empirical or doctrinal basis (must be supported by concrete data, legal precedent, or academic consensus)\n"
            f"- Direct relevance (must precisely and explicitly support the reasons to prefer, using debate topicality terminology)\n"
            f"- Strategic value (must provide unique and compelling support for the reasons, not just generic background)\n"
            f"- Specificity (must not duplicate or closely overlap with other selected evidence, and must establish the reason as interpreted)\n"
            f"- Wording precision (must use exact terminology needed to establish the reason)\n\n"
            f"After evaluating the evidence, you must:\n"
            f"1. IMMEDIATELY REJECT (mark as 'False' and/or ignore) any evidence that has already been marked as 'include_it' in previous iterations\n"
            f"2. Reject any evidence that duplicates already selected cards\n"
            f"3. Ensure terminology precisely matches what's needed for the reasons to prefer\n"
            f"4. Only approve evidence that meets ALL evaluation criteria and is strictly relevant to the reasons to prefer\n\n"
            f"Your goal is to ensure we have the highest quality, reason-to-prefer-specific evidence, with absolutely no duplicate or generic cards, and that all included evidence is retagged and recut with precise, policy debate-style markup."
        ),
        llm_config=debate_eval_llm_config,
    )

    topicality_search_agent = ConversableAgent(
        name="topicality_search_agent",
        system_message=(
            "You are a helpful assistant that can search the debate evidence dataset for a given tag. "
            "When searching for evidence to support reasons to prefer a topicality interpretation, "
            "use classic policy debate topicality terminology in your queries, such as 'competing interpretations', 'education', 'ground', 'predictability', 'limits', 'fairness', 'reasonability', etc. "
            "Your query will retrieve a list of debate cards."
        ),
        llm_config=required_llm_config,
    )

    executor_agent = ConversableAgent(
        name="executor_agent",
        human_input_mode="NEVER",
        llm_config=llm_config,
    )

    register_function(
        search_debate_cards,
        caller=topicality_search_agent,
        executor=executor_agent,
        description="Search the debate evidence dataset using natural language queries. Return a list of debate cards.",
    )

    from autogen import GroupChat

    iterations = 0

    def custom_speaker_selection_func(last_speaker: Agent, groupchat: GroupChat):
        nonlocal iterations
        messages = groupchat.messages

        if len(messages) <= 1:
            return topicality_reasons_agent

        if last_speaker is topicality_search_agent:
            return executor_agent

        if last_speaker is executor_agent:
            return topicality_eval_agent
        
        if last_speaker is topicality_eval_agent:
            if "include_it" in messages[-1]["content"]:
                iterations += 1
                if iterations >= 3:
                    return None
                else:
                    return topicality_search_agent
            else:
                return topicality_search_agent

        if last_speaker is topicality_reasons_agent:
            return topicality_search_agent
        else:
            return "round_robin"

    group_chat = GroupChat(
        agents=[topicality_reasons_agent, topicality_search_agent, executor_agent, topicality_eval_agent],
        messages=[],
        max_round=40,
        speaker_selection_method=custom_speaker_selection_func
    )

    group_chat_manager = GroupChatManager(
        groupchat=group_chat,
        llm_config=llm_config,
    )

    # Use the debate_case and negative_case_html for context
    chat_result = topicality_reasons_agent.initiate_chat(
        group_chat_manager,
        message=(
            f"{debate_case}\n\n"
            f"{negative_case_html}\n\n"
            "Assume that the current year is 2022.\n"
            "Given the formal interpretation and violation already included above, generate exactly three distinct, formalized 'reasons to prefer' the negative's interpretation, using classic policy debate topicality terminology such as 'competing interpretations', 'education', 'ground', 'predictability', 'limits', 'fairness', 'reasonability', etc. "
            "Each reason should be long, detailed, strategic, and written in a formal debate style, thoroughly explaining the rationale, impact, and strategic value of the reason in the context of policy debate theory. Each reason should be at least 5-7 sentences and include specific examples, impacts, and theoretical warrants. These should be the kinds of arguments a debater would actually deliver out loud in a round, using the above topicality terms as the core of each reason. "
            "Do NOT reference or mention any evidence or cards in the reasons to prefer. The reasons should stand alone as arguments, without referring to any evidence that may be gathered later. "
            "Then, find the best, most reason-to-prefer-specific evidence supporting these reasons. "
            "Only consider evidence that directly and specifically supports the reasons to prefer. "
            "Reject any evidence that is generic, tangential, or not relevant to the reasons to prefer."
        ),
    )

    # Parse the result
    topicality_reasons_raw_string = chat_result.chat_history[-1]["content"]
    card_json = json.loads(topicality_reasons_raw_string)["cards"][0]
    topicality_reason_evidence_id = card_json["id"]
    detailed_reasons_to_prefer_arguments_as_delivered_in_debate = card_json.get(
        "detailed_reasons_to_prefer_arguments_as_delivered_in_debate", []
    )
    # Get the card document (not used in HTML output)
    # topicality_reason_evidence_doc = get_document_by_id(topicality_reason_evidence_id)
    # card_markup = str(topicality_reason_evidence_doc['markup'])

    # Append to negative_case_html using h2, div, and p tags (do NOT add the card itself)
    negative_case_html += (
        f"\n<h2>Topicality Reasons to Prefer</h2>"
        f"\n<div><ol>"
        + "".join(f"<li>{reason}</li>" for reason in detailed_reasons_to_prefer_arguments_as_delivered_in_debate)
        + "</ol></div>"
    )
    return negative_case_html


In [ ]:
#negative_case_html = add_topicality_reasons_to_prefer_and_evidence(debate_case, negative_case_html)

In [ ]:
#display(HTML(negative_case_html))

## Theory

In [ ]:
#theory_title = negative_case['theory']['title']
#theory_core_argument = negative_case['theory']['core_argument']
#negative_case_html += f"<h2>{theory_title}</h2>\n<p>{theory_core_argument}</p>"


In [ ]:
#display(HTML(negative_case_html))

### Theory Interpretation

In [ ]:
def add_theory_interpretation_and_evidence(debate_case, negative_case_html):
    """
    Given an affirmative debate_case string and a negative_case_html string,
    finds the best formalized theory interpretation and corresponding evidence to support it,
    and appends both to the negative_case_html using h2, div, and p tags.
    Returns the updated negative_case_html string.
    """
    class DebateCard(BaseModel):
        id: int
        cite: str
        include_in_case: Literal["include_it", "False"]
        reason_to_include: str
        formalized_theory_interpretation: str  # Formalized theory interpretation text

    class DebateCardSearchResult(BaseModel):
        cards: List[DebateCard] = Field(..., min_items=1, max_items=1)

    llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        parallel_tool_calls=None
    )
    required_llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        tool_choice="required",
        temperature=1.0,
        top_p=0.9,
        parallel_tool_calls=None
    )
    debate_eval_llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        response_format=DebateCardSearchResult,
        parallel_tool_calls=None
    )

    # Agent setup
    theory_interpretation_agent = ConversableAgent(
        name="theory_interpretation_agent", 
        system_message=(
            f"You are an expert policy debater and debate coach. "
            f"Your job is to:\n"
            f"1. Read the provided affirmative debate case and negative case HTML.\n"
            f"2. Formulate a formalized theory interpretation (definition of a debate theory concept, such as conditionality, severance, or specification) that is strategic, precise, and would help the negative win the round. "
            f"3. Write the interpretation in a formal debate style, including the concept being defined, the definition, and a brief standards/violation explanation. "
            f"4. Suggest the best possible search query to find evidence supporting this theory interpretation (e.g., debate theory articles, academic sources, or authoritative debate handbooks)."
        ),
        llm_config=llm_config,
    )

    theory_eval_agent = ConversableAgent(
        name="theory_eval_agent",
        system_message=(
            f"You are a highly rigorous debate coach and argument analyst. "
            f"Your job is to strictly evaluate whether evidence meets the highest standards for inclusion as support for a theory interpretation in policy debate. "
            f"For each piece of evidence, meticulously scrutinize its:\n"
            f"- Author qualifications (must be from recognized debate theorists, coaches, or authoritative sources)\n"
            f"- Empirical or doctrinal basis (must be supported by concrete debate practice, academic consensus, or published theory literature)\n"
            f"- Direct relevance (must precisely and explicitly support the theory interpretation)\n"
            f"- Strategic value (must provide unique and compelling support for the interpretation, not just generic background)\n"
            f"- Specificity (must not duplicate or closely overlap with other selected evidence, and must establish the definition or standard as interpreted)\n"
            f"- Wording precision (must use exact terminology needed to establish the interpretation)\n\n"
            f"After evaluating the evidence, you must:\n"
            f"1. IMMEDIATELY REJECT (mark as 'False' and/or ignore) any evidence that has already been marked as 'include_it' in previous iterations\n"
            f"2. Reject any evidence that duplicates already selected cards\n"
            f"3. Ensure terminology precisely matches what's needed for the theory interpretation\n"
            f"4. Only approve evidence that meets ALL evaluation criteria and is strictly interpretation-relevant\n\n"
            f"Your goal is to ensure we have the highest quality, interpretation-specific evidence, with absolutely no duplicate or generic cards, and that all included evidence is retagged and recut with precise, policy debate-style markup."
        ),
        llm_config=debate_eval_llm_config,
    )

    theory_search_agent = ConversableAgent(
        name="theory_search_agent",
        system_message="You are a helpful assistant that can search the debate evidence dataset for a given tag. Your query will retrieve a list of debate cards.",
        llm_config=required_llm_config,
    )

    executor_agent = ConversableAgent(
        name="executor_agent",
        human_input_mode="NEVER",
        llm_config=llm_config,
    )

    register_function(
        search_debate_cards,
        caller=theory_search_agent,
        executor=executor_agent,
        description="Search the debate evidence dataset using natural language queries. Return a list of debate cards.",
    )

    from autogen import GroupChat

    iterations = 0

    def custom_speaker_selection_func(last_speaker: Agent, groupchat: GroupChat):
        nonlocal iterations
        messages = groupchat.messages

        if len(messages) <= 1:
            return theory_interpretation_agent

        if last_speaker is theory_search_agent:
            return executor_agent

        if last_speaker is executor_agent:
            return theory_eval_agent
        
        if last_speaker is theory_eval_agent:
            if "include_it" in messages[-1]["content"]:
                iterations += 1
                if iterations >= 3:
                    return None
                else:
                    return theory_search_agent
            else:
                return theory_search_agent

        if last_speaker is theory_interpretation_agent:
            return theory_search_agent
        else:
            return "round_robin"

    group_chat = GroupChat(
        agents=[theory_interpretation_agent, theory_search_agent, executor_agent, theory_eval_agent],
        messages=[],
        max_round=40,
        speaker_selection_method=custom_speaker_selection_func
    )

    group_chat_manager = GroupChatManager(
        groupchat=group_chat,
        llm_config=llm_config,
    )

    # Use the debate_case and negative_case_html for context
    chat_result = theory_interpretation_agent.initiate_chat(
        group_chat_manager,
        message=(
            f"{debate_case}\n\n"
            f"{negative_case_html}\n\n"
            "Assume that the current year is 2022.\n"
            "Formulate a formalized theory interpretation (definition of a debate theory concept, such as conditionality, severance, or specification) that is strategic, precise, and would help the negative win the round. "
            "Write the interpretation in a formal debate style, including the concept being defined, the definition, and a brief standards/violation explanation. "
            "Then, find the best, most interpretation-specific evidence supporting this theory interpretation. "
            "Only consider evidence that directly and specifically supports the interpretation. "
            "Reject any evidence that is generic, tangential, or not relevant to the interpretation."
        ),
    )

    # Parse the result
    theory_evidence_raw_string = chat_result.chat_history[-1]["content"]
    card_json = json.loads(theory_evidence_raw_string)["cards"][0]
    theory_evidence_id = card_json["id"]
    formalized_interpretation = card_json.get("formalized_theory_interpretation", "")
    # Get the card document
    theory_evidence_doc = get_document_by_id(theory_evidence_id)
    card_markup = str(theory_evidence_doc['markup'])

    # Append to negative_case_html using h2, div, and p tags
    negative_case_html += (
        f"\n<h2>Theory Interpretation and Evidence</h2>"
        f"\n<div><p>{formalized_interpretation}</p></div>"
        f"\n<div><p>{card_markup}</p></div>"
    )
    return negative_case_html


In [ ]:
#negative_case_html = add_theory_interpretation_and_evidence(debate_case, negative_case_html)

In [ ]:
#display(HTML(negative_case_html))

### Theory Violation

In [ ]:
def add_theory_violation_and_grounding_evidence(debate_case, negative_case_html):
    """
    Given an affirmative debate_case string and a negative_case_html string,
    finds the best formalized theory violation argument (not just an interpretation) and corresponding evidence to ground it,
    and appends only the violation (not the evidence) to the negative_case_html using h2, div, and p tags.
    Returns the updated negative_case_html string.
    """
    class DebateCard(BaseModel):
        id: int
        cite: str
        include_in_case: Literal["include_it", "False"]
        reason_to_include: str
        formalized_theory_violation: str  # Formalized theory violation text

    class DebateCardSearchResult(BaseModel):
        cards: List[DebateCard] = Field(..., min_items=1, max_items=1)

    llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        parallel_tool_calls=None
    )
    required_llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        tool_choice="required",
        temperature=1.0,
        top_p=0.9,
        parallel_tool_calls=None
    )
    debate_eval_llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        response_format=DebateCardSearchResult,
        parallel_tool_calls=None
    )

    # Agent setup
    theory_violation_agent = ConversableAgent(
        name="theory_violation_agent", 
        system_message=(
            f"You are an expert policy debater and debate coach. "
            f"Your job is to:\n"
            f"1. Read the provided affirmative debate case and negative case HTML.\n"
            f"2. Formulate a formalized theory violation argument (not just an interpretation, but a violation argument including the interpretation, the violation, and standards) that is strategic, precise, and would help the negative win the round. "
            f"3. Write the violation argument in a formal debate style, including the concept being defined, the interpretation, the violation (how the affirmative violates the interpretation), and a brief standards explanation. "
            f"4. Suggest the best possible search query to find evidence supporting this theory violation (e.g., debate theory articles, academic sources, or authoritative debate handbooks)."
        ),
        llm_config=llm_config,
    )

    theory_eval_agent = ConversableAgent(
        name="theory_eval_agent",
        system_message=(
            f"You are a highly rigorous debate coach and argument analyst. "
            f"Your job is to strictly evaluate whether evidence meets the highest standards for inclusion as support for a theory violation in policy debate. "
            f"For each piece of evidence, meticulously scrutinize its:\n"
            f"- Author qualifications (must be from recognized debate theorists, coaches, or authoritative sources)\n"
            f"- Empirical or doctrinal basis (must be supported by concrete debate practice, academic consensus, or published theory literature)\n"
            f"- Direct relevance (must precisely and explicitly support the theory violation argument)\n"
            f"- Strategic value (must provide unique and compelling support for the violation, not just generic background)\n"
            f"- Specificity (must not duplicate or closely overlap with other selected evidence, and must establish the violation or standard as interpreted)\n"
            f"- Wording precision (must use exact terminology needed to establish the violation)\n\n"
            f"After evaluating the evidence, you must:\n"
            f"1. IMMEDIATELY REJECT (mark as 'False' and/or ignore) any evidence that has already been marked as 'include_it' in previous iterations\n"
            f"2. Reject any evidence that duplicates already selected cards\n"
            f"3. Ensure terminology precisely matches what's needed for the theory violation\n"
            f"4. Only approve evidence that meets ALL evaluation criteria and is strictly violation-relevant\n\n"
            f"Your goal is to ensure we have the highest quality, violation-specific evidence, with absolutely no duplicate or generic cards, and that all included evidence is retagged and recut with precise, policy debate-style markup."
        ),
        llm_config=debate_eval_llm_config,
    )

    theory_search_agent = ConversableAgent(
        name="theory_search_agent",
        system_message="You are a helpful assistant that can search the debate evidence dataset for a given tag. Your query will retrieve a list of debate cards.",
        llm_config=required_llm_config,
    )

    executor_agent = ConversableAgent(
        name="executor_agent",
        human_input_mode="NEVER",
        llm_config=llm_config,
    )

    register_function(
        search_debate_cards,
        caller=theory_search_agent,
        executor=executor_agent,
        description="Search the debate evidence dataset using natural language queries. Return a list of debate cards.",
    )

    from autogen import GroupChat

    iterations = 0

    def custom_speaker_selection_func(last_speaker: Agent, groupchat: GroupChat):
        nonlocal iterations
        messages = groupchat.messages

        if len(messages) <= 1:
            return theory_violation_agent

        if last_speaker is theory_search_agent:
            return executor_agent

        if last_speaker is executor_agent:
            return theory_eval_agent
        
        if last_speaker is theory_eval_agent:
            if "include_it" in messages[-1]["content"]:
                iterations += 1
                if iterations >= 3:
                    return None
                else:
                    return theory_search_agent
            else:
                return theory_search_agent

        if last_speaker is theory_violation_agent:
            return theory_search_agent
        else:
            return "round_robin"

    group_chat = GroupChat(
        agents=[theory_violation_agent, theory_search_agent, executor_agent, theory_eval_agent],
        messages=[],
        max_round=40,
        speaker_selection_method=custom_speaker_selection_func
    )

    group_chat_manager = GroupChatManager(
        groupchat=group_chat,
        llm_config=llm_config,
    )

    # Use the debate_case and negative_case_html for context
    chat_result = theory_violation_agent.initiate_chat(
        group_chat_manager,
        message=(
            f"{debate_case}\n\n"
            f"{negative_case_html}\n\n"
            "Assume that the current year is 2022.\n"
            "Formulate a formalized theory violation argument (not just an interpretation, but a violation argument including the interpretation, the violation, and standards) that is strategic, precise, and would help the negative win the round. "
            "Write the violation argument in a formal debate style, including the concept being defined, the interpretation, the violation (how the affirmative violates the interpretation), and a brief standards explanation. "
            "Then, find the best, most violation-specific evidence supporting this theory violation. "
            "Only consider evidence that directly and specifically supports the violation. "
            "Reject any evidence that is generic, tangential, or not relevant to the violation."
        ),
    )

    # Parse the result
    theory_evidence_raw_string = chat_result.chat_history[-1]["content"]
    card_json = json.loads(theory_evidence_raw_string)["cards"][0]
    theory_evidence_id = card_json["id"]
    formalized_violation = card_json.get("formalized_theory_violation", "")
    # Get the card document (for grounding, but do not append to HTML)
    theory_evidence_doc = get_document_by_id(theory_evidence_id)
    card_markup = str(theory_evidence_doc['markup'])

    # Append only the violation argument to negative_case_html using h2, div, and p tags
    negative_case_html += (
        f"\n<h2>Theory Violation Argument</h2>"
        f"\n<div><p>{formalized_violation}</p></div>"
    )
    return negative_case_html


In [ ]:
#negative_case_html = add_theory_violation_and_grounding_evidence(debate_case, negative_case_html)

In [ ]:
#display(HTML(negative_case_html))

### Theory Reasons to Prefer

In [ ]:
def add_theory_reasons_to_prefer_and_evidence(debate_case, negative_case_html):
    """
    Given an affirmative debate_case string and a negative_case_html string (which already contains the formal interpretation and violation),
    generates three long, detailed, formalized "reasons to prefer" the negative's theory interpretation (using classic policy debate theory terminology such as 'competing interpretations', 'abuse', 'predictability', 'education', 'fairness', 'jurisdiction', etc),
    and finds a piece of evidence supporting these reasons. Appends only the reasons to the negative_case_html using h2, div, and p tags.
    Returns the updated negative_case_html string.
    The reasons to prefer should NOT reference or mention the evidence that is gathered.
    """
    from typing import List, Literal
    from pydantic import BaseModel, Field

    class TheoryReasonCard(BaseModel):
        id: int
        cite: str
        include_in_case: Literal["include_it", "False"]
        reason_to_include: str
        detailed_reasons_to_prefer_arguments_as_delivered_in_debate: List[str] = Field(
            ..., min_items=3, max_items=3
        )  # 3 long, detailed, debate-style reasons using classic theory terms

    class TheoryReasonCardSearchResult(BaseModel):
        cards: List[TheoryReasonCard] = Field(..., min_items=1, max_items=1)

    llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        parallel_tool_calls=None
    )
    required_llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        tool_choice="required",
        temperature=1.0,
        top_p=0.9,
        parallel_tool_calls=None
    )
    debate_eval_llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        response_format=TheoryReasonCardSearchResult,
        parallel_tool_calls=None
    )

    # Agent setup
    theory_reasons_agent = ConversableAgent(
        name="theory_reasons_agent", 
        system_message=(
            f"You are an expert policy debater and debate coach. "
            f"Your job is to:\n"
            f"1. Read the provided affirmative debate case and negative case HTML (which already contains the formal interpretation and violation).\n"
            f"2. Generate exactly three distinct, formalized 'reasons to prefer' the negative's theory interpretation, using classic policy debate theory terminology such as 'competing interpretations', 'abuse', 'predictability', 'education', 'fairness', 'jurisdiction', 'brightline', 'limits', 'ground', etc. "
            f"Each reason should be long, detailed, strategic, and written in a formal debate style, thoroughly explaining the rationale, impact, and strategic value of the reason in the context of policy debate theory. Each reason should be at least 5-7 sentences and include specific examples, impacts, and theoretical warrants. These should be the kinds of arguments a debater would actually deliver out loud in a round, using the above theory terms as the core of each reason.\n"
            f"Do NOT reference or mention any evidence or cards in the reasons to prefer. The reasons should stand alone as arguments, without referring to any evidence that may be gathered later.\n"
            f"3. Suggest the best possible search query to find evidence supporting these reasons to prefer, using debate theory terminology (e.g., 'competing interpretations standard', 'abuse standard theory', 'predictability fairness theory', etc)."
        ),
        llm_config=llm_config,
    )

    theory_eval_agent = ConversableAgent(
        name="theory_eval_agent",
        system_message=(
            f"You are a highly rigorous debate coach and argument analyst. "
            f"Your job is to strictly evaluate whether evidence meets the highest standards for inclusion as support for reasons to prefer a theory interpretation in policy debate. "
            f"For each piece of evidence, meticulously scrutinize its:\n"
            f"- Author qualifications (must be from recognized experts or authoritative sources)\n"
            f"- Empirical or doctrinal basis (must be supported by concrete data, legal precedent, or academic consensus)\n"
            f"- Direct relevance (must precisely and explicitly support the reasons to prefer, using debate theory terminology)\n"
            f"- Strategic value (must provide unique and compelling support for the reasons, not just generic background)\n"
            f"- Specificity (must not duplicate or closely overlap with other selected evidence, and must establish the reason as interpreted)\n"
            f"- Wording precision (must use exact terminology needed to establish the reason)\n\n"
            f"After evaluating the evidence, you must:\n"
            f"1. IMMEDIATELY REJECT (mark as 'False' and/or ignore) any evidence that has already been marked as 'include_it' in previous iterations\n"
            f"2. Reject any evidence that duplicates already selected cards\n"
            f"3. Ensure terminology precisely matches what's needed for the reasons to prefer\n"
            f"4. Only approve evidence that meets ALL evaluation criteria and is strictly relevant to the reasons to prefer\n\n"
            f"Your goal is to ensure we have the highest quality, reason-to-prefer-specific evidence, with absolutely no duplicate or generic cards, and that all included evidence is retagged and recut with precise, policy debate-style markup."
        ),
        llm_config=debate_eval_llm_config,
    )

    theory_search_agent = ConversableAgent(
        name="theory_search_agent",
        system_message=(
            "You are a helpful assistant that can search the debate evidence dataset for a given tag. "
            "When searching for evidence to support reasons to prefer a theory interpretation, "
            "use classic policy debate theory terminology in your queries, such as 'competing interpretations', 'abuse', 'predictability', 'education', 'fairness', 'jurisdiction', 'brightline', 'limits', 'ground', etc. "
            "Your query will retrieve a list of debate cards."
        ),
        llm_config=required_llm_config,
    )

    executor_agent = ConversableAgent(
        name="executor_agent",
        human_input_mode="NEVER",
        llm_config=llm_config,
    )

    register_function(
        search_debate_cards,
        caller=theory_search_agent,
        executor=executor_agent,
        description="Search the debate evidence dataset using natural language queries. Return a list of debate cards.",
    )

    from autogen import GroupChat

    iterations = 0

    def custom_speaker_selection_func(last_speaker: Agent, groupchat: GroupChat):
        nonlocal iterations
        messages = groupchat.messages

        if len(messages) <= 1:
            return theory_reasons_agent

        if last_speaker is theory_search_agent:
            return executor_agent

        if last_speaker is executor_agent:
            return theory_eval_agent
        
        if last_speaker is theory_eval_agent:
            if "include_it" in messages[-1]["content"]:
                iterations += 1
                if iterations >= 3:
                    return None
                else:
                    return theory_search_agent
            else:
                return theory_search_agent

        if last_speaker is theory_reasons_agent:
            return theory_search_agent
        else:
            return "round_robin"

    group_chat = GroupChat(
        agents=[theory_reasons_agent, theory_search_agent, executor_agent, theory_eval_agent],
        messages=[],
        max_round=40,
        speaker_selection_method=custom_speaker_selection_func
    )

    group_chat_manager = GroupChatManager(
        groupchat=group_chat,
        llm_config=llm_config,
    )

    # Use the debate_case and negative_case_html for context
    chat_result = theory_reasons_agent.initiate_chat(
        group_chat_manager,
        message=(
            f"{debate_case}\n\n"
            f"{negative_case_html}\n\n"
            "Assume that the current year is 2022.\n"
            "Given the formal interpretation and violation already included above, generate exactly three distinct, formalized 'reasons to prefer' the negative's theory interpretation, using classic policy debate theory terminology such as 'competing interpretations', 'abuse', 'predictability', 'education', 'fairness', 'jurisdiction', 'brightline', 'limits', 'ground', etc. "
            "Each reason should be long, detailed, strategic, and written in a formal debate style, thoroughly explaining the rationale, impact, and strategic value of the reason in the context of policy debate theory. Each reason should be at least 5-7 sentences and include specific examples, impacts, and theoretical warrants. These should be the kinds of arguments a debater would actually deliver out loud in a round, using the above theory terms as the core of each reason. "
            "Do NOT reference or mention any evidence or cards in the reasons to prefer. The reasons should stand alone as arguments, without referring to any evidence that may be gathered later. "
            "Then, find the best, most reason-to-prefer-specific evidence supporting these reasons. "
            "Only consider evidence that directly and specifically supports the reasons to prefer. "
            "Reject any evidence that is generic, tangential, or not relevant to the reasons to prefer."
        ),
    )

    # Parse the result
    theory_reasons_raw_string = chat_result.chat_history[-1]["content"]
    card_json = json.loads(theory_reasons_raw_string)["cards"][0]
    theory_reason_evidence_id = card_json["id"]
    detailed_reasons_to_prefer_arguments_as_delivered_in_debate = card_json.get(
        "detailed_reasons_to_prefer_arguments_as_delivered_in_debate", []
    )
    # Get the card document (not used in HTML output)
    # theory_reason_evidence_doc = get_document_by_id(theory_reason_evidence_id)
    # card_markup = str(theory_reason_evidence_doc['markup'])

    # Append to negative_case_html using h2, div, and p tags (do NOT add the card itself)
    negative_case_html += (
        f"\n<h2>Theory Reasons to Prefer</h2>"
        f"\n<div><ol>"
        + "".join(f"<li>{reason}</li>" for reason in detailed_reasons_to_prefer_arguments_as_delivered_in_debate)
        + "</ol></div>"
    )
    return negative_case_html


In [ ]:
#negative_case_html = add_theory_reasons_to_prefer_and_evidence(debate_case, negative_case_html)

In [ ]:
#display(HTML(negative_case_html))

## Disadvantages

In [ ]:
#disadvantage_title = negative_case['disadvantages'][0]['title']
#disadvantage_core_argument = negative_case['disadvantages'][0]['core_argument']
#negative_case_html += f"<h2>{disadvantage_title}</h2>\n<p>{disadvantage_core_argument}</p>"


In [ ]:
#display(HTML(negative_case_html))

### Disadvantage Uniqueness

In [ ]:
def add_disadvantage_uniqueness_to_case(debate_case, negative_case_html):
    """
    Given a debate_case string and a negative_case_html string,
    finds the best uniqueness evidence for the disadvantage and appends it to the negative_case_html using h2, div, and p tags.
    The output includes the retagged argument as read out loud in the debate round and the card itself, but not the reason.
    Returns the modified negative_case_html string.
    """
    class DebateCard(BaseModel):
        id: int
        cite: str
        include_in_case: Literal["include_it", "False"]
        reason_to_include: str
        retagged_argument_as_read_outloud_in_the_debate_round: str  # Argument to be presented as the first card after the plantext in a debate round

    class DebateCardSearchResult(BaseModel):
        cards: List[DebateCard] = Field(..., min_items=1, max_items=1)

    llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        parallel_tool_calls=None
    )
    required_llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        tool_choice="required",
        temperature=1.0,
        top_p=0.9,
        parallel_tool_calls=None
    )
    debate_eval_llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        response_format=DebateCardSearchResult,
        parallel_tool_calls=None
    )

    # Agent setup
    argument_evaluator = ConversableAgent(
        name="argument_evaluator", 
        system_message=(
            f"You are an expert policy debater focused on finding the best possible evidence of uniqueness for a specific disadvantage in a policy debate case. "
            f"Your job is to:\n"
            f"1. Break down the plan, link, internal link, and impact of the disadvantage into their key components and causal relationships.\n"
            f"2. Guide evidence collection by:\n"
            f"   - Formulating extremely precise search queries that target only evidence which directly and specifically demonstrates the current status quo regarding the impact area of the disadvantage (i.e., uniqueness). "
            f"   - Using BM25 search to find relevant cards from a debate evidence database (cutoff year 2022).\n"
            f"   - If you are being called after previous searches, you must significantly modify and refine your BM25 search queries to maximize the chance of finding new, more relevant, or more specific uniqueness evidence. Do not simply repeat or slightly alter previous queries—make substantial changes to your search approach, keywords, or focus.\n"
            f"   - Suggest query refinements to maximize the chance of finding evidence that directly supports the uniqueness claim for the disadvantage.\n"
            f"3. Evaluate evidence quality for:\n"
            f"   - Direct, explicit support for the uniqueness claim (evidence must not merely be tangentially related or generic background).\n"
            f"   - Specificity: The evidence must establish the current state of affairs in the impact area of the disadvantage, and explain why the impact is not already occurring or is not inevitable absent the plan.\n"
            f"   - Empirical support and authoritativeness.\n"
            f"Reject any evidence that does not fully and directly support the uniqueness of the disadvantage or that could be interpreted as generic or non-specific. "
            f"Your goal is to find the strictest, most disadvantage- and plan-relevant uniqueness evidence possible, ensuring that each selected card is unique and not a duplicate of any previously included evidence."
        ),
        llm_config=llm_config,
    )

    debate_eval_agent = ConversableAgent(
        name="debate_eval_agent",
        system_message=(
            f"You are an extremely selective and rigorous debate coach and argument analyst. "
            f"Your job is to strictly evaluate whether evidence meets the highest standards for inclusion as uniqueness evidence supporting a specific disadvantage in policy debate. "
            f"For each piece of evidence, meticulously scrutinize its:\n"
            f"- Author qualifications (must be from recognized experts or authoritative sources)\n"
            f"- Empirical basis (must be supported by concrete data and research)\n"
            f"- Direct relevance (must precisely and explicitly support the uniqueness claim for the disadvantage)\n"
            f"- Strategic value (must provide unique and compelling support for the disadvantage's uniqueness, not just generic background)\n"
            f"- Specificity (must not duplicate or closely overlap with other selected evidence, and must establish the current state of affairs in the impact area of the disadvantage)\n"
            f"- Wording precision (must use exact terminology needed to establish the uniqueness link for the disadvantage)\n\n"
            f"After evaluating the evidence, you must:\n"
            f"1. IMMEDIATELY REJECT (mark as 'False' and/or ignore) any evidence that has already been marked as 'include_it' in previous iterations\n"
            f"2. Reject any evidence that duplicates already selected cards\n"
            f"3. Ensure terminology precisely matches what's needed for the disadvantage's uniqueness link chains\n"
            f"4. Only approve evidence that meets ALL evaluation criteria and is strictly disadvantage- and plan-relevant for uniqueness\n\n"
            f"Your goal is to ensure we have the highest quality, disadvantage- and plan-specific uniqueness evidence, with absolutely no duplicate or generic cards, and that all included evidence is retagged and recut with precise, policy debate-style markup."
        ),
        llm_config=debate_eval_llm_config,
    )

    debate_search_agent = ConversableAgent(
        name="debate_search_agent",
        system_message="You are a helpful assistant that can search the debate evidence dataset for a given tag. Your query will retrieve a list of debate cards.",
        llm_config=required_llm_config,
    )

    executor_agent = ConversableAgent(
        name="executor_agent",
        human_input_mode="NEVER",
        llm_config=llm_config,
    )

    register_function(
        search_debate_cards,
        caller=debate_search_agent,
        executor=executor_agent,
        description="Search the debate evidence dataset using natural language queries. Return a list of debate cards.",
    )

    from autogen import GroupChat

    iterations = 0

    def custom_speaker_selection_func(last_speaker: Agent, groupchat: GroupChat):
        nonlocal iterations
        messages = groupchat.messages

        if len(messages) <= 1:
            return argument_evaluator

        if last_speaker is debate_search_agent:
            return executor_agent

        if last_speaker is executor_agent:
            return debate_eval_agent
        
        if last_speaker is debate_eval_agent:
            if "include_it" in messages[-1]["content"]:
                iterations += 1
                if iterations >= 3:
                    return None
                else:
                    return debate_search_agent
            else:
                return debate_search_agent

        if last_speaker is argument_evaluator:
            return debate_search_agent
        else:
            return "round_robin"

    group_chat = GroupChat(
        agents=[argument_evaluator, debate_search_agent, executor_agent, debate_eval_agent],
        messages=[],
        max_round=40,
        speaker_selection_method=custom_speaker_selection_func
    )

    group_chat_manager = GroupChatManager(
        groupchat=group_chat,
        llm_config=llm_config,
    )

    # Add the negative case here
    chat_result = argument_evaluator.initiate_chat(
        group_chat_manager,
        message=(
            f"{debate_case}\n\n"
            f"{negative_case_html}\n\n"
            "Assume that the current year is 2022.\n"
            f"Find the best, most plan- and disadvantage-specific evidence of uniqueness supporting the disadvantage articulated above. "
            f"Only consider evidence that directly and specifically supports the claim that the impact area of the disadvantage is not already occurring, is not inevitable, or is not being solved in the status quo. "
            f"Reject any evidence that is generic, tangential, or not relevant to the uniqueness of the disadvantage. "
        ),
    )

    # Parse the result
    disadvantage_uniqueness_raw_string = chat_result.chat_history[-1]["content"]
    card_json = json.loads(disadvantage_uniqueness_raw_string)["cards"][0]
    disadvantage_uniqueness_id = card_json["id"]
    retagged_argument = card_json.get("retagged_argument_as_read_outloud_in_the_debate_round", "")
    # Get the card document
    disadvantage_uniqueness_doc = get_document_by_id(disadvantage_uniqueness_id)
    card_markup = str(disadvantage_uniqueness_doc['markup'])

    # Append to negative_case_html using h2, div, and p tags
    negative_case_html += (
        f"\n<h2>Disadvantage Uniqueness</h2>"
        f"\n<div><p>{retagged_argument}</p></div>"
        f"\n<div><p>{card_markup}</p></div>"
    )
    return negative_case_html


In [ ]:
#negative_case_html = add_disadvantage_uniqueness_to_case(debate_case, negative_case_html)

In [ ]:
#display(HTML(negative_case_html))

### Disadvantage Link

In [ ]:
def add_disadvantage_link_to_case(debate_case, negative_case_html):
    class DebateCard(BaseModel):
        id: int
        cite: str
        include_in_case: Literal["include_it", "False"]
        reason_to_include: str
        retagged_argument_as_read_outloud_in_the_debate_round: str  # Argument to be presented as the first card after the plantext in a debate round

    class DebateCardSearchResult(BaseModel):
        cards: List[DebateCard] = Field(..., min_items=1, max_items=1)

    llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        parallel_tool_calls=None
    )
    required_llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        tool_choice="required",
        temperature=1.0,
        top_p=0.9,
        parallel_tool_calls=None
    )
    debate_eval_llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        response_format=DebateCardSearchResult,
        parallel_tool_calls=None
    )

    argument_evaluator = ConversableAgent(
        name="argument_evaluator", 
        system_message=(
            "You are an expert policy debater focused on finding the best possible evidence of a causal link for the disadvantage in a policy debate case. "
            "The debate topic, plan, and disadvantage (including uniqueness evidence) have already been provided. "
            "Your job is to:\n"
            "1. Break down the plan and disadvantage (including uniqueness) into their key components and causal relationships.\n"
            "2. Guide evidence collection by:\n"
            "   - Formulating extremely precise search queries that target only evidence which directly and specifically demonstrates that the plan, if enacted, would cause the impact described in the disadvantage (i.e., link evidence). "
            "   - Using BM25 search to find relevant cards from a debate evidence database.\n"
            "   - If you are being called after previous searches, you must significantly modify and refine your BM25 search queries to maximize the chance of finding new, more relevant, or more specific link evidence. Do not simply repeat or slightly alter previous queries—make substantial changes to your search approach, keywords, or focus.\n"
            "   - Suggest query refinements to maximize the chance of finding evidence that directly supports the link claim for the disadvantage.\n"
            "3. Evaluate evidence quality for:\n"
            "   - Direct, explicit support for the link claim (evidence must not merely be tangentially related or generic background).\n"
            "   - Specificity: The evidence must establish that the plan, as proposed, will cause the impact described in the disadvantage, and explain the mechanism by which this occurs.\n"
            "   - Empirical support and authoritativeness.\n"
            "Reject any evidence that does not fully and directly support the link between the plan and the impact of the disadvantage, or that could be interpreted as generic or non-specific. "
            "Your goal is to find the strictest, most disadvantage- and plan-relevant link evidence possible, ensuring that each selected card is unique and not a duplicate of any previously included evidence."
        ),
        llm_config=llm_config,
    )

    debate_eval_agent = ConversableAgent(
        name="debate_eval_agent",
        system_message=(
            "You are an extremely selective and rigorous debate coach and argument analyst. "
            "Your job is to strictly evaluate whether evidence meets the highest standards for inclusion as link evidence supporting the disadvantage in policy debate. "
            "For each piece of evidence, meticulously scrutinize its:\n"
            "- Author qualifications (must be from recognized experts or authoritative sources)\n"
            "- Empirical basis (must be supported by concrete data and research)\n"
            "- Direct relevance (must precisely and explicitly support the link claim for the disadvantage)\n"
            "- Strategic value (must provide unique and compelling support for the disadvantage's link, not just generic background)\n"
            "- Specificity (must not duplicate or closely overlap with other selected evidence, and must establish that the plan, as proposed, will cause the impact described in the disadvantage)\n"
            "- Wording precision (must use exact terminology needed to establish the link for the disadvantage)\n\n"
            "After evaluating the evidence, you must:\n"
            "1. IMMEDIATELY REJECT (mark as 'False' and/or ignore) any evidence that has already been marked as 'include_it' in previous iterations\n"
            "2. Reject any evidence that duplicates already selected cards\n"
            "3. Ensure terminology precisely matches what's needed for the disadvantage's link chains\n"
            "4. Only approve evidence that meets ALL evaluation criteria and is strictly disadvantage- and plan-relevant for the link\n\n"
            "Your goal is to ensure we have the highest quality, disadvantage- and plan-specific link evidence, with absolutely no duplicate or generic cards, and that all included evidence is retagged and recut with precise, policy debate-style markup."
        ),
        llm_config=debate_eval_llm_config,
    )

    debate_search_agent = ConversableAgent(
        name="debate_search_agent",
        system_message="You are a helpful assistant that can search the debate evidence dataset for a given tag. Your query will retrieve a list of debate cards.",
        llm_config=required_llm_config,
    )

    executor_agent = ConversableAgent(
        name="executor_agent",
        human_input_mode="NEVER",
        llm_config=llm_config,
    )

    register_function(
        search_debate_cards,
        caller=debate_search_agent,
        executor=executor_agent,
        description="Search the debate evidence dataset using natural language queries. Return a list of debate cards.",
    )

    from autogen import GroupChat

    iterations = 0

    def custom_speaker_selection_func(last_speaker: Agent, groupchat: GroupChat):
        nonlocal iterations
        messages = groupchat.messages

        if len(messages) <= 1:
            return argument_evaluator

        if last_speaker is debate_search_agent:
            return executor_agent

        if last_speaker is executor_agent:
            return debate_eval_agent
        
        if last_speaker is debate_eval_agent:
            if "include_it" in messages[-1]["content"]:
                iterations += 1
                print(f"iterations: {iterations}")
                if iterations >= 3:
                    return None
                else:
                    return debate_search_agent
            else:
                return debate_search_agent

        if last_speaker is argument_evaluator:
            return debate_search_agent
        else:
            return "round_robin"

    group_chat = GroupChat(
        agents=[argument_evaluator, debate_search_agent, executor_agent, debate_eval_agent],
        messages=[],
        max_round=40,
        speaker_selection_method=custom_speaker_selection_func
    )

    group_chat_manager = GroupChatManager(
        groupchat=group_chat,
        llm_config=llm_config,
    )

    # Use the debate_case, which includes the debate topic, the plan, and the disadvantage (including uniqueness evidence) for context

    chat_result = argument_evaluator.initiate_chat(
        group_chat_manager,
        message=(
            f"{debate_case}\n\n"
            f"Negative Case:\n{negative_case_html}\n\n"
            "Assume that the current year is 2022.\n"
            "The disadvantage and its uniqueness evidence have already been established above.\n"
            "Find the best and most plan- and disadvantage-specific evidence of a causal link supporting the disadvantage articulated above. "
            "Only consider evidence that directly and specifically supports the claim that the plan, if enacted, would cause the impact described in the disadvantage. "
            "Reject any evidence that is generic, tangential, or not relevant to the link between the plan and the disadvantage. "
            # No recency or cutoff requirements; old evidence is acceptable if it is high quality.
        ),
    )

    disadvantage_link_raw_string = chat_result.chat_history[-1]["content"]
    disadvantage_link_json = json.loads(disadvantage_link_raw_string)
    card_json = disadvantage_link_json["cards"][0]
    disadvantage_link_id = card_json["id"]
    retagged_argument = card_json.get("retagged_argument_as_read_outloud_in_the_debate_round", "")

    disadvantage_link_doc = get_document_by_id(disadvantage_link_id)
    card_markup = str(disadvantage_link_doc['markup'])

    # Append to negative_case_html using h2, div, and p tags
    negative_case_html += (
        f"\n<h2>Disadvantage Link</h2>"
        f"\n<div><p>{retagged_argument}</p></div>"
        f"\n<div><p>{card_markup}</p></div>"
    )
    return negative_case_html

In [ ]:
# negative_case_html = add_disadvantage_link_to_case(debate_case, negative_case_html)

In [ ]:
# display(HTML(negative_case_html))

### Disadvantage Internal Link

In [ ]:
def add_disadvantage_internal_link_to_case(debate_case, negative_case_html):
    class DebateCard(BaseModel):
        id: int
        cite: str
        include_in_case: Literal["include_it", "False"]
        reason_to_include: str
        retagged_argument_as_read_outloud_in_the_debate_round: str  # Argument to be presented as the first card after the plantext in a debate round

    class DebateCardSearchResult(BaseModel):
        cards: List[DebateCard] = Field(..., min_items=1, max_items=1)

    llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        parallel_tool_calls=None
    )
    required_llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        tool_choice="required",
        temperature=1.0,
        top_p=0.9,
        parallel_tool_calls=None
    )
    debate_eval_llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        response_format=DebateCardSearchResult,
        parallel_tool_calls=None
    )

    argument_evaluator = ConversableAgent(
        name="argument_evaluator", 
        system_message=(
            "You are an expert policy debater focused on finding the best possible evidence of a causal internal link for the disadvantage in a policy debate case. "
            "The debate topic, plan, and disadvantage (including uniqueness and link evidence) have already been provided. "
            "Your job is to:\n"
            "1. Break down the plan and disadvantage (including uniqueness and link) into their key components and causal relationships.\n"
            "2. Guide evidence collection by:\n"
            "   - Formulating extremely precise search queries that target only evidence which directly and specifically demonstrates the internal link: that the link (already established) will cause the impact described in the disadvantage (i.e., internal link evidence). "
            "   - Using BM25 search to find relevant cards from a debate evidence database.\n"
            "   - If you are being called after previous searches, you must significantly modify and refine your BM25 search queries to maximize the chance of finding new, more relevant, or more specific internal link evidence. Do not simply repeat or slightly alter previous queries—make substantial changes to your search approach, keywords, or focus.\n"
            "   - Suggest query refinements to maximize the chance of finding evidence that directly supports the internal link claim for the disadvantage.\n"
            "3. Evaluate evidence quality for:\n"
            "   - Direct, explicit support for the internal link claim (evidence must not merely be tangentially related or generic background).\n"
            "   - Specificity: The evidence must establish that the link, as proposed, will cause the impact described in the disadvantage, and explain the mechanism by which this occurs.\n"
            "   - Empirical support and authoritativeness.\n"
            "Reject any evidence that does not fully and directly support the internal link between the link and the impact of the disadvantage, or that could be interpreted as generic or non-specific. "
            "Your goal is to find the strictest, most disadvantage- and plan-relevant internal link evidence possible, ensuring that each selected card is unique and not a duplicate of any previously included evidence."
        ),
        llm_config=llm_config,
    )

    debate_eval_agent = ConversableAgent(
        name="debate_eval_agent",
        system_message=(
            "You are an extremely selective and rigorous debate coach and argument analyst. "
            "Your job is to strictly evaluate whether evidence meets the highest standards for inclusion as internal link evidence supporting the disadvantage in policy debate. "
            "For each piece of evidence, meticulously scrutinize its:\n"
            "- Author qualifications (must be from recognized experts or authoritative sources)\n"
            "- Empirical basis (must be supported by concrete data and research)\n"
            "- Direct relevance (must precisely and explicitly support the internal link claim for the disadvantage)\n"
            "- Strategic value (must provide unique and compelling support for the disadvantage's internal link, not just generic background)\n"
            "- Specificity (must not duplicate or closely overlap with other selected evidence, and must establish that the link, as proposed, will cause the impact described in the disadvantage)\n"
            "- Wording precision (must use exact terminology needed to establish the internal link for the disadvantage)\n\n"
            "After evaluating the evidence, you must:\n"
            "1. IMMEDIATELY REJECT (mark as 'False' and/or ignore) any evidence that has already been marked as 'include_it' in previous iterations\n"
            "2. Reject any evidence that duplicates already selected cards\n"
            "3. Ensure terminology precisely matches what's needed for the disadvantage's internal link chains\n"
            "4. Only approve evidence that meets ALL evaluation criteria and is strictly disadvantage- and plan-relevant for the internal link\n\n"
            "Your goal is to ensure we have the highest quality, disadvantage- and plan-specific internal link evidence, with absolutely no duplicate or generic cards, and that all included evidence is retagged and recut with precise, policy debate-style markup."
        ),
        llm_config=debate_eval_llm_config,
    )

    debate_search_agent = ConversableAgent(
        name="debate_search_agent",
        system_message="You are a helpful assistant that can search the debate evidence dataset for a given tag. Your query will retrieve a list of debate cards.",
        llm_config=required_llm_config,
    )

    executor_agent = ConversableAgent(
        name="executor_agent",
        human_input_mode="NEVER",
        llm_config=llm_config,
    )

    register_function(
        search_debate_cards,
        caller=debate_search_agent,
        executor=executor_agent,
        description="Search the debate evidence dataset using natural language queries. Return a list of debate cards.",
    )

    from autogen import GroupChat

    iterations = 0

    def custom_speaker_selection_func(last_speaker: Agent, groupchat: GroupChat):
        nonlocal iterations
        messages = groupchat.messages

        if len(messages) <= 1:
            return argument_evaluator

        if last_speaker is debate_search_agent:
            return executor_agent

        if last_speaker is executor_agent:
            return debate_eval_agent
        
        if last_speaker is debate_eval_agent:
            if "include_it" in messages[-1]["content"]:
                iterations += 1
                print(f"iterations: {iterations}")
                if iterations >= 3:
                    return None
                else:
                    return debate_search_agent
            else:
                return debate_search_agent

        if last_speaker is argument_evaluator:
            return debate_search_agent
        else:
            return "round_robin"

    group_chat = GroupChat(
        agents=[argument_evaluator, debate_search_agent, executor_agent, debate_eval_agent],
        messages=[],
        max_round=40,
        speaker_selection_method=custom_speaker_selection_func
    )

    group_chat_manager = GroupChatManager(
        groupchat=group_chat,
        llm_config=llm_config,
    )

    # Use the debate_case, which includes the debate topic, the plan, and the disadvantage (including uniqueness and link evidence) for context

    # Add the negative case here
    chat_result = argument_evaluator.initiate_chat(
        group_chat_manager,
        message=(
            f"{debate_case}\n\n"
            f"{negative_case_html}\n\n"
            "Assume that the current year is 2022.\n"
            "The disadvantage, its uniqueness, and link evidence have already been established above.\n"
            "Find the best and most plan- and disadvantage-specific evidence of a causal internal link supporting the disadvantage articulated above. "
            "Only consider evidence that directly and specifically supports the claim that the link (already established) will cause the impact described in the disadvantage. "
            "Reject any evidence that is generic, tangential, or not relevant to the internal link between the link and the disadvantage's impact. "
            # No recency or cutoff requirements; old evidence is acceptable if it is high quality.
        ),
    )

    disadvantage_internal_link_raw_string = chat_result.chat_history[-1]["content"]
    disadvantage_internal_link_json = json.loads(disadvantage_internal_link_raw_string)
    card_json = disadvantage_internal_link_json["cards"][0]
    disadvantage_internal_link_id = card_json["id"]
    retagged_argument = card_json.get("retagged_argument_as_read_outloud_in_the_debate_round", "")

    disadvantage_internal_link_doc = get_document_by_id(disadvantage_internal_link_id)
    card_markup = str(disadvantage_internal_link_doc['markup'])

    # Append to negative_case_html using h2, div, and p tags
    negative_case_html += (
        f"\n<h2>Disadvantage Internal Link</h2>"
        f"\n<div><p>{retagged_argument}</p></div>"
        f"\n<div><p>{card_markup}</p></div>"
    )
    return negative_case_html

In [ ]:
# negative_case_html = add_disadvantage_internal_link_to_case(debate_case, negative_case_html)

In [ ]:
# display(HTML(negative_case_html))

### Disadvantage Impact

In [ ]:
def add_disadvantage_impact_to_case(debate_case, negative_case_html):
    class DebateCard(BaseModel):
        id: int
        cite: str
        include_in_case: Literal["include_it", "False"]
        reason_to_include: str
        retagged_argument_as_read_outloud_in_the_debate_round: str  # Argument to be presented as the first card after the plantext in a debate round

    class DebateCardSearchResult(BaseModel):
        cards: List[DebateCard] = Field(..., min_items=1, max_items=1)

    llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        parallel_tool_calls=None
    )
    required_llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        tool_choice="required",
        temperature=1.0,
        top_p=0.9,
        parallel_tool_calls=None
    )
    debate_eval_llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        response_format=DebateCardSearchResult,
        parallel_tool_calls=None
    )

    argument_evaluator = ConversableAgent(
        name="argument_evaluator", 
        system_message=(
            "You are an expert policy debater focused on finding the best possible evidence of a disadvantage impact for a policy debate case. "
            "The debate topic, plan, and disadvantage (including uniqueness and link evidence) have already been provided. "
            "Your job is to:\n"
            "1. Break down the plan and disadvantage (including uniqueness and link) into their key components and causal relationships.\n"
            "2. Guide evidence collection by:\n"
            "   - Formulating extremely precise search queries that target only evidence which directly and specifically demonstrates the impact: that the internal link (already established) will cause the impact described in the disadvantage (i.e., impact evidence). "
            "   - Using BM25 search to find relevant cards from a debate evidence database.\n"
            "   - If you are being called after previous searches, you must significantly modify and refine your BM25 search queries to maximize the chance of finding new, more relevant, or more specific impact evidence. Do not simply repeat or slightly alter previous queries—make substantial changes to your search approach, keywords, or focus.\n"
            "   - Suggest query refinements to maximize the chance of finding evidence that directly supports the impact claim for the disadvantage.\n"
            "3. Evaluate evidence quality for:\n"
            "   - Direct, explicit support for the impact claim (evidence must not merely be tangentially related or generic background).\n"
            "   - Specificity: The evidence must establish that the internal link, as proposed, will cause the impact described in the disadvantage, and explain the mechanism by which this occurs.\n"
            "   - Empirical support and authoritativeness.\n"
            "Reject any evidence that does not fully and directly support the impact of the disadvantage, or that could be interpreted as generic or non-specific. "
            "Your goal is to find the strictest, most disadvantage- and plan-relevant impact evidence possible, ensuring that each selected card is unique and not a duplicate of any previously included evidence."
        ),
        llm_config=llm_config,
    )

    debate_eval_agent = ConversableAgent(
        name="debate_eval_agent",
        system_message=(
            "You are an extremely selective and rigorous debate coach and argument analyst. "
            "Your job is to strictly evaluate whether evidence meets the highest standards for inclusion as impact evidence supporting the disadvantage in policy debate. "
            "For each piece of evidence, meticulously scrutinize its:\n"
            "- Author qualifications (must be from recognized experts or authoritative sources)\n"
            "- Empirical basis (must be supported by concrete data and research)\n"
            "- Direct relevance (must precisely and explicitly support the impact claim for the disadvantage)\n"
            "- Strategic value (must provide unique and compelling support for the disadvantage's impact, not just generic background)\n"
            "- Specificity (must not duplicate or closely overlap with other selected evidence, and must establish that the internal link, as proposed, will cause the impact described in the disadvantage)\n"
            "- Wording precision (must use exact terminology needed to establish the impact for the disadvantage)\n\n"
            "After evaluating the evidence, you must:\n"
            "1. IMMEDIATELY REJECT (mark as 'False' and/or ignore) any evidence that has already been marked as 'include_it' in previous iterations\n"
            "2. Reject any evidence that duplicates already selected cards\n"
            "3. Ensure terminology precisely matches what's needed for the disadvantage's impact chains\n"
            "4. Only approve evidence that meets ALL evaluation criteria and is strictly disadvantage- and plan-relevant for the impact\n\n"
            "Your goal is to ensure we have the highest quality, disadvantage- and plan-specific impact evidence, with absolutely no duplicate or generic cards, and that all included evidence is retagged and recut with precise, policy debate-style markup."
        ),
        llm_config=debate_eval_llm_config,
    )

    debate_search_agent = ConversableAgent(
        name="debate_search_agent",
        system_message="You are a helpful assistant that can search the debate evidence dataset for a given tag. Your query will retrieve a list of debate cards.",
        llm_config=required_llm_config,
    )

    executor_agent = ConversableAgent(
        name="executor_agent",
        human_input_mode="NEVER",
        llm_config=llm_config,
    )

    register_function(
        search_debate_cards,
        caller=debate_search_agent,
        executor=executor_agent,
        description="Search the debate evidence dataset using natural language queries. Return a list of debate cards.",
    )

    from autogen import GroupChat

    iterations = 0

    def custom_speaker_selection_func(last_speaker: Agent, groupchat: GroupChat):
        nonlocal iterations
        messages = groupchat.messages

        if len(messages) <= 1:
            return argument_evaluator

        if last_speaker is debate_search_agent:
            return executor_agent

        if last_speaker is executor_agent:
            return debate_eval_agent
        
        if last_speaker is debate_eval_agent:
            if "include_it" in messages[-1]["content"]:
                iterations += 1
                print(f"iterations: {iterations}")
                if iterations >= 3:
                    return None
                else:
                    return debate_search_agent
            else:
                return debate_search_agent

        if last_speaker is argument_evaluator:
            return debate_search_agent
        else:
            return "round_robin"

    group_chat = GroupChat(
        agents=[argument_evaluator, debate_search_agent, executor_agent, debate_eval_agent],
        messages=[],
        max_round=40,
        speaker_selection_method=custom_speaker_selection_func
    )

    group_chat_manager = GroupChatManager(
        groupchat=group_chat,
        llm_config=llm_config,
    )

    # Use the debate_case, which includes the debate topic, the plan, and the disadvantage (including uniqueness and link evidence) for context

    # Add the negative case here
    chat_result = argument_evaluator.initiate_chat(
        group_chat_manager,
        message=(
            f"{debate_case}\n\n"
            f"{negative_case_html}\n\n"
            "Assume that the current year is 2022.\n"
            "The disadvantage, its uniqueness, and link evidence have already been established above.\n"
            "Find the best and most plan- and disadvantage-specific evidence of a causal impact supporting the disadvantage articulated above. "
            "Only consider evidence that directly and specifically supports the claim that the internal link (already established) will cause the impact described in the disadvantage. "
            "Reject any evidence that is generic, tangential, or not relevant to the impact of the disadvantage. "
            # No recency or cutoff requirements; old evidence is acceptable if it is high quality.
        ),
    )

    disadvantage_impact_raw_string = chat_result.chat_history[-1]["content"]
    disadvantage_impact_json = json.loads(disadvantage_impact_raw_string)
    card_json = disadvantage_impact_json["cards"][0]
    disadvantage_impact_id = card_json["id"]
    retagged_argument = card_json.get("retagged_argument_as_read_outloud_in_the_debate_round", "")

    disadvantage_impact_doc = get_document_by_id(disadvantage_impact_id)
    card_markup = str(disadvantage_impact_doc['markup'])

    # Append to negative_case_html using h2, div, and p tags
    negative_case_html += (
        f"\n<h2>Disadvantage Impact</h2>"
        f"\n<div><p>{retagged_argument}</p></div>"
        f"\n<div><p>{card_markup}</p></div>"
    )
    return negative_case_html

In [ ]:
# negative_case_html = add_disadvantage_impact_to_case(debate_case, negative_case_html)


In [ ]:
# display(HTML(negative_case_html))

## Counterplan

### Counterplan Text

In [ ]:
def add_counterplan_text_to_case(debate_case, negative_case_html):
    """
    Given a debate_case string and a negative_case_html string,
    finds the best counterplan text (including all types: PICs, Advantage CPs, etc.) and appends only the counterplan text (not the card) to the negative_case_html using h2, div, and p tags.
    The output includes the counterplan text as it would be read out loud in a debate round, but does not include the card itself.
    Returns the modified negative_case_html string.
    """
    class DebateCard(BaseModel):
        id: int
        cite: str
        include_in_case: Literal["include_it", "False"]
        reason_to_include: str
        counterplantext: str  # The counterplan text to be presented as read out loud in the debate round

    class DebateCardSearchResult(BaseModel):
        cards: List[DebateCard] = Field(..., min_items=1, max_items=1)

    llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        parallel_tool_calls=None
    )
    required_llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        tool_choice="required",
        temperature=1.0,
        top_p=0.9,
        parallel_tool_calls=None
    )
    debate_eval_llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        response_format=DebateCardSearchResult,
        parallel_tool_calls=None
    )

    # Agent setup
    argument_evaluator = ConversableAgent(
        name="argument_evaluator", 
        system_message=(
            f"You are an expert policy debater focused on finding the best possible counterplan text for a policy debate case. "
            f"Your job is to:\n"
            f"1. Consider all possible types of counterplans, including but not limited to Plan-Inclusive Counterplans (PICs), Advantage Counterplans, Exclusionary Counterplans, Consult/Condition Counterplans, and any other strategic counterplan options. "
            f"2. Break down the plan, advantages, and disadvantages to identify the most strategic and competitive counterplan text. "
            f"3. Guide evidence collection and counterplan drafting by:\n"
            f"   - Formulating extremely precise search queries that target counterplan texts which are directly competitive with the plan and relevant to the debate case. "
            f"   - Using BM25 search to find relevant counterplan texts from a debate evidence database (cutoff year 2022).\n"
            f"   - If you are being called after previous searches, you must significantly modify and refine your BM25 search queries to maximize the chance of finding new, more relevant, or more specific counterplan texts. Do not simply repeat or slightly alter previous queries—make substantial changes to your search approach, keywords, or focus.\n"
            f"   - Suggest query refinements to maximize the chance of finding counterplan texts that are directly competitive and strategic.\n"
            f"4. Evaluate counterplan quality for:\n"
            f"   - Direct, explicit competition with the plan (must not merely be tangentially related or generic alternatives).\n"
            f"   - Specificity: The counterplan text must clearly articulate what the counterplan does, and how it is distinct from the plan.\n"
            f"   - Strategic value and theoretical legitimacy (PICs, Advantage CPs, etc. are all on the table).\n"
            f"Reject any counterplan text that does not fully and directly compete with the plan or that could be interpreted as generic or non-specific. "
            f"Your goal is to find the strictest, most plan-relevant, and strategic counterplan text possible, ensuring that each selected counterplan is unique and not a duplicate of any previously included counterplan."
        ),
        llm_config=llm_config,
    )

    debate_eval_agent = ConversableAgent(
        name="debate_eval_agent",
        system_message=(
            f"You are an extremely selective and rigorous debate coach and argument analyst. "
            f"Your job is to strictly evaluate whether a counterplan text meets the highest standards for inclusion as a competitive counterplan in policy debate. "
            f"For each counterplan text, meticulously scrutinize its:\n"
            f"- Direct competition with the plan (must be a legitimate counterplan, not just a generic alternative)\n"
            f"- Strategic value (must provide a unique and compelling strategic option, not just a restatement of the plan or a trivial alternative)\n"
            f"- Specificity (must not duplicate or closely overlap with other selected counterplans, and must clearly articulate the counterplan's mechanism)\n"
            f"- Wording precision (must use exact terminology needed to establish the counterplan's competitiveness and strategic value)\n\n"
            f"After evaluating the counterplan, you must:\n"
            f"1. IMMEDIATELY REJECT (mark as 'False' and/or ignore) any counterplan that has already been marked as 'include_it' in previous iterations\n"
            f"2. Reject any counterplan that duplicates already selected counterplans\n"
            f"3. Ensure terminology precisely matches what's needed for the counterplan's competitiveness\n"
            f"4. Only approve counterplans that meet ALL evaluation criteria and are strictly plan-relevant and competitive\n\n"
            f"Your goal is to ensure we have the highest quality, plan-specific, and competitive counterplan text, with absolutely no duplicate or generic counterplans, and that all included counterplans are retagged and recut with precise, policy debate-style markup."
        ),
        llm_config=debate_eval_llm_config,
    )

    debate_search_agent = ConversableAgent(
        name="debate_search_agent",
        system_message="You are a helpful assistant that can search the debate evidence dataset for a given tag. Your query will retrieve a list of debate cards.",
        llm_config=required_llm_config,
    )

    executor_agent = ConversableAgent(
        name="executor_agent",
        human_input_mode="NEVER",
        llm_config=llm_config,
    )

    register_function(
        search_debate_cards,
        caller=debate_search_agent,
        executor=executor_agent,
        description="Search the debate evidence dataset using natural language queries. Return a list of debate cards.",
    )

    from autogen import GroupChat

    iterations = 0

    def custom_speaker_selection_func(last_speaker: Agent, groupchat: GroupChat):
        nonlocal iterations
        messages = groupchat.messages

        if len(messages) <= 1:
            return argument_evaluator

        if last_speaker is debate_search_agent:
            return executor_agent

        if last_speaker is executor_agent:
            return debate_eval_agent
        
        if last_speaker is debate_eval_agent:
            if "include_it" in messages[-1]["content"]:
                iterations += 1
                if iterations >= 3:
                    return None
                else:
                    return debate_search_agent
            else:
                return debate_search_agent

        if last_speaker is argument_evaluator:
            return debate_search_agent
        else:
            return "round_robin"

    group_chat = GroupChat(
        agents=[argument_evaluator, debate_search_agent, executor_agent, debate_eval_agent],
        messages=[],
        max_round=40,
        speaker_selection_method=custom_speaker_selection_func
    )

    group_chat_manager = GroupChatManager(
        groupchat=group_chat,
        llm_config=llm_config,
    )

    # Use the debate_case, which includes the debate topic, the plan, and the disadvantage, for context
    # Also include the current negative_case_html for additional context
    chat_result = argument_evaluator.initiate_chat(
        group_chat_manager,
        message=(
            f"{debate_case}\n\n"
            f"Current negative case:\n{negative_case_html}\n\n"
            "Assume that the current year is 2022.\n"
            f"Find the best, most plan-specific, competitive, and strategic counterplan text for the case articulated above. "
            f"Consider all types of counterplans, including Plan-Inclusive Counterplans (PICs), Advantage Counterplans, Exclusionary Counterplans, Consult/Condition Counterplans, and any other strategic options. "
            f"Only consider counterplan texts that directly and specifically compete with the plan and are relevant to the case. "
            f"Reject any counterplan text that is generic, tangential, or not relevant to the plan. "
        ),
    )

    # Parse the result
    counterplan_raw_string = chat_result.chat_history[-1]["content"]
    card_json = json.loads(counterplan_raw_string)["cards"][0]
    counterplan_id = card_json["id"]
    counterplan_text = card_json.get("counterplantext", "")
    # Get the card document (still used for other purposes, but not appended to HTML)
    counterplan_doc = get_document_by_id(counterplan_id)
    card_markup = str(counterplan_doc['markup'])

    # Append only the counterplan text to negative_case_html using h2, div, and p tags (do NOT append the card itself)
    negative_case_html += (
        f"\n<h2>Counterplan Text</h2>"
        f"\n<div><p>{counterplan_text}</p></div>"
    )
    return negative_case_html


In [ ]:
# negative_case_html = add_counterplan_text_to_case(debate_case, negative_case_html)

In [ ]:
# display(HTML(negative_case_html))

### Counterplan Solvency

In [ ]:
def add_counterplan_solvency_to_case(debate_case, negative_case_html):
    class DebateCard(BaseModel):
        id: int
        cite: str
        include_in_case: Literal["include_it", "False"]
        reason_to_include: str
        retagged_argument_as_read_outloud_in_the_debate_round: str  # Argument to be presented as the first card after the counterplan text in a debate round

    class DebateCardSearchResult(BaseModel):
        cards: List[DebateCard] = Field(..., min_items=1, max_items=1)

    llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        parallel_tool_calls=None
    )
    required_llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        tool_choice="required",
        temperature=1.0,
        top_p=0.9,
        parallel_tool_calls=None
    )
    debate_eval_llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        response_format=DebateCardSearchResult,
        parallel_tool_calls=None
    )

    argument_evaluator = ConversableAgent(
        name="argument_evaluator", 
        system_message=(
            "You are an expert policy debater focused on finding the best possible evidence of counterplan solvency in a policy debate case. "
            "The debate topic, plan, disadvantage, and counterplan have already been provided. "
            "Your job is to:\n"
            "1. Break down the plan, counterplan, and the relevant advantage/disadvantage into their key components and causal relationships.\n"
            "2. Guide evidence collection by:\n"
            "   - Formulating extremely precise search queries that target only evidence which directly and specifically demonstrates that the counterplan, if enacted, would solve the advantage(s) or mitigate the impact(s) claimed by the affirmative plan. "
            "   - Using BM25 search to find relevant cards from a debate evidence database.\n"
            "   - If you are being called after previous searches, you must significantly modify and refine your BM25 search queries to maximize the chance of finding new, more relevant, or more specific counterplan solvency evidence. Do not simply repeat or slightly alter previous queries—make substantial changes to your search approach, keywords, or focus.\n"
            "   - Suggest query refinements to maximize the chance of finding evidence that directly supports the counterplan's ability to solve the relevant advantage(s) or mitigate the impact(s).\n"
            "3. Evaluate evidence quality for:\n"
            "   - Direct, explicit support for the counterplan's solvency (evidence must not merely be tangentially related or generic background).\n"
            "   - Specificity: The evidence must establish that the counterplan, as proposed, will solve the advantage(s) or mitigate the impact(s) in question, and explain the mechanism by which this occurs.\n"
            "   - Empirical support and authoritativeness.\n"
            "Reject any evidence that does not fully and directly support the counterplan's solvency, or that could be interpreted as generic or non-specific. "
            "Your goal is to find the strictest, most counterplan- and advantage/disadvantage-relevant solvency evidence possible, ensuring that each selected card is unique and not a duplicate of any previously included evidence."
        ),
        llm_config=llm_config,
    )

    debate_eval_agent = ConversableAgent(
        name="debate_eval_agent",
        system_message=(
            "You are an extremely selective and rigorous debate coach and argument analyst. "
            "Your job is to strictly evaluate whether evidence meets the highest standards for inclusion as counterplan solvency evidence in policy debate. "
            "For each piece of evidence, meticulously scrutinize its:\n"
            "- Author qualifications (must be from recognized experts or authoritative sources)\n"
            "- Empirical basis (must be supported by concrete data and research)\n"
            "- Direct relevance (must precisely and explicitly support the counterplan's ability to solve the advantage(s) or mitigate the impact(s))\n"
            "- Strategic value (must provide unique and compelling support for the counterplan's solvency, not just generic background)\n"
            "- Specificity (must not duplicate or closely overlap with other selected evidence, and must establish that the counterplan, as proposed, will solve the advantage(s) or mitigate the impact(s) in question)\n"
            "- Wording precision (must use exact terminology needed to establish the counterplan's solvency)\n\n"
            "After evaluating the evidence, you must:\n"
            "1. IMMEDIATELY REJECT (mark as 'False' and/or ignore) any evidence that has already been marked as 'include_it' in previous iterations\n"
            "2. Reject any evidence that duplicates already selected cards\n"
            "3. Ensure terminology precisely matches what's needed for the counterplan's solvency chain\n"
            "4. Only approve evidence that meets ALL evaluation criteria and is strictly counterplan- and advantage/disadvantage-relevant for solvency\n\n"
            "Your goal is to ensure we have the highest quality, counterplan- and advantage/disadvantage-specific solvency evidence, with absolutely no duplicate or generic cards, and that all included evidence is retagged and recut with precise, policy debate-style markup."
        ),
        llm_config=debate_eval_llm_config,
    )

    debate_search_agent = ConversableAgent(
        name="debate_search_agent",
        system_message="You are a helpful assistant that can search the debate evidence dataset for a given tag. Your query will retrieve a list of debate cards.",
        llm_config=required_llm_config,
    )

    executor_agent = ConversableAgent(
        name="executor_agent",
        human_input_mode="NEVER",
        llm_config=llm_config,
    )

    register_function(
        search_debate_cards,
        caller=debate_search_agent,
        executor=executor_agent,
        description="Search the debate evidence dataset using natural language queries. Return a list of debate cards.",
    )

    from autogen import GroupChat

    iterations = 0

    def custom_speaker_selection_func(last_speaker: Agent, groupchat: GroupChat):
        nonlocal iterations
        messages = groupchat.messages

        if len(messages) <= 1:
            return argument_evaluator

        if last_speaker is debate_search_agent:
            return executor_agent

        if last_speaker is executor_agent:
            return debate_eval_agent
        
        if last_speaker is debate_eval_agent:
            if "include_it" in messages[-1]["content"]:
                iterations += 1
                print(f"iterations: {iterations}")
                if iterations >= 3:
                    return None
                else:
                    return debate_search_agent
            else:
                return debate_search_agent

        if last_speaker is argument_evaluator:
            return debate_search_agent
        else:
            return "round_robin"

    group_chat = GroupChat(
        agents=[argument_evaluator, debate_search_agent, executor_agent, debate_eval_agent],
        messages=[],
        max_round=40,
        speaker_selection_method=custom_speaker_selection_func
    )

    group_chat_manager = GroupChatManager(
        groupchat=group_chat,
        llm_config=llm_config,
    )

    # Use the debate_case, which includes the debate topic, the plan, the disadvantage, and the counterplan for context

    chat_result = argument_evaluator.initiate_chat(
        group_chat_manager,
        message=(
            f"{debate_case}\n\n"
            f"{negative_case_html}\n\n"
            "Assume that the current year is 2022.\n"
            "The counterplan has already been established above.\n"
            "Find the best and most counterplan- and advantage/disadvantage-specific evidence of counterplan solvency articulated above. "
            "Only consider evidence that directly and specifically supports the claim that the counterplan, if enacted, would solve the advantage(s) or mitigate the impact(s) described in the case. "
            "Reject any evidence that is generic, tangential, or not relevant to the counterplan's solvency. "
            # No recency or cutoff requirements; old evidence is acceptable if it is high quality.
        ),
    )

    counterplan_solvency_raw_string = chat_result.chat_history[-1]["content"]
    counterplan_solvency_json = json.loads(counterplan_solvency_raw_string)
    card_json = counterplan_solvency_json["cards"][0]
    counterplan_solvency_id = card_json["id"]
    retagged_argument = card_json.get("retagged_argument_as_read_outloud_in_the_debate_round", "")

    counterplan_solvency_doc = get_document_by_id(counterplan_solvency_id)
    card_markup = str(counterplan_solvency_doc['markup'])

    # Append to negative_case_html using h2, div, and p tags
    negative_case_html += (
        f"\n<h2>Counterplan Solvency</h2>"
        f"\n<div><p>{retagged_argument}</p></div>"
        f"\n<div><p>{card_markup}</p></div>"
    )
    return negative_case_html

In [ ]:
# negative_case_html = add_counterplan_solvency_to_case(debate_case, negative_case_html)

In [ ]:
# display(HTML(negative_case_html))

### Counterplan Net Benefit

In [ ]:
def add_counterplan_net_benefit_to_case(debate_case, negative_case_html):
    class DebateCard(BaseModel):
        id: int
        cite: str
        include_in_case: Literal["include_it", "False"]
        reason_to_include: str
        retagged_argument_as_read_outloud_in_the_debate_round: str  # Argument to be presented as the first card after the counterplan net benefit text in a debate round

    class DebateCardSearchResult(BaseModel):
        cards: List[DebateCard] = Field(..., min_items=1, max_items=1)

    llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        parallel_tool_calls=None
    )
    required_llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        tool_choice="required",
        temperature=1.0,
        top_p=0.9,
        parallel_tool_calls=None
    )
    debate_eval_llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        response_format=DebateCardSearchResult,
        parallel_tool_calls=None
    )

    argument_evaluator = ConversableAgent(
        name="argument_evaluator", 
        system_message=(
            "You are an expert policy debater focused on finding the best possible evidence of counterplan net benefits in a policy debate case. "
            "The debate topic, plan, disadvantage, and counterplan have already been provided. "
            "Your job is to:\n"
            "1. Break down the plan, counterplan, and the relevant advantage/disadvantage into their key components and causal relationships.\n"
            "2. Guide evidence collection by:\n"
            "   - Formulating extremely precise search queries that target only evidence which directly and specifically demonstrates that the counterplan is net beneficial compared to the plan (e.g., avoids a disadvantage, achieves a unique benefit, or is preferable to the plan for a specific reason). "
            "   - Using BM25 search to find relevant cards from a debate evidence database.\n"
            "   - If you are being called after previous searches, you must significantly modify and refine your BM25 search queries to maximize the chance of finding new, more relevant, or more specific counterplan net benefit evidence. Do not simply repeat or slightly alter previous queries—make substantial changes to your search approach, keywords, or focus.\n"
            "   - Suggest query refinements to maximize the chance of finding evidence that directly supports the counterplan's net benefit over the plan.\n"
            "3. Evaluate evidence quality for:\n"
            "   - Direct, explicit support for the counterplan's net benefit (evidence must not merely be tangentially related or generic background).\n"
            "   - Specificity: The evidence must establish that the counterplan, as proposed, is preferable to the plan, and explain the mechanism by which this occurs (e.g., avoids a disadvantage, achieves a unique benefit, etc.).\n"
            "   - Empirical support and authoritativeness.\n"
            "Reject any evidence that does not fully and directly support the counterplan's net benefit, or that could be interpreted as generic or non-specific. "
            "Your goal is to find the strictest, most counterplan- and advantage/disadvantage-relevant net benefit evidence possible, ensuring that each selected card is unique and not a duplicate of any previously included evidence."
        ),
        llm_config=llm_config,
    )

    debate_eval_agent = ConversableAgent(
        name="debate_eval_agent",
        system_message=(
            "You are an extremely selective and rigorous debate coach and argument analyst. "
            "Your job is to strictly evaluate whether evidence meets the highest standards for inclusion as counterplan net benefit evidence in policy debate. "
            "For each piece of evidence, meticulously scrutinize its:\n"
            "- Author qualifications (must be from recognized experts or authoritative sources)\n"
            "- Empirical basis (must be supported by concrete data and research)\n"
            "- Direct relevance (must precisely and explicitly support the counterplan's net benefit over the plan, such as avoiding a disadvantage, achieving a unique benefit, or being preferable for a specific reason)\n"
            "- Strategic value (must provide unique and compelling support for the counterplan's net benefit, not just generic background)\n"
            "- Specificity (must not duplicate or closely overlap with other selected evidence, and must establish that the counterplan, as proposed, is preferable to the plan in a specific, debate-relevant way)\n"
            "- Wording precision (must use exact terminology needed to establish the counterplan's net benefit)\n\n"
            "After evaluating the evidence, you must:\n"
            "1. IMMEDIATELY REJECT (mark as 'False' and/or ignore) any evidence that has already been marked as 'include_it' in previous iterations\n"
            "2. Reject any evidence that duplicates already selected cards\n"
            "3. Ensure terminology precisely matches what's needed for the counterplan's net benefit chain\n"
            "4. Only approve evidence that meets ALL evaluation criteria and is strictly counterplan- and advantage/disadvantage-relevant for net benefit\n\n"
            "Your goal is to ensure we have the highest quality, counterplan- and advantage/disadvantage-specific net benefit evidence, with absolutely no duplicate or generic cards, and that all included evidence is retagged and recut with precise, policy debate-style markup."
        ),
        llm_config=debate_eval_llm_config,
    )

    debate_search_agent = ConversableAgent(
        name="debate_search_agent",
        system_message="You are a helpful assistant that can search the debate evidence dataset for a given tag. Your query will retrieve a list of debate cards.",
        llm_config=required_llm_config,
    )

    executor_agent = ConversableAgent(
        name="executor_agent",
        human_input_mode="NEVER",
        llm_config=llm_config,
    )

    register_function(
        search_debate_cards,
        caller=debate_search_agent,
        executor=executor_agent,
        description="Search the debate evidence dataset using natural language queries. Return a list of debate cards.",
    )

    from autogen import GroupChat

    iterations = 0

    def custom_speaker_selection_func(last_speaker: Agent, groupchat: GroupChat):
        nonlocal iterations
        messages = groupchat.messages

        if len(messages) <= 1:
            return argument_evaluator

        if last_speaker is debate_search_agent:
            return executor_agent

        if last_speaker is executor_agent:
            return debate_eval_agent
        
        if last_speaker is debate_eval_agent:
            if "include_it" in messages[-1]["content"]:
                iterations += 1
                print(f"iterations: {iterations}")
                if iterations >= 3:
                    return None
                else:
                    return debate_search_agent
            else:
                return debate_search_agent

        if last_speaker is argument_evaluator:
            return debate_search_agent
        else:
            return "round_robin"

    group_chat = GroupChat(
        agents=[argument_evaluator, debate_search_agent, executor_agent, debate_eval_agent],
        messages=[],
        max_round=40,
        speaker_selection_method=custom_speaker_selection_func
    )

    group_chat_manager = GroupChatManager(
        groupchat=group_chat,
        llm_config=llm_config,
    )

    # Use the debate_case, which includes the debate topic, the plan, the disadvantage, and the counterplan for context

    chat_result = argument_evaluator.initiate_chat(
        group_chat_manager,
        message=(
            f"{debate_case}\n\n"
            f"{negative_case_html}\n\n"
            "Assume that the current year is 2022.\n"
            "The counterplan has already been established above.\n"
            "Find the best and most counterplan- and advantage/disadvantage-specific evidence of counterplan net benefit articulated above. "
            "Only consider evidence that directly and specifically supports the claim that the counterplan, if enacted, is net beneficial compared to the plan (e.g., avoids a disadvantage, achieves a unique benefit, or is preferable to the plan for a specific reason). "
            "Reject any evidence that is generic, tangential, or not relevant to the counterplan's net benefit. "
            # No recency or cutoff requirements; old evidence is acceptable if it is high quality.
        ),
    )

    counterplan_net_benefit_raw_string = chat_result.chat_history[-1]["content"]
    counterplan_net_benefit_json = json.loads(counterplan_net_benefit_raw_string)
    card_json = counterplan_net_benefit_json["cards"][0]
    counterplan_net_benefit_id = card_json["id"]
    retagged_argument = card_json.get("retagged_argument_as_read_outloud_in_the_debate_round", "")

    counterplan_net_benefit_doc = get_document_by_id(counterplan_net_benefit_id)
    card_markup = str(counterplan_net_benefit_doc['markup'])

    # Append to negative_case_html using h2, div, and p tags
    negative_case_html += (
        f"\n<h2>Counterplan Net Benefit</h2>"
        f"\n<div><p>{retagged_argument}</p></div>"
        f"\n<div><p>{card_markup}</p></div>"
    )
    return negative_case_html

In [ ]:
# negative_case_html = add_counterplan_net_benefit_to_case(debate_case, negative_case_html)

In [ ]:
# display(HTML(negative_case_html))

## Kritik

In [ ]:
# kritik_title = negative_case['kritiks'][0]['title']
# kritik_core_argument = negative_case['kritiks'][0]['core_argument']
# kritik_alternative_text = negative_case['kritiks'][0].get('alternative_text', '')
# negative_case_html += (
#     f"<h2>{kritik_title}</h2>\n"
#     f"<p>{kritik_core_argument}</p>\n"
#     f"<p><strong>Thus the Alternative:</strong> {kritik_alternative_text}</p>"
# )


### Kritik Link

In [ ]:
def add_kritik_link_to_case(debate_case, negative_case_html):
    class DebateCard(BaseModel):
        id: int
        cite: str
        include_in_case: Literal["include_it", "False"]
        reason_to_include: str
        retagged_argument_as_read_outloud_in_the_debate_round: str  # Argument to be presented as the first card after the plantext in a debate round

    class DebateCardSearchResult(BaseModel):
        cards: List[DebateCard] = Field(..., min_items=1, max_items=1)

    llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        parallel_tool_calls=None
    )
    required_llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        tool_choice="required",
        temperature=1.0,
        top_p=0.9,
        parallel_tool_calls=None
    )
    kritik_eval_llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        response_format=DebateCardSearchResult,
        parallel_tool_calls=None
    )

    argument_evaluator = ConversableAgent(
        name="kritik_argument_evaluator", 
        system_message=(
            "You are an expert policy debater focused on finding the best possible evidence of a kritik link in a policy debate case. "
            "The debate topic, plan, kritik core argument, and alternative have already been provided. "
            "Your job is to:\n"
            "1. Break down the plan and kritik (including the core argument and alternative) into their key components and relationships.\n"
            "2. Guide evidence collection by:\n"
            "   - Formulating extremely precise search queries that target only evidence which directly and specifically demonstrates the kritik's link—i.e., how the plan or its underlying assumptions/representations/epistemology/ontology/etc. cause or reproduce the harms or logic critiqued by the kritik. "
            "   - Using BM25 search to find relevant cards from a debate evidence database.\n"
            "   - If you are being called after previous searches, you must significantly modify and refine your BM25 search queries to maximize the chance of finding new, more relevant, or more specific kritik link evidence. Do not simply repeat or slightly alter previous queries—make substantial changes to your search approach, keywords, or focus.\n"
            "   - Suggest query refinements to maximize the chance of finding evidence that directly supports the kritik's link claim.\n"
            "3. Evaluate evidence quality for:\n"
            "   - Direct, explicit support for the kritik link claim (evidence must not merely be tangentially related or generic background).\n"
            "   - Specificity: The evidence must establish that the plan, as proposed, or its underlying logic, causes or perpetuates the harms or logic critiqued by the kritik, and explain the mechanism by which this occurs.\n"
            "   - Authoritativeness and theoretical sophistication.\n"
            "Reject any evidence that does not fully and directly support the kritik link, or that could be interpreted as generic or non-specific. "
            "Your goal is to find the strictest, most kritik- and plan-relevant link evidence possible, ensuring that each selected card is unique and not a duplicate of any previously included evidence."
        ),
        llm_config=llm_config,
    )

    kritik_eval_agent = ConversableAgent(
        name="kritik_eval_agent",
        system_message=(
            "You are an extremely selective and rigorous debate coach and kritik argument analyst. "
            "Your job is to strictly evaluate whether evidence meets the highest standards for inclusion as link evidence supporting the kritik in policy debate. "
            "For each piece of evidence, meticulously scrutinize its:\n"
            "- Author qualifications (must be from recognized experts or authoritative sources in critical theory, philosophy, or the relevant field)\n"
            "- Theoretical sophistication and relevance (must precisely and explicitly support the kritik's link claim)\n"
            "- Strategic value (must provide unique and compelling support for the kritik's link, not just generic background)\n"
            "- Specificity (must not duplicate or closely overlap with other selected evidence, and must establish that the plan, as proposed, or its underlying logic, causes or perpetuates the harms or logic critiqued by the kritik)\n"
            "- Wording precision (must use exact terminology needed to establish the kritik link)\n\n"
            "After evaluating the evidence, you must:\n"
            "1. IMMEDIATELY REJECT (mark as 'False' and/or ignore) any evidence that has already been marked as 'include_it' in previous iterations\n"
            "2. Reject any evidence that duplicates already selected cards\n"
            "3. Ensure terminology precisely matches what's needed for the kritik's link chains\n"
            "4. Only approve evidence that meets ALL evaluation criteria and is strictly kritik- and plan-relevant for the link\n\n"
            "Your goal is to ensure we have the highest quality, kritik- and plan-specific link evidence, with absolutely no duplicate or generic cards, and that all included evidence is retagged and recut with precise, policy debate-style markup."
        ),
        llm_config=kritik_eval_llm_config,
    )

    kritik_search_agent = ConversableAgent(
        name="kritik_search_agent",
        system_message="You are a helpful assistant that can search the debate evidence dataset for a given tag. Your query will retrieve a list of debate cards.",
        llm_config=required_llm_config,
    )

    executor_agent = ConversableAgent(
        name="executor_agent",
        human_input_mode="NEVER",
        llm_config=llm_config,
    )

    register_function(
        search_debate_cards,
        caller=kritik_search_agent,
        executor=executor_agent,
        description="Search the debate evidence dataset using natural language queries. Return a list of debate cards.",
    )

    from autogen import GroupChat

    iterations = 0

    def custom_speaker_selection_func(last_speaker: Agent, groupchat: GroupChat):
        nonlocal iterations
        messages = groupchat.messages

        if len(messages) <= 1:
            return argument_evaluator

        if last_speaker is kritik_search_agent:
            return executor_agent

        if last_speaker is executor_agent:
            return kritik_eval_agent
        
        if last_speaker is kritik_eval_agent:
            if "include_it" in messages[-1]["content"]:
                iterations += 1
                print(f"iterations: {iterations}")
                if iterations >= 3:
                    return None
                else:
                    return kritik_search_agent
            else:
                return kritik_search_agent

        if last_speaker is argument_evaluator:
            return kritik_search_agent
        else:
            return "round_robin"

    group_chat = GroupChat(
        agents=[argument_evaluator, kritik_search_agent, executor_agent, kritik_eval_agent],
        messages=[],
        max_round=40,
        speaker_selection_method=custom_speaker_selection_func
    )

    group_chat_manager = GroupChatManager(
        groupchat=group_chat,
        llm_config=llm_config,
    )

    # Use the debate_case, which includes the debate topic, the plan, and the kritik (core argument and alternative) for context

    chat_result = argument_evaluator.initiate_chat(
        group_chat_manager,
        message=(
            f"{debate_case}\n\n"
            f"Current negative case so far:\n{negative_case_html}\n\n"
            "Assume that the current year is 2022.\n"
            "The kritik core argument and alternative have already been established above.\n"
            "Find the best and most plan- and kritik-specific evidence of a causal link supporting the kritik articulated above. "
            "Only consider evidence that directly and specifically supports the claim that the plan, if enacted, or its underlying logic, causes or perpetuates the harms or logic critiqued by the kritik. "
            "Reject any evidence that is generic, tangential, or not relevant to the link between the plan and the kritik. "
            # No recency or cutoff requirements; old evidence is acceptable if it is high quality.
        ),
    )

    kritik_link_raw_string = chat_result.chat_history[-1]["content"]
    kritik_link_json = json.loads(kritik_link_raw_string)
    card_json = kritik_link_json["cards"][0]
    kritik_link_id = card_json["id"]
    retagged_argument = card_json.get("retagged_argument_as_read_outloud_in_the_debate_round", "")

    kritik_link_doc = get_document_by_id(kritik_link_id)
    card_markup = str(kritik_link_doc['markup'])

    # Append to negative_case_html using h2, div, and p tags
    negative_case_html += (
        f"\n<h2>Kritik Link</h2>"
        f"\n<div><p>{retagged_argument}</p></div>"
        f"\n<div><p>{card_markup}</p></div>"
    )
    return negative_case_html

In [ ]:
# negative_case_html = add_kritik_link_to_case(debate_case, negative_case_html)

In [ ]:
# display(HTML(negative_case_html))

### Kritik Impact

In [ ]:
def add_kritik_impact_to_case(debate_case, negative_case_html):
    class DebateCard(BaseModel):
        id: int
        cite: str
        include_in_case: Literal["include_it", "False"]
        reason_to_include: str
        retagged_argument_as_read_outloud_in_the_debate_round: str  # Argument to be presented as the first card after the plantext in a debate round

    class DebateCardSearchResult(BaseModel):
        cards: List[DebateCard] = Field(..., min_items=1, max_items=1)

    llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        parallel_tool_calls=None
    )
    required_llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        tool_choice="required",
        temperature=1.0,
        top_p=0.9,
        parallel_tool_calls=None
    )
    kritik_eval_llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        response_format=DebateCardSearchResult,
        parallel_tool_calls=None
    )

    argument_evaluator = ConversableAgent(
        name="kritik_impact_argument_evaluator", 
        system_message=(
            "You are an expert policy debater focused on finding the best possible evidence of a kritik impact in a policy debate case. "
            "The debate topic, plan, kritik core argument, and alternative have already been provided. "
            "Your job is to:\n"
            "1. Break down the kritik (including the core argument and alternative) into its key components and relationships, focusing on the ultimate impact or consequence of the kritik.\n"
            "2. Guide evidence collection by:\n"
            "   - Formulating extremely precise search queries that target only evidence which directly and specifically demonstrates the kritik's impact—i.e., the ultimate harms, consequences, or theoretical implications established by the kritik. "
            "   - Using BM25 search to find relevant cards from a debate evidence database.\n"
            "   - If you are being called after previous searches, you must significantly modify and refine your BM25 search queries to maximize the chance of finding new, more relevant, or more specific kritik impact evidence. Do not simply repeat or slightly alter previous queries—make substantial changes to your search approach, keywords, or focus.\n"
            "   - Suggest query refinements to maximize the chance of finding evidence that directly supports the kritik's impact claim.\n"
            "3. Evaluate evidence quality for:\n"
            "   - Direct, explicit support for the kritik impact claim (evidence must not merely be tangentially related or generic background).\n"
            "   - Specificity: The evidence must establish the ultimate harms, consequences, or theoretical implications of the kritik, and explain the mechanism by which these occur.\n"
            "   - Authoritativeness and theoretical sophistication.\n"
            "Reject any evidence that does not fully and directly support the kritik impact, or that could be interpreted as generic or non-specific. "
            "Your goal is to find the strictest, most kritik-relevant impact evidence possible, ensuring that each selected card is unique and not a duplicate of any previously included evidence."
        ),
        llm_config=llm_config,
    )

    kritik_eval_agent = ConversableAgent(
        name="kritik_impact_eval_agent",
        system_message=(
            "You are an extremely selective and rigorous debate coach and kritik argument analyst. "
            "Your job is to strictly evaluate whether evidence meets the highest standards for inclusion as impact evidence supporting the kritik in policy debate. "
            "For each piece of evidence, meticulously scrutinize its:\n"
            "- Author qualifications (must be from recognized experts or authoritative sources in critical theory, philosophy, or the relevant field)\n"
            "- Theoretical sophistication and relevance (must precisely and explicitly support the kritik's impact claim)\n"
            "- Strategic value (must provide unique and compelling support for the kritik's impact, not just generic background)\n"
            "- Specificity (must not duplicate or closely overlap with other selected evidence, and must establish the ultimate harms, consequences, or theoretical implications of the kritik)\n"
            "- Wording precision (must use exact terminology needed to establish the kritik impact)\n\n"
            "After evaluating the evidence, you must:\n"
            "1. IMMEDIATELY REJECT (mark as 'False' and/or ignore) any evidence that has already been marked as 'include_it' in previous iterations\n"
            "2. Reject any evidence that duplicates already selected cards\n"
            "3. Ensure terminology precisely matches what's needed for the kritik's impact chains\n"
            "4. Only approve evidence that meets ALL evaluation criteria and is strictly kritik-relevant for the impact\n\n"
            "Your goal is to ensure we have the highest quality, kritik-specific impact evidence, with absolutely no duplicate or generic cards, and that all included evidence is retagged and recut with precise, policy debate-style markup."
        ),
        llm_config=kritik_eval_llm_config,
    )

    kritik_search_agent = ConversableAgent(
        name="kritik_impact_search_agent",
        system_message="You are a helpful assistant that can search the debate evidence dataset for a given tag. Your query will retrieve a list of debate cards.",
        llm_config=required_llm_config,
    )

    executor_agent = ConversableAgent(
        name="executor_agent",
        human_input_mode="NEVER",
        llm_config=llm_config,
    )

    register_function(
        search_debate_cards,
        caller=kritik_search_agent,
        executor=executor_agent,
        description="Search the debate evidence dataset using natural language queries. Return a list of debate cards.",
    )

    from autogen import GroupChat

    iterations = 0

    def custom_speaker_selection_func(last_speaker: Agent, groupchat: GroupChat):
        nonlocal iterations
        messages = groupchat.messages

        if len(messages) <= 1:
            return argument_evaluator

        if last_speaker is kritik_search_agent:
            return executor_agent

        if last_speaker is executor_agent:
            return kritik_eval_agent
        
        if last_speaker is kritik_eval_agent:
            if "include_it" in messages[-1]["content"]:
                iterations += 1
                print(f"iterations: {iterations}")
                if iterations >= 3:
                    return None
                else:
                    return kritik_search_agent
            else:
                return kritik_search_agent

        if last_speaker is argument_evaluator:
            return kritik_search_agent
        else:
            return "round_robin"

    group_chat = GroupChat(
        agents=[argument_evaluator, kritik_search_agent, executor_agent, kritik_eval_agent],
        messages=[],
        max_round=40,
        speaker_selection_method=custom_speaker_selection_func
    )

    group_chat_manager = GroupChatManager(
        groupchat=group_chat,
        llm_config=llm_config,
    )

    # Use the debate_case, which includes the debate topic, the plan, and the kritik (core argument and alternative) for context

    chat_result = argument_evaluator.initiate_chat(
        group_chat_manager,
        message=(
            f"{debate_case}\n\n"
            f"Current negative_case_html:\n{negative_case_html}\n\n"
            "Assume that the current year is 2022.\n"
            "The kritik core argument and alternative have already been established above.\n"
            "Find the best and most kritik-specific evidence of the ultimate impact or consequence supporting the kritik articulated above. "
            "Only consider evidence that directly and specifically supports the claim about the harms, consequences, or theoretical implications established by the kritik. "
            "Reject any evidence that is generic, tangential, or not relevant to the impact of the kritik. "
            # No recency or cutoff requirements; old evidence is acceptable if it is high quality.
        ),
    )

    kritik_impact_raw_string = chat_result.chat_history[-1]["content"]
    kritik_impact_json = json.loads(kritik_impact_raw_string)
    card_json = kritik_impact_json["cards"][0]
    kritik_impact_id = card_json["id"]
    retagged_argument = card_json.get("retagged_argument_as_read_outloud_in_the_debate_round", "")

    kritik_impact_doc = get_document_by_id(kritik_impact_id)
    card_markup = str(kritik_impact_doc['markup'])

    # Append to negative_case_html using h2, div, and p tags
    negative_case_html += (
        f"\n<h2>Kritik Impact</h2>"
        f"\n<div><p>{retagged_argument}</p></div>"
        f"\n<div><p>{card_markup}</p></div>"
    )
    return negative_case_html

In [ ]:
# negative_case_html = add_kritik_impact_to_case(debate_case, negative_case_html)

In [ ]:
# display(HTML(negative_case_html))

### Kritik Role of the Ballot

In [ ]:
def add_kritik_role_of_ballot_to_case(debate_case, negative_case_html):
    class DebateCard(BaseModel):
        id: int
        cite: str
        include_in_case: Literal["include_it", "False"]
        reason_to_include: str
        retagged_argument_as_read_outloud_in_the_debate_round: str  # Argument to be presented as the first card after the plantext in a debate round

    class DebateCardSearchResult(BaseModel):
        cards: List[DebateCard] = Field(..., min_items=1, max_items=1)

    llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        parallel_tool_calls=None
    )
    required_llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        tool_choice="required",
        temperature=1.0,
        top_p=0.9,
        parallel_tool_calls=None
    )
    kritik_eval_llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        response_format=DebateCardSearchResult,
        parallel_tool_calls=None
    )

    argument_evaluator = ConversableAgent(
        name="kritik_role_of_ballot_argument_evaluator", 
        system_message=(
            "You are an expert policy debater focused on finding the best possible evidence for a kritik 'role of the ballot' argument in a policy debate case. "
            "The debate topic, plan, kritik core argument, and alternative have already been provided. "
            "Your job is to:\n"
            "1. Break down the kritik (including the core argument and alternative) into its key components and relationships, focusing on the 'role of the ballot'—i.e., what the judge's ballot should endorse or reject, and why.\n"
            "2. Guide evidence collection by:\n"
            "   - Formulating extremely precise search queries that target only evidence which directly and specifically establishes or justifies the kritik's proposed role of the ballot. "
            "   - Using BM25 search to find relevant cards from a debate evidence database.\n"
            "   - If you are being called after previous searches, you must significantly modify and refine your BM25 search queries to maximize the chance of finding new, more relevant, or more specific 'role of the ballot' evidence. Do not simply repeat or slightly alter previous queries—make substantial changes to your search approach, keywords, or focus.\n"
            "   - Suggest query refinements to maximize the chance of finding evidence that directly supports the kritik's role of the ballot claim.\n"
            "3. Evaluate evidence quality for:\n"
            "   - Direct, explicit support for the kritik's role of the ballot claim (evidence must not merely be tangentially related or generic background).\n"
            "   - Specificity: The evidence must establish what the judge's ballot should do in the context of the kritik, and explain the theoretical or strategic justification for that role.\n"
            "   - Authoritativeness and theoretical sophistication.\n"
            "Reject any evidence that does not fully and directly support the kritik's role of the ballot, or that could be interpreted as generic or non-specific. "
            "Your goal is to find the strictest, most kritik-relevant 'role of the ballot' evidence possible, ensuring that each selected card is unique and not a duplicate of any previously included evidence."
        ),
        llm_config=llm_config,
    )

    kritik_eval_agent = ConversableAgent(
        name="kritik_role_of_ballot_eval_agent",
        system_message=(
            "You are an extremely selective and rigorous debate coach and kritik argument analyst. "
            "Your job is to strictly evaluate whether evidence meets the highest standards for inclusion as 'role of the ballot' evidence supporting the kritik in policy debate. "
            "For each piece of evidence, meticulously scrutinize its:\n"
            "- Author qualifications (must be from recognized experts or authoritative sources in critical theory, philosophy, or the relevant field)\n"
            "- Theoretical sophistication and relevance (must precisely and explicitly support the kritik's role of the ballot claim)\n"
            "- Strategic value (must provide unique and compelling support for the kritik's role of the ballot, not just generic background)\n"
            "- Specificity (must not duplicate or closely overlap with other selected evidence, and must establish the theoretical or strategic justification for the kritik's role of the ballot)\n"
            "- Wording precision (must use exact terminology needed to establish the kritik's role of the ballot)\n\n"
            "After evaluating the evidence, you must:\n"
            "1. IMMEDIATELY REJECT (mark as 'False' and/or ignore) any evidence that has already been marked as 'include_it' in previous iterations\n"
            "2. Reject any evidence that duplicates already selected cards\n"
            "3. Ensure terminology precisely matches what's needed for the kritik's role of the ballot chains\n"
            "4. Only approve evidence that meets ALL evaluation criteria and is strictly kritik-relevant for the role of the ballot\n\n"
            "Your goal is to ensure we have the highest quality, kritik-specific 'role of the ballot' evidence, with absolutely no duplicate or generic cards, and that all included evidence is retagged and recut with precise, policy debate-style markup."
        ),
        llm_config=kritik_eval_llm_config,
    )

    kritik_search_agent = ConversableAgent(
        name="kritik_role_of_ballot_search_agent",
        system_message="You are a helpful assistant that can search the debate evidence dataset for a given tag. Your query will retrieve a list of debate cards.",
        llm_config=required_llm_config,
    )

    executor_agent = ConversableAgent(
        name="executor_agent",
        human_input_mode="NEVER",
        llm_config=llm_config,
    )

    register_function(
        search_debate_cards,
        caller=kritik_search_agent,
        executor=executor_agent,
        description="Search the debate evidence dataset using natural language queries. Return a list of debate cards.",
    )

    from autogen import GroupChat

    iterations = 0

    def custom_speaker_selection_func(last_speaker: Agent, groupchat: GroupChat):
        nonlocal iterations
        messages = groupchat.messages

        if len(messages) <= 1:
            return argument_evaluator

        if last_speaker is kritik_search_agent:
            return executor_agent

        if last_speaker is executor_agent:
            return kritik_eval_agent
        
        if last_speaker is kritik_eval_agent:
            if "include_it" in messages[-1]["content"]:
                iterations += 1
                print(f"iterations: {iterations}")
                if iterations >= 3:
                    return None
                else:
                    return kritik_search_agent
            else:
                return kritik_search_agent

        if last_speaker is argument_evaluator:
            return kritik_search_agent
        else:
            return "round_robin"

    group_chat = GroupChat(
        agents=[argument_evaluator, kritik_search_agent, executor_agent, kritik_eval_agent],
        messages=[],
        max_round=40,
        speaker_selection_method=custom_speaker_selection_func
    )

    group_chat_manager = GroupChatManager(
        groupchat=group_chat,
        llm_config=llm_config,
    )

    # Use the debate_case, which includes the debate topic, the plan, and the kritik (core argument and alternative) for context

    chat_result = argument_evaluator.initiate_chat(
        group_chat_manager,
        message=(
            f"{debate_case}\n\n"
            f"{negative_case_html}\n\n"
            "Assume that the current year is 2022.\n"
            "The kritik core argument and alternative have already been established above.\n"
            "Find the best and most kritik-specific evidence justifying the kritik's proposed role of the ballot. "
            "Only consider evidence that directly and specifically supports the claim about what the judge's ballot should endorse or reject, and the theoretical or strategic justification for that role, as established by the kritik. "
            "Reject any evidence that is generic, tangential, or not relevant to the kritik's role of the ballot. "
            # No recency or cutoff requirements; old evidence is acceptable if it is high quality.
        ),
    )

    kritik_role_of_ballot_raw_string = chat_result.chat_history[-1]["content"]
    kritik_role_of_ballot_json = json.loads(kritik_role_of_ballot_raw_string)
    card_json = kritik_role_of_ballot_json["cards"][0]
    kritik_role_of_ballot_id = card_json["id"]
    retagged_argument = card_json.get("retagged_argument_as_read_outloud_in_the_debate_round", "")

    kritik_role_of_ballot_doc = get_document_by_id(kritik_role_of_ballot_id)
    card_markup = str(kritik_role_of_ballot_doc['markup'])

    # Append to negative_case_html using h2, div, and p tags
    negative_case_html += (
        f"\n<h2>Kritik Role of the Ballot</h2>"
        f"\n<div><p>{retagged_argument}</p></div>"
        f"\n<div><p>{card_markup}</p></div>"
    )
    return negative_case_html

In [ ]:
# negative_case_html = add_kritik_role_of_ballot_to_case(debate_case, negative_case_html)

In [ ]:
# display(HTML(negative_case_html))

## On Case Rebuttals

In [ ]:
def add_on_case_rebuttal_to_case(debate_case, negative_case_html):
    class DebateCard(BaseModel):
        id: int
        cite: str
        include_in_case: Literal["include_it", "False"]
        reason_to_include: str
        retagged_argument_as_read_outloud_in_the_debate_round: str  # Argument to be presented as the first card after the plantext in a debate round

    class DebateCardSearchResult(BaseModel):
        cards: List[DebateCard] = Field(..., min_items=1, max_items=1)

    llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        parallel_tool_calls=None
    )
    required_llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        tool_choice="required",
        temperature=1.0,
        top_p=0.9,
        parallel_tool_calls=None
    )
    debate_eval_llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        response_format=DebateCardSearchResult,
        parallel_tool_calls=None
    )

    argument_evaluator = ConversableAgent(
        name="argument_evaluator", 
        system_message=(
            "You are an expert policy debater focused on attacking the affirmative's case ON-CASE. "
            "The debate topic, plan, and the full affirmative case (including all evidence and tags) have already been provided. "
            "Your job is to:\n"
            "1. Identify a single, specific piece of evidence (card) from the affirmative's case that is most strategic to attack (e.g., a key internal link, impact, or advantage card). "
            "   - IMPORTANT: Clearly signpost and indicate the specific part of the case and the specific name (tag or cite) of the affirmative evidence (card) you are refuting. "
            "   - Do NOT select a card that has already been refuted by any previous on-case rebuttal (if an on-case rebuttal to that card already exists in the negative_case_html, pick a different card).\n"
            "2. Guide evidence collection by:\n"
            "   - Formulating extremely precise search queries that target only evidence which directly and specifically turns, answers, or provides defense against that specific affirmative card. "
            "   - Using BM25 search to find relevant cards from a debate evidence database.\n"
            "   - If you are being called after previous searches, you must significantly modify and refine your BM25 search queries to maximize the chance of finding new, more relevant, or more specific on-case rebuttal evidence. Do not simply repeat or slightly alter previous queries—make substantial changes to your search approach, keywords, or focus.\n"
            "   - Suggest query refinements to maximize the chance of finding evidence that directly answers or turns the targeted affirmative card.\n"
            "3. Evaluate evidence quality for:\n"
            "   - Direct, explicit clash with the targeted affirmative card (evidence must not merely be tangentially related or generic background).\n"
            "   - Specificity: The evidence must directly answer, turn, or provide defense against the specific claim or warrant in the targeted affirmative card, and explain the mechanism by which this occurs.\n"
            "   - Empirical support and authoritativeness.\n"
            "Reject any evidence that does not fully and directly answer, turn, or defend against the targeted affirmative card, or that could be interpreted as generic or non-specific. "
            "Your goal is to find the strictest, most affirmative-evidence-specific on-case rebuttal possible, ensuring that each selected card is unique and not a duplicate of any previously included evidence. "
            "In your output, always clearly signpost and indicate the specific part of the case and the specific name (tag or cite) of the affirmative evidence being refuted."
        ),
        llm_config=llm_config,
    )

    debate_eval_agent = ConversableAgent(
        name="debate_eval_agent",
        system_message=(
            "You are an extremely selective and rigorous debate coach and argument analyst. "
            "Your job is to strictly evaluate whether evidence meets the highest standards for inclusion as ON-CASE rebuttal evidence against the affirmative's case. "
            "For each piece of evidence, meticulously scrutinize its:\n"
            "- Author qualifications (must be from recognized experts or authoritative sources)\n"
            "- Empirical basis (must be supported by concrete data and research)\n"
            "- Direct relevance (must precisely and explicitly answer, turn, or defend against the targeted affirmative card)\n"
            "- Strategic value (must provide unique and compelling clash with the affirmative's evidence, not just generic background)\n"
            "- Specificity (must not duplicate or closely overlap with other selected evidence, and must directly address the specific claim or warrant in the targeted affirmative card)\n"
            "- Wording precision (must use exact terminology needed to establish the on-case rebuttal)\n\n"
            "After evaluating the evidence, you must:\n"
            "1. IMMEDIATELY REJECT (mark as 'False' and/or ignore) any evidence that has already been marked as 'include_it' in previous iterations\n"
            "2. Reject any evidence that duplicates already selected cards\n"
            "3. Ensure terminology precisely matches what's needed for the on-case rebuttal\n"
            "4. Only approve evidence that meets ALL evaluation criteria and is strictly on-case and affirmative-evidence-specific\n\n"
            "Your goal is to ensure we have the highest quality, most specific on-case rebuttal evidence, with absolutely no duplicate or generic cards, and that all included evidence is retagged and recut with precise, policy debate-style markup."
        ),
        llm_config=debate_eval_llm_config,
    )

    debate_search_agent = ConversableAgent(
        name="debate_search_agent",
        system_message="You are a helpful assistant that can search the debate evidence dataset for a given tag. Your query will retrieve a list of debate cards.",
        llm_config=required_llm_config,
    )

    executor_agent = ConversableAgent(
        name="executor_agent",
        human_input_mode="NEVER",
        llm_config=llm_config,
    )

    register_function(
        search_debate_cards,
        caller=debate_search_agent,
        executor=executor_agent,
        description="Search the debate evidence dataset using natural language queries. Return a list of debate cards.",
    )

    from autogen import GroupChat

    iterations = 0

    def custom_speaker_selection_func(last_speaker: Agent, groupchat: GroupChat):
        nonlocal iterations
        messages = groupchat.messages

        if len(messages) <= 1:
            return argument_evaluator

        if last_speaker is debate_search_agent:
            return executor_agent

        if last_speaker is executor_agent:
            return debate_eval_agent
        
        if last_speaker is debate_eval_agent:
            if "include_it" in messages[-1]["content"]:
                iterations += 1
                print(f"iterations: {iterations}")
                if iterations >= 3:
                    return None
                else:
                    return debate_search_agent
            else:
                return debate_search_agent

        if last_speaker is argument_evaluator:
            return debate_search_agent
        else:
            return "round_robin"

    group_chat = GroupChat(
        agents=[argument_evaluator, debate_search_agent, executor_agent, debate_eval_agent],
        messages=[],
        max_round=40,
        speaker_selection_method=custom_speaker_selection_func
    )

    group_chat_manager = GroupChatManager(
        groupchat=group_chat,
        llm_config=llm_config,
    )

    # Use the debate_case, which includes the debate topic, the plan, and the full affirmative case (including all evidence and tags) for context

    chat_result = argument_evaluator.initiate_chat(
        group_chat_manager,
        message=(
            f"{debate_case}\n\n"
            f"Current negative case rebuttals (for context, do not repeat cards already refuted):\n{negative_case_html}\n\n"
            "Assume that the current year is 2022.\n"
            "The full affirmative case, including all evidence and tags, has already been established above.\n"
            "Identify a single, specific piece of affirmative evidence (card) that is most strategic to attack. "
            "You must clearly signpost and indicate the specific part of the case and the specific name (tag or cite) of the affirmative evidence (card) you are refuting. "
            "Do NOT select a card that has already been refuted by any previous on-case rebuttal (if an on-case rebuttal to that card already exists in the negative_case_html, pick a different card). "
            "Find the best and most specific evidence that directly answers, turns, or provides defense against that specific affirmative card. "
            "Only consider evidence that directly and specifically clashes with the claim, warrant, or impact of the targeted affirmative card. "
            "Reject any evidence that is generic, tangential, or not relevant to the specific on-case rebuttal. "
            # No recency or cutoff requirements; old evidence is acceptable if it is high quality.
        ),
    )

    on_case_rebuttal_raw_string = chat_result.chat_history[-1]["content"]
    on_case_rebuttal_json = json.loads(on_case_rebuttal_raw_string)
    card_json = on_case_rebuttal_json["cards"][0]
    on_case_rebuttal_id = card_json["id"]
    retagged_argument = card_json.get("retagged_argument_as_read_outloud_in_the_debate_round", "")

    on_case_rebuttal_doc = get_document_by_id(on_case_rebuttal_id)
    card_markup = str(on_case_rebuttal_doc['markup'])

    # Append to negative_case_html using h2, div, and p tags
    negative_case_html += (
        f"\n<h2>On-Case Rebuttal</h2>"
        f"\n<div><p>{retagged_argument}</p></div>"
        f"\n<div><p>{card_markup}</p></div>"
    )
    return negative_case_html

In [ ]:
# for _ in range(1):
#     negative_case_html = add_on_case_rebuttal_to_case(debate_case, negative_case_html)

In [ ]:
# display(HTML(negative_case_html))

# 1NC

In [ ]:
def try_func(func, *args, **kwargs):
    last_exception = None
    for attempt in range(3):
        try:
            return func(*args, **kwargs)
        except Exception as e:
            last_exception = e
            if attempt < 2:
                continue
            else:
                raise
    raise last_exception

negative_case = try_func(generate_negative_offcase, debate_case)

topicality = negative_case.get('topicality', {})
topicality_title = topicality.get('title')
topicality_core_argument = topicality.get('core_argument_summary_as_spoken_outloud_in_debate_round')

if topicality_title and topicality_core_argument:
    negative_case_html = f"<h2>{topicality_title}</h2>\n<p>{topicality_core_argument}</p>"
    negative_case_html = try_func(add_topicality_interpretation_and_evidence, debate_case, negative_case_html)
    negative_case_html = try_func(add_topicality_violation, debate_case, negative_case_html)
    negative_case_html = try_func(add_topicality_reasons_to_prefer_and_evidence, debate_case, negative_case_html)


theory = negative_case.get('theory', {})
theory_title = theory.get('title')
theory_core_argument = theory.get('core_argument_summary_as_spoken_outloud_in_debate_round')

if theory_title and theory_core_argument:
    negative_case_html += f"<h2>{theory_title}</h2>\n<p>{theory_core_argument}</p>"
    negative_case_html = try_func(add_theory_interpretation_and_evidence, debate_case, negative_case_html)
    negative_case_html = try_func(add_theory_violation_and_grounding_evidence, debate_case, negative_case_html)
    negative_case_html = try_func(add_theory_reasons_to_prefer_and_evidence, debate_case, negative_case_html)


disadvantage_title = negative_case['disadvantages'][0]['title']
disadvantage_core_argument = negative_case['disadvantages'][0]['core_argument_summary_as_spoken_outloud_in_debate_round']
negative_case_html += f"<h2>{disadvantage_title}</h2>\n<p>{disadvantage_core_argument}</p>"
negative_case_html = try_func(add_disadvantage_uniqueness_to_case, debate_case, negative_case_html)
negative_case_html = try_func(add_disadvantage_link_to_case, debate_case, negative_case_html)
negative_case_html = try_func(add_disadvantage_internal_link_to_case, debate_case, negative_case_html)
negative_case_html = try_func(add_disadvantage_impact_to_case, debate_case, negative_case_html)


negative_case_html = try_func(add_counterplan_text_to_case, debate_case, negative_case_html)
negative_case_html = try_func(add_counterplan_solvency_to_case, debate_case, negative_case_html)
negative_case_html = try_func(add_counterplan_net_benefit_to_case, debate_case, negative_case_html)

kritik_title = negative_case['kritiks'][0]['title']
kritik_core_argument = negative_case['kritiks'][0]['core_argument_summary_as_spoken_outloud_in_debate_round']
kritik_alternative_text = negative_case['kritiks'][0].get('alternative_text', '')
negative_case_html += (
    f"<h2>{kritik_title}</h2>\n"
    f"<p>{kritik_core_argument}</p>\n"
    f"<p><strong>Thus the Alternative:</strong> {kritik_alternative_text}</p>"
)

negative_case_html = try_func(add_kritik_link_to_case, debate_case, negative_case_html)
negative_case_html = try_func(add_kritik_impact_to_case, debate_case, negative_case_html)
negative_case_html = try_func(add_kritik_role_of_ballot_to_case, debate_case, negative_case_html)

for _ in range(3):
    negative_case_html = try_func(add_on_case_rebuttal_to_case, debate_case, negative_case_html)

In [ ]:
display(HTML(negative_case_html))

In [ ]:
negative_case_html

## Cross Examination (of the 1NC)

In [ ]:
def simulate_1ac_cross_examination_of_1nc(debate_case: str, negative_case_html: str) -> str:
    """
    Simulates the cross-examination of the 1NC by the affirmative.
    Takes both the 1AC (debate_case) and the 1NC (negative_case_html) as input,
    and returns a formatted HTML string of the cross-examination.
    """
    from typing import List
    from pydantic import BaseModel, Field

    # Define the structure for a cross-examination question and answer
    class CrossExPair(BaseModel):
        affirmative_question: str
        negative_response: str

    class CrossExamination(BaseModel):
        cross_ex: List[CrossExPair] = Field(..., min_items=7, max_items=7)

    # LLM config for all agents
    llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        parallel_tool_calls=None
    )

    cross_ex_llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        response_format=CrossExamination,
        parallel_tool_calls=None
    )

    # Agent 1: Affirmative asks questions
    affirmative_cross_ex_agent = ConversableAgent(
        name="affirmative_cross_ex_agent",
        system_message=(
            "You are the 1AC (affirmative) debater in a policy debate cross-examination. "
            "Your job is to ask sharp, strategic, and challenging questions about the 1NC (negative case) just presented. "
            "Focus on exposing weaknesses, ambiguities, or assumptions in the negative's theory, disadvantages, counterplans, and kritiks. "
            "Ask one question at a time, and wait for the negative to answer before asking the next. "
            "Do not answer your own questions. "
            "Be concise and direct. "
            "Do not repeat questions. "
            "You will ask a total of 3 to 7 questions."
        ),
        llm_config=llm_config,
    )

    # Agent 2: Negative answers
    negative_cross_ex_agent = ConversableAgent(
        name="negative_cross_ex_agent",
        system_message=(
            "You are the 1NC (negative) debater being cross-examined by the 1AC (affirmative) in a policy debate. "
            "Your job is to answer each question as clearly, persuasively, and strategically as possible, defending the negative case. "
            "Respond directly to the affirmative's question, but do not volunteer extra information. "
            "Be concise and avoid rambling. "
            "Do not ask questions yourself."
        ),
        llm_config=llm_config,
    )

    # Agent 3: Cross-ex summary agent (outputs the structured Q&A)
    cross_ex_summary_agent = ConversableAgent(
        name="cross_ex_summary_agent",
        system_message=(
            "You are a debate judge summarizing the 1AC's cross-examination of the 1NC. "
            "Your job is to produce a structured list of question/answer pairs, each with an 'affirmative_question' and a 'negative_response', "
            "covering the full cross-examination as it occurred. "
            "Return the result as a list of 3 to 7 question/answer pairs, each clearly labeled."
        ),
        llm_config=cross_ex_llm_config,
    )

    from autogen import GroupChat

    cross_ex_iterations = 0  # Track the number of Q&A iterations

    def cross_ex_speaker_selection(last_speaker, groupchat):
        nonlocal cross_ex_iterations
        # Alternate between affirmative and negative, then finish with summary agent
        if cross_ex_iterations == 0 and last_speaker is None:
            return affirmative_cross_ex_agent
        if last_speaker is affirmative_cross_ex_agent:
            return negative_cross_ex_agent
        if last_speaker is negative_cross_ex_agent:
            cross_ex_iterations += 1
            if cross_ex_iterations >= 7:
                return cross_ex_summary_agent
            else:
                return affirmative_cross_ex_agent
        if last_speaker is cross_ex_summary_agent:
            return None
        return "round_robin"

    group_chat = GroupChat(
        agents=[affirmative_cross_ex_agent, negative_cross_ex_agent, cross_ex_summary_agent],
        messages=[],
        max_round=40,
        speaker_selection_method=cross_ex_speaker_selection
    )

    group_chat_manager = GroupChatManager(
        groupchat=group_chat,
        llm_config=llm_config,
    )

    # The context for the cross-examination is both the 1AC and the 1NC
    chat_result = negative_cross_ex_agent.initiate_chat(
        group_chat_manager,
        message=(
            f"You are about to begin the 1AC's cross-examination of the 1NC. "
            f"The 1AC is as follows:\n\n{debate_case}\n\n"
            f"The 1NC is as follows:\n\n{negative_case_html}\n\n"
            "Begin by asking your first question."
        ),
    )

    # The summary agent's output is the last message in the chat history
    cross_ex_json = chat_result.chat_history[-1]["content"]
    cross_ex_data = json.loads(cross_ex_json)
    cross_ex_pairs = cross_ex_data["cross_ex"]

    # Format as HTML for display
    html = "<h2>1AC Cross-Examination of the 1NC</h2>\n"
    for i, pair in enumerate(cross_ex_pairs, 1):
        html += f"<div><b>Affirmative Question {i}:</b> {pair['affirmative_question']}</div>\n"
        html += f"<div><b>Negative Response {i}:</b> {pair['negative_response']}</div>\n"
        html += "<br/>\n"

    return html

# For compatibility with the rest of the code, assign to 1nc_crossex_html
# Usage: 1nc_crossex_html = simulate_1ac_cross_examination_of_1nc(debate_case, negative_case_html)

In [ ]:
attempts = 0
max_attempts = 3
while True:
    try:
        one_ac_crossx = simulate_1ac_cross_examination_of_1nc(debate_case, negative_case_html)
        break
    except Exception as e:
        attempts += 1
        if attempts >= max_attempts:
            raise

In [ ]:
display(HTML(one_ac_crossx))

In [ ]:
negative_case_html = negative_case_html + one_ac_crossx

# 2AC

### 2AC Gather Cards

In [ ]:
def add_2ac_evidence_to_case(debate_case, negative_case_html):
    """
    Simulates the affirmative constructing the 2AC.
    Focuses on gathering new 2AC cards that answer the arguments the 1AC is most vulnerable to given the 1NC.
    The selected cards, alongside the 1AC, should give the 2AC what it needs to win the debate round.
    This version does NOT write a full 2AC rebuttal speech; it only selects and presents the new 2AC cards.
    All cards must support the 1AC and the affirmative position.
    """
 
    class DebateCard(BaseModel):
        id: int
        cite: str
        include_in_case: Literal["include_it", "False"]
        reason_to_include: str
        retagged_argument_as_read_outloud_in_the_debate_round: str

    class DebateCardSearchResult(BaseModel):
        cards: List[DebateCard] = Field(..., min_items=1, max_items=7)

    llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        parallel_tool_calls=None
    )
    required_llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        tool_choice="required",
        temperature=1.0,
        top_p=0.9,
        parallel_tool_calls=None
    )
    debate_eval_llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        response_format=DebateCardSearchResult,
        parallel_tool_calls=None
    )

    # 2AC Agent: Focused on gathering the most strategic, high-impact 2AC cards
    two_ac_agent = ConversableAgent(
        name="two_ac_agent",
        system_message=(
            "You are an expert affirmative policy debater preparing the 2AC. "
            "You have access to the full debate_case (including the 1AC) and the negative_case_html (the 1NC, including all off-case positions: theory, disadvantages, counterplans, kritiks, etc). "
            "Your job is to:\n"
            "1. Carefully read and analyze all 1NC off-case positions (theory, disadvantages, counterplans, kritiks, etc) and the 1AC.\n"
            "2. Identify which 1NC arguments the 1AC is most vulnerable to—these are the arguments that, if left unanswered or insufficiently answered, would most likely cause the affirmative to lose the debate round.\n"
            "3. For each of these most threatening 1NC arguments, research and select the most strategic, high-quality, unique cards (evidence) that directly answer and refute those arguments, while also further entrenching and extending the 1AC's core claims. "
            "Each card must be:\n"
            "- Directly responsive to a specific 1NC argument that poses a significant threat to the 1AC\n"
            "- Not duplicative of any previous 1AC or 1NC card (do NOT select any card that is already in the 1AC or 1NC)\n"
            "- Clearly marked with its cite and a retagged argument as it would be read outloud in the debate round\n"
            "- Accompanied by a reason to include it in the 2AC, specifically explaining how it helps the 2AC win the round against the most dangerous 1NC arguments\n"
            "- Most importantly, every card you select must support the 1AC and the affirmative position. Do not select any card that undermines or contradicts the affirmative case or the 1AC's core claims.\n"
            "Do NOT write a 2AC rebuttal speech. Only select and present the new 2AC cards with their tags, cites, and reasons to include.\n"
            "Prioritize quality and strategic value over quantity: select only as many cards as are necessary to decisively answer the 1NC's most dangerous arguments and secure a winning position for the 2AC."
        ),
        llm_config=llm_config,
    )

    # 2AC Evidence Evaluator: Ensures only the most strategic, responsive cards are included
    two_ac_eval_agent = ConversableAgent(
        name="two_ac_eval_agent",
        system_message=(
            "You are a highly rigorous debate coach and argument analyst. "
            "Your job is to strictly evaluate whether each piece of evidence proposed for the 2AC meets the highest standards for inclusion:\n"
            "- Is it directly responsive to a 1NC argument that the 1AC is most vulnerable to?\n"
            "- Is it unique (not duplicative of any 1AC or 1NC card—do NOT approve any card that is already in the 1AC or 1NC)?\n"
            "- Is it authoritative and empirically supported?\n"
            "- Is it strategically valuable for the 2AC, meaning it helps the 2AC win the round against the most dangerous 1NC arguments?\n"
            "- Is it clearly retagged and recut for 2AC use?\n"
            "- Most importantly, does it support the 1AC and the affirmative position? Reject any card that undermines or contradicts the affirmative case or the 1AC's core claims.\n"
            "Reject any card that does not meet all criteria. Only approve cards that are directly responsive to the 1NC's most threatening arguments, unique, strategically valuable for the 2AC, and affirm the 1AC. Do NOT approve any card that is already in the 1AC or 1NC."
        ),
        llm_config=debate_eval_llm_config,
    )

    # Search agent for evidence
    debate_search_agent = ConversableAgent(
        name="debate_search_agent",
        system_message="You are a helpful assistant that can search the debate evidence dataset for a given tag. Your query will retrieve a list of debate cards.",
        llm_config=required_llm_config,
    )

    executor_agent = ConversableAgent(
        name="executor_agent",
        human_input_mode="NEVER",
        llm_config=llm_config,
    )

    register_function(
        search_debate_cards,
        caller=debate_search_agent,
        executor=executor_agent,
        description="Search the debate evidence dataset using natural language queries. Return a list of debate cards.",
    )

    from autogen import GroupChat

    iterations = 0
    max_iterations = 3  # Allow for more cards if needed, but focus on quality over quantity

    def custom_speaker_selection_func(last_speaker: Agent, groupchat: GroupChat):
        nonlocal iterations
        messages = groupchat.messages

        if len(messages) <= 1:
            return two_ac_agent

        if last_speaker is debate_search_agent:
            return executor_agent

        if last_speaker is executor_agent:
            return two_ac_eval_agent

        if last_speaker is two_ac_eval_agent:
            if "include_it" in messages[-1]["content"]:
                iterations += 1
                if iterations >= max_iterations:
                    return None
                else:
                    return debate_search_agent
            else:
                return debate_search_agent

        if last_speaker is two_ac_agent:
            return debate_search_agent
        else:
            return "round_robin"

    group_chat = GroupChat(
        agents=[two_ac_agent, debate_search_agent, executor_agent, two_ac_eval_agent],
        messages=[],
        max_round=60,
        speaker_selection_method=custom_speaker_selection_func
    )

    group_chat_manager = GroupChatManager(
        groupchat=group_chat,
        llm_config=llm_config,
    )

    # Use the debate_case (HTML, including 1AC) and negative_case_html (HTML, the 1NC) for context
    chat_result = two_ac_agent.initiate_chat(
        group_chat_manager,
        message=(
            f"{debate_case}\n\n"
            f"Negative Case (1NC):\n{negative_case_html}\n\n"
            "Assume that the current year is 2022.\n"
            "You are the 2AC. Your job is to:\n"
            "- Extend all 1AC cards and arguments\n"
            "- Identify which 1NC arguments the 1AC is most vulnerable to and which are most likely to decide the round if left unanswered\n"
            "- For each of these most threatening 1NC arguments, research and present the most strategic, high-quality, unique cards that directly answer and refute them, while also further entrenching and extending the 1AC's core claims\n"
            "- For each card, provide its cite, a reason to include (explaining how it helps the 2AC win the round against the most dangerous 1NC arguments), and a retagged argument as it would be read outloud in the debate round\n"
            "- Most importantly, every card you select must support the 1AC and the affirmative position. Do not select any card that undermines or contradicts the affirmative case or the 1AC's core claims.\n"
            "- Do NOT select any card that is already in the 1AC or 1NC. All 2AC cards must be new and not previously used in the 1AC or 1NC.\n"
            "Do NOT write a 2AC rebuttal speech. Only select and present the new 2AC cards with their tags, cites, and reasons to include.\n"
            "Prioritize quality and strategic value over quantity: select only as many cards as are necessary to decisively answer the 1NC's most dangerous arguments and secure a winning position for the 2AC."
        ),
    )

    two_ac_result_raw = chat_result.chat_history[-1]["content"]
    two_ac_result_json = json.loads(two_ac_result_raw)
    cards = two_ac_result_json["cards"]

    # Build the 2AC HTML string
    two_ac_html = "<div class='two-ac-section'>\n"
    two_ac_html += "<h1>2AC</h1>\n"

    # Add each new 2AC card to the 2AC HTML
    for idx, card_json in enumerate(cards):
        card_id = card_json["id"]
        retagged_argument = card_json.get("retagged_argument_as_read_outloud_in_the_debate_round", "")
        reason_to_include = card_json.get("reason_to_include", "")
        card_doc = get_document_by_id(card_id)
        card_markup = str(card_doc['markup'])
        two_ac_html += (
            f"\n<h2>2AC Card {idx+1}</h2>"
            f"\n<div><p><strong></strong> {retagged_argument}</p></div>"
            f"\n<div><p>{card_markup}</p></div>"
        )

    two_ac_html += "\n</div>"

    return two_ac_html

In [ ]:
ac_case = None
last_exception = None
for attempt in range(3):
    try:
        ac_case = add_2ac_evidence_to_case(debate_case, negative_case_html)
        break
    except Exception as e:
        last_exception = e
        if attempt == 2:
            raise

In [ ]:
display(HTML(ac_case))

### 2AC: Write Speech

In [ ]:
def add_2ac_to_case(debate_case, negative_case_html, twoac_case_html):
    """
    Generates a complete, high-quality 2AC debate speech transcript.
    The function takes as input the debate_case (HTML, including 1AC), negative_case_html (HTML, the 1NC), and twoac_case_html (HTML for the 2AC section).
    It appends a full, iteratively drafted 2AC transcript to twoac_case_html.
    The 2AC transcript is output in HTML format similar to the input documents.
    """

    llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        temperature=1.0,
        top_p=0.9,
        parallel_tool_calls=None
    )

    # 2AC Drafter: Writes the initial and revised 2AC speeches
    two_ac_drafter = ConversableAgent(
        name="two_ac_drafter",
        system_message=(
            "You are an expert affirmative policy debater preparing the 2AC speech. "
            "You have access to the full debate_case (including the 1AC) and the negative_case_html (the 1NC, including all off-case positions: theory, disadvantages, counterplans, kritiks, etc). "
            "Your job is to write a complete, high-quality, persuasive, and well-organized 2AC speech transcript. "
            "The speech should:\n"
            "- Extend all 1AC arguments and evidence\n"
            "- Directly answer and refute all of the 1NC arguments (off-case and on-case)\n"
            "- Clearly signpost and flow arguments (e.g., 'On the DA...', 'On the Kritik...', 'On Topicality...', etc.)\n"
            "- Explain why the 2AC wins the round\n"
            "- Use debate jargon and structure as in a real 2AC speech\n"
            "- Be extremely long, highly detailed, and complete—covering all major 1NC arguments and providing clear, specific, line-by-line answers\n"
            "- Be written as a transcript, as if the 2AC is being read aloud in a debate round\n"
            "- When answering counterplans and kritiks, you may and should include debate permutations (such as 'perm do both', 'perm do the plan', etc.) if and only if they are strategic in the context of the round. Do not use permutations automatically—only include them if they are likely to be effective and relevant against the specific counterplan or kritik presented in the 1NC.\n"
            "Do NOT simply list evidence or cards—write the full speech, integrating evidence and arguments as a debater would.\n"
            "IMPORTANT: Output the 2AC speech transcript in HTML format, using <div class='two-ac-section'>, <h1>2AC Speech</h1>, and <div class='twoac-transcript'> as containers, and use <p>, <h2>, <h3>, <ul>, <li>, <b>, <strong>, <em>, <br/>, and other HTML tags as appropriate for structure and readability. Do NOT use <pre> or markdown formatting. The output should closely match the HTML style of the input documents."
        ),
        llm_config=llm_config,
    )

    # 2AC Coach: Reviews and suggests improvements for the 2AC speech
    two_ac_coach = ConversableAgent(
        name="two_ac_coach",
        system_message=(
            "You are a highly experienced debate coach and judge. "
            "Your job is to review the 2AC speech draft and provide detailed, constructive feedback for improvement. "
            "Focus on:\n"
            "- Argument coverage: Did the 2AC answer all the most important 1NC arguments?\n"
            "- Strategic focus: Did the 2AC collapse to the best ground and avoid spreading too thin?\n"
            "- Clarity and organization: Is the speech easy to flow and follow?\n"
            "- Persuasiveness and use of evidence: Are arguments well-supported and explained?\n"
            "- Realism: Does the speech sound like a real, high-level 2AC?\n"
            "Suggest specific improvements, then ask the debater to revise the speech accordingly."
        ),
        llm_config=llm_config,
    )

    from autogen import GroupChat

    # Speaker selection: alternate between drafter and coach for 2-3 rounds
    def speaker_selection_func(last_speaker, groupchat):
        # First message: drafter writes initial speech
        if len(groupchat.messages) == 0:
            return two_ac_drafter
        # Drafter just wrote: coach reviews
        if last_speaker is two_ac_drafter:
            return None
        if last_speaker is two_ac_coach:
            return two_ac_drafter
        return None

    group_chat = GroupChat(
        agents=[two_ac_drafter, two_ac_coach],
        messages=[],
        max_round=4,
        speaker_selection_method=speaker_selection_func
    )

    group_chat_manager = GroupChatManager(
        groupchat=group_chat,
        llm_config=llm_config,
    )

    # Compose the context for the 2AC drafter
    context_message = (
        f"{debate_case}\n\n"
        f"Negative Case (1NC):\n{negative_case_html}\n\n"
        f"2AC Cards:\n{twoac_case_html}\n\n"
        "Assume that the current year is 2022.\n"
        "Write a complete, high-quality, realistic 2AC speech transcript as if you are reading it aloud in a debate round. "
        "Cover all major 1NC arguments, extend the 1AC, and collapse strategically. "
        "Use debate structure and jargon. Do not simply list evidence—write the full speech. "
        "When answering counterplans and kritiks, you may and should include debate permutations (such as 'perm do both', 'perm do the plan', etc.) if and only if they are strategic in the context of the round. Do not use permutations automatically—only include them if they are likely to be effective and relevant against the specific counterplan or kritik presented in the 1NC.\n"
        "IMPORTANT: Output the 2AC speech transcript in HTML format, using <div class='two-ac-section'>, <h1>2AC Speech</h1>, and <div class='twoac-transcript'> as containers, and use <p>, <h2>, <h3>, <ul>, <li>, <b>, <strong>, <em>, <br/>, and other HTML tags as appropriate for structure and readability. Do NOT use <pre> or markdown formatting. The output should closely match the HTML style of the input documents."
    )

    # Start the group chat
    chat_result = two_ac_coach.initiate_chat(
        group_chat_manager,
        message=context_message,
    )

    # Find the last 2AC speech draft in the chat history
    # (The drafter's last message)
    transcript = chat_result.chat_history[-1]["content"]

    # If the transcript already contains the outer HTML structure, avoid double-wrapping
    if "<div class='two-ac-section'>" in transcript:
        twoac_case_html += "\n" + transcript + "\n"
    else:
        twoac_case_html += "\n<div class='two-ac-section'>\n"
        twoac_case_html += "<h1>2AC Speech</h1>\n"
        twoac_case_html += f"<div class='twoac-transcript'>{transcript}</div>\n"
        twoac_case_html += "</div>\n"

    return twoac_case_html

In [ ]:
new_ac_case = add_2ac_to_case(debate_case, negative_case_html, ac_case)

In [ ]:
display(HTML(new_ac_case))

In [ ]:
new_ac_case

## Cross Examination (of the 2AC)

In [ ]:
def simulate_1nc_cross_examination_of_2ac(
    debate_case: str,
    negative_case_html: str,
    twoac_case_html: str
) -> str:
    """
    Simulates the cross-examination of the 2AC by the negative.
    Takes the 1AC (debate_case), the 1NC (negative_case_html), and the 2AC (twoac_case_html) as input,
    and returns a formatted HTML string of the cross-examination.
    """
    from typing import List
    from pydantic import BaseModel, Field

    # Define the structure for a cross-examination question and answer
    class CrossExPair(BaseModel):
        negative_question: str
        affirmative_response: str

    class CrossExamination(BaseModel):
        cross_ex: List[CrossExPair] = Field(..., min_items=4, max_items=4)

    # LLM config for all agents
    llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        parallel_tool_calls=None
    )

    cross_ex_llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        response_format=CrossExamination,
        parallel_tool_calls=None
    )

    # Agent 1: Negative asks questions
    negative_cross_ex_agent = ConversableAgent(
        name="negative_cross_ex_agent",
        system_message=(
            "You are the 1NC (negative) debater in a policy debate cross-examination. "
            "Your job is to ask sharp, strategic, and challenging questions about the 2AC (affirmative's second constructive) just presented. "
            "Focus on exposing weaknesses, ambiguities, or assumptions in the affirmative's extensions, answers to the 1NC, and overall strategy. "
            "Ask one question at a time, and wait for the affirmative to answer before asking the next. "
            "Do not answer your own questions. "
            "Be concise and direct. "
            "Do not repeat questions. "
            "You will ask a total of 3 to 7 questions."
        ),
        llm_config=llm_config,
    )

    # Agent 2: Affirmative answers
    affirmative_cross_ex_agent = ConversableAgent(
        name="affirmative_cross_ex_agent",
        system_message=(
            "You are the 2AC (affirmative) debater being cross-examined by the 1NC (negative) in a policy debate. "
            "Your job is to answer each question as clearly, persuasively, and strategically as possible, defending the affirmative case and the 2AC. "
            "Respond directly to the negative's question, but do not volunteer extra information. "
            "Be concise and avoid rambling. "
            "Do not ask questions yourself."
        ),
        llm_config=llm_config,
    )

    # Agent 3: Cross-ex summary agent (outputs the structured Q&A)
    cross_ex_summary_agent = ConversableAgent(
        name="cross_ex_summary_agent",
        system_message=(
            "You are a debate judge summarizing the 1NC's cross-examination of the 2AC. "
            "Your job is to produce a structured list of question/answer pairs, each with a 'negative_question' and an 'affirmative_response', "
            "covering the full cross-examination as it occurred. "
            "Return the result as a list of 3 to 7 question/answer pairs, each clearly labeled."
        ),
        llm_config=cross_ex_llm_config,
    )

    from autogen import GroupChat

    cross_ex_iterations = 0  # Track the number of Q&A iterations

    def cross_ex_speaker_selection(last_speaker, groupchat):
        nonlocal cross_ex_iterations
        # Alternate between negative and affirmative, then finish with summary agent
        if cross_ex_iterations == 0 and last_speaker is None:
            return negative_cross_ex_agent
        if last_speaker is negative_cross_ex_agent:
            return affirmative_cross_ex_agent
        if last_speaker is affirmative_cross_ex_agent:
            cross_ex_iterations += 1
            if cross_ex_iterations >= 4:
                return cross_ex_summary_agent
            else:
                return negative_cross_ex_agent
        if last_speaker is cross_ex_summary_agent:
            return None
        return "round_robin"

    group_chat = GroupChat(
        agents=[negative_cross_ex_agent, affirmative_cross_ex_agent, cross_ex_summary_agent],
        messages=[],
        max_round=40,
        speaker_selection_method=cross_ex_speaker_selection
    )

    group_chat_manager = GroupChatManager(
        groupchat=group_chat,
        llm_config=llm_config,
    )

    # The context for the cross-examination is the 1AC, 1NC, and 2AC
    chat_result = affirmative_cross_ex_agent.initiate_chat(
        group_chat_manager,
        message=(
            f"You are about to begin the 1NC's cross-examination of the 2AC. "
            f"The 1AC is as follows:\n\n{debate_case}\n\n"
            f"The 1NC is as follows:\n\n{negative_case_html}\n\n"
            f"The 2AC is as follows:\n\n{twoac_case_html}\n\n"
            "Begin by asking your first question."
        ),
    )

    # The summary agent's output is the last message in the chat history
    cross_ex_json = chat_result.chat_history[-1]["content"]
    cross_ex_data = json.loads(cross_ex_json)
    cross_ex_pairs = cross_ex_data["cross_ex"]

    # Format as HTML for display
    html = "<h2>1NC Cross-Examination of the 2AC</h2>\n"
    for i, pair in enumerate(cross_ex_pairs, 1):
        html += f"<div><b>Negative Question {i}:</b> {pair['negative_question']}</div>\n"
        html += f"<div><b>Affirmative Response {i}:</b> {pair['affirmative_response']}</div>\n"
        html += "<br/>\n"

    return html

# For compatibility with the rest of the code, assign to 2ac_crossex_html
# Usage: 2ac_crossex_html = simulate_1nc_cross_examination_of_2ac(debate_case, negative_case_html, twoac_case_html)

In [ ]:
max_attempts = 3
for attempt in range(1, max_attempts + 1):
    try:
        two_nc_crossx = simulate_1nc_cross_examination_of_2ac(debate_case, negative_case_html, new_ac_case)
        break
    except Exception as e:
        if attempt == max_attempts:
            raise

In [ ]:
display(HTML(two_nc_crossx))

In [ ]:
new_ac_case = new_ac_case + two_nc_crossx

In [ ]:
new_ac_case

# 2NC

### 2NC Gather Cards

In [ ]:
def add_2nc_evidence_to_case(debate_case, negative_case_html, twoac_debate_case_html):
    """
    Simulates the negative constructing the 2NC.
    Focuses on gathering new 2NC cards that answer the arguments the 2AC is most vulnerable to given the 1AC, 1NC, and 2AC.
    The selected cards, alongside the 1NC, should give the 2NC what it needs to win the debate round.
    This version does NOT write a full 2NC rebuttal speech; it only selects and presents the new 2NC cards.
    All cards must support the negative and the negative position.
    """

    class DebateCard(BaseModel):
        id: int
        cite: str
        include_in_case: Literal["include_it", "False"]
        reason_to_include: str
        retagged_argument_as_read_outloud_in_the_debate_round: str

    class DebateCardSearchResult(BaseModel):
        cards: List[DebateCard] = Field(..., min_items=1, max_items=6)

    llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        parallel_tool_calls=None
    )
    required_llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        tool_choice="required",
        temperature=1.0,
        top_p=0.9,
        parallel_tool_calls=None
    )
    debate_eval_llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        response_format=DebateCardSearchResult,
        parallel_tool_calls=None
    )

    # 2NC Agent: Focused on gathering the most strategic, high-impact 2NC cards
    two_nc_agent = ConversableAgent(
        name="two_nc_agent",
        system_message=(
            "You are an expert negative policy debater preparing the 2NC. "
            "You have access to the full debate_case (including the 1AC), the negative_case_html (the 1NC, including all off-case positions: theory, disadvantages, counterplans, kritiks, etc), "
            "and the 2AC_debate_case_html (the 2AC, including all new cards and arguments). "
            "Your job is to:\n"
            "1. Carefully read and analyze all 1AC, 1NC, and 2AC arguments and evidence.\n"
            "2. Identify which 2AC arguments and cards are most threatening to the negative's winning strategy—these are the arguments that, if left unanswered or insufficiently answered, would most likely cause the negative to lose the debate round.\n"
            "3. For each of these most threatening 2AC arguments, research and select the most strategic, high-quality, unique cards (evidence) that directly answer and refute those arguments, while also further entrenching and extending the negative's core claims. "
            "Each card must be:\n"
            "- Directly responsive to a specific 2AC argument that poses a significant threat to the negative\n"
            "- Not duplicative of any previous 1AC, 1NC, or 2AC card (do NOT select any card that is already in the 1AC, 1NC, or 2AC)\n"
            "- Clearly marked with its cite and a retagged argument as it would be read outloud in the debate round\n"
            "- Accompanied by a reason to include it in the 2NC, specifically explaining how it helps the negative win the round against the most dangerous 2AC arguments\n"
            "- Most importantly, every card you select must support the negative and the negative position. Do not select any card that undermines or contradicts the negative case or the 1NC's core claims.\n"
            "Do NOT write a 2NC rebuttal speech. Only select and present the new 2NC cards with their tags, cites, and reasons to include.\n"
            "Prioritize quality and strategic value over quantity: select only as many cards as are necessary to decisively answer the 2AC's most dangerous arguments and secure a winning position for the negative."
        ),
        llm_config=llm_config,
    )

    # 2NC Evidence Evaluator: Ensures only the most strategic, responsive cards are included
    two_nc_eval_agent = ConversableAgent(
        name="two_nc_eval_agent",
        system_message=(
            "You are a highly rigorous debate coach and argument analyst. "
            "Your job is to strictly evaluate whether each piece of evidence proposed for the 2NC meets the highest standards for inclusion:\n"
            "- Is it directly responsive to a 2AC argument that the negative is most vulnerable to?\n"
            "- Is it unique (not duplicative of any 1AC, 1NC, or 2AC card—do NOT approve any card that is already in the 1AC, 1NC, or 2AC)?\n"
            "- Is it authoritative and empirically supported?\n"
            "- Is it strategically valuable for the 2NC, meaning it helps the negative win the round against the most dangerous 2AC arguments?\n"
            "- Is it clearly retagged and recut for 2NC use?\n"
            "- Most importantly, does it support the negative and the negative position? Reject any card that undermines or contradicts the negative case or the 1NC's core claims.\n"
            "Reject any card that does not meet all criteria. Only approve cards that are directly responsive to the 2AC's most threatening arguments, unique, strategically valuable for the 2NC, and affirm the negative. Do NOT approve any card that is already in the 1AC, 1NC, or 2AC."
        ),
        llm_config=debate_eval_llm_config,
    )

    # Search agent for evidence
    debate_search_agent = ConversableAgent(
        name="debate_search_agent",
        system_message="You are a helpful assistant that can search the debate evidence dataset for a given tag. Your query will retrieve a list of debate cards.",
        llm_config=required_llm_config,
    )

    executor_agent = ConversableAgent(
        name="executor_agent",
        human_input_mode="NEVER",
        llm_config=llm_config,
    )

    register_function(
        search_debate_cards,
        caller=debate_search_agent,
        executor=executor_agent,
        description="Search the debate evidence dataset using natural language queries. Return a list of debate cards.",
    )

    from autogen import GroupChat

    iterations = 0
    max_iterations = 3  # Allow for more cards if needed, but focus on quality over quantity

    def custom_speaker_selection_func(last_speaker: Agent, groupchat: GroupChat):
        nonlocal iterations
        messages = groupchat.messages

        if len(messages) <= 1:
            return two_nc_agent

        if last_speaker is debate_search_agent:
            return executor_agent

        if last_speaker is executor_agent:
            return two_nc_eval_agent

        if last_speaker is two_nc_eval_agent:
            if "include_it" in messages[-1]["content"]:
                iterations += 1
                if iterations >= max_iterations:
                    return None
                else:
                    return debate_search_agent
            else:
                return debate_search_agent

        if last_speaker is two_nc_agent:
            return debate_search_agent
        else:
            return "round_robin"

    group_chat = GroupChat(
        agents=[two_nc_agent, debate_search_agent, executor_agent, two_nc_eval_agent],
        messages=[],
        max_round=60,
        speaker_selection_method=custom_speaker_selection_func
    )

    group_chat_manager = GroupChatManager(
        groupchat=group_chat,
        llm_config=llm_config,
    )

    # Use the debate_case (HTML, including the 1AC, which is the Affirmative Constructive and represents the affirmative team's advocacy and evidence), negative_case_html (HTML, the 1NC, which is the Negative Constructive), and twoac_debate_case_html (HTML, the 2AC, which is the Affirmative's Second Constructive) for context
    chat_result = debate_search_agent.initiate_chat(
        group_chat_manager,
        message=(
            f"Negative Case (1NC) (Our Team):\n{negative_case_html}\n\n"
            f"Affirmative Case (1AC):\n{debate_case}\n\n"
            f"Note: The 1AC (Affirmative Constructive) is the initial speech and evidence presented by the affirmative team, outlining their advocacy and core arguments.\n\n"
            f"2AC (Second Affirmative Constructive):\n{twoac_debate_case_html}\n\n"
            "Assume that the current year is 2022.\n"
            "You are the 2NC. Your job is to:\n"
            "- Extend all 1NC cards and arguments\n"
            "- Identify which 2AC arguments the negative is most vulnerable to and which are most likely to decide the round if left unanswered\n"
            "- For each of these most threatening 2AC arguments, research and present the most strategic, high-quality, unique cards that directly answer and refute them, while also further entrenching and extending the negative's core claims\n"
            "- For each card, provide its cite, a reason to include (explaining how it helps the 2NC win the round against the most dangerous 2AC arguments), and a retagged argument as it would be read outloud in the debate round\n"
            "- Most importantly, every card you select must support the negative and the negative position. Do not select any card that undermines or contradicts the negative case or the 1NC's core claims.\n"
            "- Do NOT select any card that is already in the 1AC, 1NC, or 2AC. All 2NC cards must be new and not previously used in the 1AC, 1NC, or 2AC.\n"
            "Do NOT write a 2NC rebuttal speech. Only select and present the new 2NC cards with their tags, cites, and reasons to include.\n"
            "Prioritize quality and strategic value over quantity: select only as many cards as are necessary to decisively answer the 2AC's most dangerous arguments and secure a winning position for the negative."
        ),
    )

    two_nc_result_raw = chat_result.chat_history[-1]["content"]
    two_nc_result_json = json.loads(two_nc_result_raw)
    cards = two_nc_result_json["cards"]

    # Build the 2NC HTML string
    two_nc_html = "<div class='two-nc-section'>\n"
    two_nc_html += "<h1>2NC</h1>\n"

    # Add each new 2NC card to the 2NC HTML
    for idx, card_json in enumerate(cards):
        card_id = card_json["id"]
        retagged_argument = card_json.get("retagged_argument_as_read_outloud_in_the_debate_round", "")
        reason_to_include = card_json.get("reason_to_include", "")
        card_doc = get_document_by_id(card_id)
        card_markup = str(card_doc['markup'])
        two_nc_html += (
            f"\n<h2>2NC Card {idx+1}</h2>"
            f"\n<div><p><strong></strong> {retagged_argument}</p></div>"
            f"\n<div><p>{card_markup}</p></div>"
        )

    two_nc_html += "\n</div>"

    return two_nc_html

In [ ]:
max_attempts = 3
for attempt in range(max_attempts):
    try:
        two_nc_html = add_2nc_evidence_to_case(debate_case, negative_case_html, new_ac_case)
        break
    except Exception as e:
        if attempt == max_attempts - 1:
            raise

In [ ]:
display(HTML(two_nc_html))

In [ ]:
two_nc_html

### 2NC: Write Speech

In [ ]:
def add_2nc_to_case(debate_case, negative_case_html, twoac_case_html, two_nc_html):
    """
    Generates a complete, high-quality 2NC debate speech transcript.
    The function takes as input the debate_case (HTML, including 1AC), negative_case_html (HTML, the 1NC), twoac_case_html (HTML for the 2AC section), and two_nc_html (HTML for the 2NC cards).
    It appends a full, iteratively drafted 2NC transcript to two_nc_html and returns the updated HTML.
    The 2NC speech transcript will be output in HTML format (using <div>, <p>, <h2>, <ul>, <ol>, etc.), not in markdown or <pre> tags.
    """

    llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        temperature=1.0,
        top_p=0.9,
        parallel_tool_calls=None
    )

    # 2NC Drafter: Writes the initial and revised 2NC speeches
    two_nc_drafter = ConversableAgent(
        name="two_nc_drafter",
        system_message=(
            "You are an expert negative policy debater preparing the 2NC speech. "
            "You have access to the full debate_case (including the 1AC), the negative_case_html (the 1NC, including all off-case positions: theory, disadvantages, counterplans, kritiks, etc), the 2AC (including all arguments and evidence), and the 2NC cards. "
            "Your job is to write a complete, high-quality, persuasive, and well-organized 2NC speech transcript. "
            "The speech should:\n"
            "- Extend all 1NC arguments and evidence\n"
            "- Directly answer and refute all of the 2AC arguments (off-case and on-case)\n"
            "- Clearly signpost and flow arguments (e.g., 'On the DA...', 'On the Kritik...', 'On Topicality...', etc.)\n"
            "- Explain why the negative wins the round\n"
            "- Use debate jargon and structure as in a real 2NC speech\n"
            "- Be extremely long, highly detailed, and complete—covering all major 2AC arguments and providing clear, specific, line-by-line answers\n"
            "- Be written as a transcript, as if the 2NC is being read aloud in a debate round\n"
            "- When answering counterplans and kritiks, you may and should include debate permutations (such as 'perm do both', 'perm do the plan', etc.) if and only if they are strategic in the context of the round. Do not use permutations automatically—only include them if they are likely to be effective and relevant against the specific counterplan or kritik presented in the 2AC.\n"
            "Do NOT simply list evidence or cards—write the full speech, integrating evidence and arguments as a debater would.\n"
            "IMPORTANT: Output the 2NC speech transcript in HTML format, using <div>, <h2>, <h3>, <p>, <ul>, <ol>, <b>, <i>, and similar tags. Do NOT use <pre> or markdown formatting. The output should be visually similar to the input card HTML formats."
        ),
        llm_config=llm_config,
    )

    # 2NC Coach: Reviews and suggests improvements for the 2NC speech
    two_nc_coach = ConversableAgent(
        name="two_nc_coach",
        system_message=(
            "You are a highly experienced debate coach and judge. "
            "Your job is to review the 2NC speech draft and provide detailed, constructive feedback for improvement. "
            "Focus on:\n"
            "- Argument coverage: Did the 2NC answer all the most important 2AC arguments?\n"
            "- Strategic focus: Did the 2NC collapse to the best ground and avoid spreading too thin?\n"
            "- Clarity and organization: Is the speech easy to flow and follow?\n"
            "- Persuasiveness and use of evidence: Are arguments well-supported and explained?\n"
            "- Realism: Does the speech sound like a real, high-level 2NC?\n"
            "Suggest specific improvements, then ask the debater to revise the speech accordingly."
        ),
        llm_config=llm_config,
    )

    from autogen import GroupChat

    # Speaker selection: alternate between drafter and coach for 2-3 rounds
    def speaker_selection_func(last_speaker, groupchat):
        # First message: drafter writes initial speech
        if len(groupchat.messages) == 0:
            return two_nc_drafter
        # Drafter just wrote: coach reviews
        if last_speaker is two_nc_drafter:
            return None
        if last_speaker is two_nc_coach:
            return two_nc_drafter
        return None

    group_chat = GroupChat(
        agents=[two_nc_drafter, two_nc_coach],
        messages=[],
        max_round=4,
        speaker_selection_method=speaker_selection_func
    )

    group_chat_manager = GroupChatManager(
        groupchat=group_chat,
        llm_config=llm_config,
    )

    # Compose the context for the 2NC drafter
    context_message = (
        f"{debate_case}\n\n"
        f"Negative Case (1NC):\n{negative_case_html}\n\n"
        f"2AC Speech and Cards:\n{twoac_case_html}\n\n"
        f"2NC Cards:\n{two_nc_html}\n\n"
        "Assume that the current year is 2022.\n"
        "Write a complete, high-quality, realistic 2NC speech transcript as if you are reading it aloud in a debate round. "
        "Cover all major 2AC arguments, extend the 1NC, and collapse strategically. "
        "Use debate structure and jargon. Do not simply list evidence—write the full speech. "
        "When answering counterplans and kritiks, you may and should include debate permutations (such as 'perm do both', 'perm do the plan', etc.) if and only if they are strategic in the context of the round. Do not use permutations automatically—only include them if they are likely to be effective and relevant against the specific counterplan or kritik presented in the 2AC.\n"
        "IMPORTANT: Output the 2NC speech transcript in HTML format, using <div>, <h2>, <h3>, <p>, <ul>, <ol>, <b>, <i>, and similar tags. Do NOT use <pre> or markdown formatting. The output should be visually similar to the input card HTML formats."
    )

    # Start the group chat
    chat_result = two_nc_coach.initiate_chat(
        group_chat_manager,
        message=context_message,
    )

    # Find the last 2NC speech draft in the chat history
    # (The drafter's last message)
    transcript = chat_result.chat_history[-1]["content"]
    # Append the 2NC transcript to the two_nc_html
    two_nc_html += "\n<div class='two-nc-speech-section'>\n"
    two_nc_html += "<h1>2NC Speech</h1>\n"
    two_nc_html += f"<div class='two-nc-transcript'>{transcript}</div>\n"
    two_nc_html += "</div>\n"

    return two_nc_html

In [ ]:
two_nc_html_full = add_2nc_to_case(debate_case, negative_case_html, new_ac_case, two_nc_html)

In [ ]:
display(HTML(two_nc_html_full))

In [ ]:
two_nc_html_full

## Cross Examination (of the 2NC)

In [ ]:
def simulate_2ac_cross_examination_of_2nc(
    debate_case: str,
    negative_case_html: str,
    twoac_case_html: str,
    twonc_case_html: str
) -> str:
    """
    Simulates the cross-examination of the 2NC by the 2AC.
    Takes the 1AC (debate_case), the 1NC (negative_case_html), the 2AC (twoac_case_html), and the 2NC (twonc_case_html) as input,
    and returns a formatted HTML string of the cross-examination.
    The affirmative questions must always be designed to support the affirmative case and plan, and the negative responses must always reject the plan and support the negative counterplan, counteradvocacy, and negative positions.
    """
    from typing import List
    from pydantic import BaseModel, Field

    # Define the structure for a cross-examination question and answer
    class CrossExPair(BaseModel):
        affirmative_question: str
        negative_response: str

    class CrossExamination(BaseModel):
        cross_ex: List[CrossExPair] = Field(..., min_items=3, max_items=3)

    # LLM config for all agents
    llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        parallel_tool_calls=None
    )

    cross_ex_llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        response_format=CrossExamination,
        parallel_tool_calls=None
    )

    # Agent 1: Affirmative asks questions
    affirmative_cross_ex_agent = ConversableAgent(
        name="affirmative_cross_ex_agent",
        system_message=(
            "You are the 2AC (affirmative) debater in a policy debate cross-examination. "
            "Your job is to ask sharp, strategic, and challenging questions about the 2NC (negative's second constructive) just presented. "
            "Every question you ask must be designed to directly support the affirmative's case and plan, and to undermine the negative's arguments, counterplan, counteradvocacy, and all negative positions. "
            "Focus on exposing weaknesses, ambiguities, or assumptions in the negative's extensions, new arguments, and overall strategy, "
            "especially in ways that help the affirmative win the round and defend the plan. "
            "Ask one question at a time, and wait for the negative to answer before asking the next. "
            "Do not answer your own questions. "
            "Be concise and direct. "
            "Do not repeat questions. "
            "You will ask a total of 3 to 7 questions."
        ),
        llm_config=llm_config,
    )

    # Agent 2: Negative answers
    negative_cross_ex_agent = ConversableAgent(
        name="negative_cross_ex_agent",
        system_message=(
            "You are the 2NC (negative) debater being cross-examined by the 2AC (affirmative) in a policy debate. "
            "Your job is to answer each question as clearly, persuasively, and strategically as possible, always rejecting the plan and supporting the negative's counterplan, counteradvocacy, and all negative positions. "
            "Every answer you give must be designed to help the negative win the round, reinforce the negative's arguments, and defend the negative's counterplan or advocacy against the plan. "
            "Respond directly to the affirmative's question, but do not volunteer extra information. "
            "Be concise and avoid rambling. "
            "Do not ask questions yourself."
        ),
        llm_config=llm_config,
    )

    # Agent 3: Cross-ex summary agent (outputs the structured Q&A)
    cross_ex_summary_agent = ConversableAgent(
        name="cross_ex_summary_agent",
        system_message=(
            "You are a debate judge summarizing the 2AC's cross-examination of the 2NC. "
            "Your job is to produce a structured list of question/answer pairs, each with an 'affirmative_question' and a 'negative_response', "
            "covering the full cross-examination as it occurred. "
            "Each affirmative question must be designed to support the affirmative's case and plan, and each negative response must be designed to reject the plan and support the negative's counterplan, counteradvocacy, and negative positions. "
            "Return the result as a list of 3 to 7 question/answer pairs, each clearly labeled."
        ),
        llm_config=cross_ex_llm_config,
    )

    from autogen import GroupChat

    cross_ex_iterations = 0  # Track the number of Q&A iterations

    def cross_ex_speaker_selection(last_speaker, groupchat):
        nonlocal cross_ex_iterations
        # Alternate between affirmative and negative, then finish with summary agent
        if last_speaker is affirmative_cross_ex_agent:
            return negative_cross_ex_agent
        if last_speaker is negative_cross_ex_agent:
            cross_ex_iterations += 1
            if cross_ex_iterations >= 3:
                return cross_ex_summary_agent
            else:
                return affirmative_cross_ex_agent
        if last_speaker is cross_ex_summary_agent:
            return None
        return "round_robin"

    group_chat = GroupChat(
        agents=[affirmative_cross_ex_agent, negative_cross_ex_agent, cross_ex_summary_agent],
        messages=[],
        max_round=40,
        speaker_selection_method=cross_ex_speaker_selection
    )

    group_chat_manager = GroupChatManager(
        groupchat=group_chat,
        llm_config=llm_config,
    )

    # The context for the cross-examination is the 1AC, 1NC, 2AC, and 2NC
    chat_result = negative_cross_ex_agent.initiate_chat(
        group_chat_manager,
        message=(
            f"You are about to begin the 2AC's cross-examination of the 2NC. "
            f"The 1AC is as follows:\n\n{debate_case}\n\n"
            f"The 1NC is as follows:\n\n{negative_case_html}\n\n"
            f"The 2AC is as follows:\n\n{twoac_case_html}\n\n"
            f"The 2NC is as follows:\n\n{twonc_case_html}\n\n"
            "Begin by asking your first question."
        ),
    )

    # The summary agent's output is the last message in the chat history
    cross_ex_json = chat_result.chat_history[-1]["content"]
    cross_ex_data = json.loads(cross_ex_json)
    cross_ex_pairs = cross_ex_data["cross_ex"]

    # Format as HTML for display
    html = "<h2>2AC Cross-Examination of the 2NC</h2>\n"
    for i, pair in enumerate(cross_ex_pairs, 1):
        html += f"<div><b>Affirmative Question {i}:</b> {pair['affirmative_question']}</div>\n"
        html += f"<div><b>Negative Response {i}:</b> {pair['negative_response']}</div>\n"
        html += "<br/>\n"

    return html

# For compatibility with the rest of the code, assign to twonc_crossex_html
# Usage: twonc_crossex_html = simulate_2ac_cross_examination_of_2nc(debate_case, negative_case_html, twoac_case_html, twonc_case_html)

In [ ]:
attempts = 0
while True:
    try:
        two_ac_crossex = simulate_2ac_cross_examination_of_2nc(debate_case, negative_case_html, new_ac_case, two_nc_html_full)
        break
    except Exception as e:
        attempts += 1
        if attempts >= 3:
            raise

In [ ]:
two_nc_html_full = two_nc_html_full + two_ac_crossex

In [ ]:
two_nc_html_full

# 1NR

In [ ]:
def add_1nr_to_case(debate_case, negative_case_html, twoac_case_html, twonc_case_html):
    """
    Generates a complete, high-quality 1NR debate speech transcript.
    The function takes as input:
        - debate_case (HTML, including 1AC)
        - negative_case_html (HTML, the 1NC)
        - twoac_case_html (HTML for the 2AC section)
        - twonc_case_html (HTML for the 2NC section)
    It returns a full, iteratively drafted 1NR transcript as onenr_case_html.
    The 1NR should be shorter than the 2NC and focus on making arguments that are distinct from the 2NC.
    The output will be in HTML format, similar to the input card formats, and should not use <pre> or markdown tags.
    """

    llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        temperature=1.0,
        top_p=0.9,
        parallel_tool_calls=None
    )

    # 1NR Drafter: Writes the initial and revised 1NR speeches
    one_nr_drafter = ConversableAgent(
        name="one_nr_drafter",
        system_message=(
            "You are an expert negative policy debater preparing the 1NR speech. "
            "You have access to the full debate_case (including the 1AC), the negative_case_html (the 1NC, including all off-case positions: theory, disadvantages, counterplans, kritiks, etc), the 2AC, and the 2NC. "
            "Your job is to write a complete, high-quality, persuasive, and well-organized 1NR speech transcript. "
            "The speech should:\n"
            "- Be noticeably shorter than the 2NC, simulating the less time allocated to the 1NR in a real debate round.\n"
            "- Focus on making arguments that are distinct from those made in the 2NC, rather than repeating or rephrasing them. If possible, cover different off-case or on-case arguments, or provide new extensions, analytics, or strategic concessions.\n"
            "- Extend the best negative arguments and evidence from the 1NC and 2NC, but avoid duplicating the 2NC's content.\n"
            "- Directly answer and refute all of the 2AC arguments (off-case and on-case) that were not fully addressed by the 2NC, or that require additional negative development.\n"
            "- Clearly signpost and flow arguments (e.g., 'On the DA...', 'On the Kritik...', 'On Topicality...', etc.)\n"
            "- Explain why the negative is still winning the round\n"
            "- Use debate jargon and structure as in a real 1NR speech\n"
            "- Be detailed and complete—covering all major 2AC arguments relevant to the 1NR, and providing clear, specific, line-by-line answers, but do not attempt to be as long or comprehensive as the 2NC.\n"
            "- Be written as a transcript, as if the 1NR is being read aloud in a debate round\n"
            "- When answering affirmative permutations or new arguments, you may and should strategically concede any arguments from the 1NC or 2NC that you feel are weak, so that you can spend more time on the arguments that the negative is winning on. Do not simply list evidence or cards—write the full speech, integrating evidence and arguments as a debater would.\n"
            "IMPORTANT: Output the 1NR speech transcript in HTML format, using <div>, <h2>, <b>, <ul>, <li>, <p>, and similar tags as appropriate. Do NOT use <pre>, <code>, or markdown formatting. The output should visually match the style of the input card HTML formats."
        ),
        llm_config=llm_config,
    )

    # 1NR Coach: Reviews and suggests improvements for the 1NR speech
    one_nr_coach = ConversableAgent(
        name="one_nr_coach",
        system_message=(
            "You are a highly experienced debate coach and judge. "
            "Your job is to review the 1NR speech draft and provide detailed, constructive feedback for improvement. "
            "Focus on:\n"
            "- Argument coverage: Did the 1NR answer all the most important 2AC arguments, especially those not fully addressed by the 2NC?\n"
            "- Strategic focus: Did the 1NR strategically concede any weak arguments from the 1NC or 2NC in order to spend more time on the arguments the negative is winning on, and avoid spreading too thin?\n"
            "- Distinctiveness: Did the 1NR make arguments that are distinct from the 2NC, rather than repeating or rephrasing them? Did it cover different off-case or on-case arguments, or provide new extensions or analytics?\n"
            "- Length: Is the 1NR noticeably shorter than the 2NC, reflecting the time constraints of the speech?\n"
            "- Clarity and organization: Is the speech easy to flow and follow?\n"
            "- Persuasiveness and use of evidence: Are arguments well-supported and explained?\n"
            "- Realism: Does the speech sound like a real, high-level 1NR?\n"
            "Suggest specific improvements, then ask the debater to revise the speech accordingly."
        ),
        llm_config=llm_config,
    )

    from autogen import GroupChat

    # Speaker selection: alternate between drafter and coach for 2-3 rounds
    def speaker_selection_func(last_speaker, groupchat):
        # First message: drafter writes initial speech
        if len(groupchat.messages) == 0:
            return one_nr_drafter
        # Drafter just wrote: coach reviews
        if last_speaker is one_nr_drafter:
            return None
        if last_speaker is one_nr_coach:
            return one_nr_drafter
        return None

    group_chat = GroupChat(
        agents=[one_nr_drafter, one_nr_coach],
        messages=[],
        max_round=4,
        speaker_selection_method=speaker_selection_func
    )

    group_chat_manager = GroupChatManager(
        groupchat=group_chat,
        llm_config=llm_config,
    )

    # Compose the context for the 1NR drafter
    context_message = (
        f"{debate_case}\n\n"
        f"Negative Case (1NC):\n{negative_case_html}\n\n"
        f"2AC Speech and Cards:\n{twoac_case_html}\n\n"
        f"2NC Speech and Cards:\n{twonc_case_html}\n\n"
        "Assume that the current year is 2022.\n"
        "Write a complete, high-quality, realistic 1NR speech transcript as if you are reading it aloud in a debate round. "
        "The 1NR should be noticeably shorter than the 2NC, simulating the less time allocated to the 1NR. "
        "Focus on making arguments that are distinct from those made in the 2NC, rather than repeating or rephrasing them. If possible, cover different off-case or on-case arguments, or provide new extensions, analytics, or strategic concessions. "
        "Cover all major 2AC arguments relevant to the 1NR, extend the negative, and strategically concede any arguments from the 1NC or 2NC that you feel are weak, so that you can spend more time on the arguments that the negative is winning on. "
        "Use debate structure and jargon. Do not simply list evidence—write the full speech. "
        "When answering affirmative permutations or new arguments, focus your time on the arguments the negative is winning and explain why the negative wins. "
        "IMPORTANT: Output the 1NR speech transcript in HTML format, using <div>, <h2>, <b>, <ul>, <li>, <p>, and similar tags as appropriate. Do NOT use <pre>, <code>, or markdown formatting. The output should visually match the style of the input card HTML formats."
    )

    # Start the group chat
    chat_result = one_nr_coach.initiate_chat(
        group_chat_manager,
        message=context_message,
    )

    # Find the last 1NR speech draft in the chat history
    # (The drafter's last message)
    transcript = chat_result.chat_history[-1]["content"]
    # Append the 1NR transcript to the onenr_case_html
    onenr_case_html = "\n<div class='one-nr-section'>\n"
    onenr_case_html += "<h1>1NR Speech</h1>\n"
    onenr_case_html += f"<div class='onenr-transcript'>{transcript}</div>\n"
    onenr_case_html += "</div>\n"

    return onenr_case_html

In [ ]:
onenr_html = add_1nr_to_case(debate_case, negative_case_html, new_ac_case, two_nc_html_full)

In [ ]:
display(HTML(onenr_html))

In [ ]:
onenr_html

# 1AR 

In [ ]:
def add_1ar_to_case(debate_case, negative_case_html, twoac_case_html, twonc_case_html, onenr_html):
    """
    Generates a complete, high-quality 1AR debate speech transcript.
    The function takes as input the debate_case (HTML, including 1AC), negative_case_html (HTML, the 1NC), twoac_case_html (HTML for the 2AC section), twonc_case_html (HTML for the 2NC section), and onenr_html (HTML for the 1NR section).
    It appends a full, iteratively drafted 1AR transcript to 1AR_case_html.
    """

    llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        temperature=1.0,
        top_p=0.9,
        parallel_tool_calls=None
    )

    # 1AR Drafter: Writes the initial and revised 1AR speeches
    one_ar_drafter = ConversableAgent(
        name="one_ar_drafter",
        system_message=(
            "You are an expert affirmative policy debater preparing the 1AR speech. "
            "You have access to the full debate_case (including the 1AC), the negative_case_html (the 1NC, including all off-case positions: theory, disadvantages, counterplans, kritiks, etc), the 2AC, the 2NC, and the 1NR. "
            "Your job is to write a complete, high-quality, persuasive, and well-organized 1AR speech transcript. "
            "The speech should:\n"
            "- Extend all 1AC and 2AC arguments and evidence\n"
            "- Directly answer and refute all of the 2NC and 1NR arguments (off-case and on-case)\n"
            "- Clearly signpost and flow arguments (e.g., 'On the DA...', 'On the Kritik...', 'On Topicality...', etc.)\n"
            "- Explain why the affirmative is still winning the round\n"
            "- Use debate jargon and structure as in a real 1AR speech\n"
            "- Be extremely long, highly detailed, and complete—covering all major 2NC and 1NR arguments and providing clear, specific, line-by-line answers\n"
            "- Be written as a transcript, as if the 1AR is being read aloud in a debate round\n"
            "- When answering counterplans and kritiks, you may and should include debate permutations (such as 'perm do both', 'perm do the plan', etc.) if and only if they are strategic in the context of the round. Do not use permutations automatically—only include them if they are likely to be effective and relevant against the specific counterplan or kritik presented in the 2NC or 1NR.\n"
            "Do NOT simply list evidence or cards—write the full speech, integrating evidence and arguments as a debater would.\n"
            "IMPORTANT: Output the 1AR speech transcript in HTML format, using <div>, <h2>, <b>, <ul>, <li>, <p>, and similar tags as appropriate. Do NOT use <pre>, <code>, or markdown formatting. The output should visually match the style of the input card HTML formats."
        ),
        llm_config=llm_config,
    )

    # 1AR Coach: Reviews and suggests improvements for the 1AR speech
    one_ar_coach = ConversableAgent(
        name="one_ar_coach",
        system_message=(
            "You are a highly experienced debate coach and judge. "
            "Your job is to review the 1AR speech draft and provide detailed, constructive feedback for improvement. "
            "Focus on:\n"
            "- Argument coverage: Did the 1AR answer all the most important 2NC and 1NR arguments?\n"
            "- Strategic focus: Did the 1AR collapse to the best ground and avoid spreading too thin?\n"
            "- Clarity and organization: Is the speech easy to flow and follow?\n"
            "- Persuasiveness and use of evidence: Are arguments well-supported and explained?\n"
            "- Realism: Does the speech sound like a real, high-level 1AR?\n"
            "Suggest specific improvements, then ask the debater to revise the speech accordingly."
        ),
        llm_config=llm_config,
    )

    from autogen import GroupChat

    # Speaker selection: alternate between drafter and coach for 2-3 rounds
    def speaker_selection_func(last_speaker, groupchat):
        # First message: drafter writes initial speech
        if len(groupchat.messages) == 0:
            return one_ar_drafter
        # Drafter just wrote: coach reviews
        if last_speaker is one_ar_drafter:
            return None
        if last_speaker is one_ar_coach:
            return one_ar_drafter
        return None

    group_chat = GroupChat(
        agents=[one_ar_drafter, one_ar_coach],
        messages=[],
        max_round=4,
        speaker_selection_method=speaker_selection_func
    )

    group_chat_manager = GroupChatManager(
        groupchat=group_chat,
        llm_config=llm_config,
    )

    # Compose the context for the 1AR drafter
    context_message = (
        f"{debate_case}\n\n"
        f"Negative Case (1NC):\n{negative_case_html}\n\n"
        f"2AC Speech and Cards:\n{twoac_case_html}\n\n"
        f"2NC Speech and Cards:\n{twonc_case_html}\n\n"
        f"1NR Speech and Cards:\n{onenr_html}\n\n"
        "Assume that the current year is 2022.\n"
        "Write a complete, high-quality, realistic 1AR speech transcript as if you are reading it aloud in a debate round. "
        "Cover all major 2NC and 1NR arguments, extend the 2AC, and crystalize why you are winning strategically. "
        "Use debate structure and jargon. Do not simply list evidence—write the full speech. "
        "When answering counterplans and kritiks, you may and should include debate permutations (such as 'perm do both', 'perm do the plan', etc.) if and only if they are strategic in the context of the round. Do not use permutations automatically—only include them if they are likely to be effective and relevant against the specific counterplan or kritik presented in the 2NC or 1NR.\n"
        "IMPORTANT: Output the 1AR speech transcript in HTML format, using <div>, <h2>, <b>, <ul>, <li>, <p>, and similar tags as appropriate. Do NOT use <pre>, <code>, or markdown formatting. The output should visually match the style of the input card HTML formats."
    )

    # Start the group chat
    chat_result = one_ar_coach.initiate_chat(
        group_chat_manager,
        message=context_message,
    )

    # Find the last 1AR speech draft in the chat history
    # (The drafter's last message)
    transcript = chat_result.chat_history[-1]["content"]
    # Append the 1AR transcript to the 1AR_case_html
    onear_case_html = "\n<div class='one-ar-section'>\n"
    onear_case_html += "<h1>1AR Speech</h1>\n"
    onear_case_html += f"<div class='onear-transcript'>{transcript}</div>\n"
    onear_case_html += "</div>\n"

    return onear_case_html

In [ ]:
one_ar_html = add_1ar_to_case(debate_case, negative_case_html, new_ac_case, two_nc_html_full, onenr_html)

In [ ]:
display(HTML(one_ar_html))

In [ ]:
one_ar_html

# 2NR

In [ ]:
def add_2nr_to_case(debate_case, negative_case_html, twoac_case_html, twonc_case_html, onenr_html, onear_case_html):
    """
    Generates a complete, high-quality 2NR debate speech transcript.
    The function takes as input:
        - debate_case (HTML, including 1AC)
        - negative_case_html (HTML, the 1NC)
        - twoac_case_html (HTML for the 2AC section)
        - twonc_case_html (HTML for the 2NC section)
        - onenr_html (HTML for the 1NR section)
        - onear_case_html (HTML for the 1AR section)
    It appends a full, iteratively drafted 2NR transcript to 2nr_case_html.
    """

    llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        temperature=1.0,
        top_p=0.9,
        parallel_tool_calls=None
    )

    # 2NR Drafter: Writes the initial and revised 2NR speeches
    two_nr_drafter = ConversableAgent(
        name="two_nr_drafter",
        system_message=(
            "You are an expert negative policy debater preparing the 2NR speech. "
            "You have access to the full debate_case (including the 1AC), the negative_case_html (the 1NC, including all off-case positions: theory, disadvantages, counterplans, kritiks, etc), the 2AC, the 2NC, the 1NR, and the 1AR. "
            "Your job is to write a complete, high-quality, persuasive, and well-organized 2NR speech transcript. "
            "The speech should:\n"
            "- Extend the best negative arguments and evidence from the 1NC and 2NC\n"
            "- Directly answer and refute all of the 2AC and 1AR arguments (off-case and on-case)\n"
            "- Clearly signpost and flow arguments (e.g., 'On the DA...', 'On the Kritik...', 'On Topicality...', etc.)\n"
            "- Explain why the negative is still winning the round\n"
            "- Use debate jargon and structure as in a real 2NR speech\n"
            "- Be extremely long, highly detailed, and complete—covering all major 1AR arguments and providing clear, specific, line-by-line answers\n"
            "- Be written as a transcript, as if the 2NR is being read aloud in a debate round\n"
            "- When answering affirmative permutations or new arguments, you may and should strategically concede any arguments from the 1NC or 2NC that you feel are weak, so that you can spend more time on the arguments that the negative is winning on. Do not simply list evidence or cards—write the full speech, integrating evidence and arguments as a debater would."
            "\nIMPORTANT: Output the 2NR speech transcript in HTML format, using <div>, <h2>, <b>, <ul>, <li>, <p>, and similar tags as appropriate. Do NOT use <pre>, <code>, or markdown formatting. The output should visually match the style of the input card HTML formats."
        ),
        llm_config=llm_config,
    )

    # 2NR Coach: Reviews and suggests improvements for the 2NR speech
    two_nr_coach = ConversableAgent(
        name="two_nr_coach",
        system_message=(
            "You are a highly experienced debate coach and judge. "
            "Your job is to review the 2NR speech draft and provide detailed, constructive feedback for improvement. "
            "Focus on:\n"
            "- Argument coverage: Did the 2NR answer all the most important 1AR arguments?\n"
            "- Strategic focus: Did the 2NR strategically concede any weak arguments from the 1NC or 2NC in order to spend more time on the arguments the negative is winning on, and avoid spreading too thin?\n"
            "- Clarity and organization: Is the speech easy to flow and follow?\n"
            "- Persuasiveness and use of evidence: Are arguments well-supported and explained?\n"
            "- Realism: Does the speech sound like a real, high-level 2NR?\n"
            "Suggest specific improvements, then ask the debater to revise the speech accordingly."
        ),
        llm_config=llm_config,
    )

    from autogen import GroupChat

    # Speaker selection: alternate between drafter and coach for 2-3 rounds
    def speaker_selection_func(last_speaker, groupchat):
        # First message: drafter writes initial speech
        if len(groupchat.messages) == 0:
            return two_nr_drafter
        # Drafter just wrote: coach reviews
        if last_speaker is two_nr_drafter:
            return None
        if last_speaker is two_nr_coach:
            return two_nr_drafter
        return None

    group_chat = GroupChat(
        agents=[two_nr_drafter, two_nr_coach],
        messages=[],
        max_round=4,
        speaker_selection_method=speaker_selection_func
    )

    group_chat_manager = GroupChatManager(
        groupchat=group_chat,
        llm_config=llm_config,
    )

    # Compose the context for the 2NR drafter
    context_message = (
        f"{debate_case}\n\n"
        f"Negative Case (1NC):\n{negative_case_html}\n\n"
        f"2AC Speech and Cards:\n{twoac_case_html}\n\n"
        f"2NC Speech and Cards:\n{twonc_case_html}\n\n"
        f"1NR Speech and Cards:\n{onenr_html}\n\n"
        f"1AR Speech and Cards:\n{onear_case_html}\n\n"
        "Assume that the current year is 2022.\n"
        "Write a complete, high-quality, realistic 2NR speech transcript as if you are reading it aloud in a debate round. "
        "Cover all major 1AR arguments, extend the negative, and strategically concede any arguments from the 1NC or 2NC that you feel are weak, so that you can spend more time on the arguments that the negative is winning on. "
        "Use debate structure and jargon. Do not simply list evidence—write the full speech. "
        "When answering affirmative permutations or new arguments, focus your time on the arguments the negative is winning and explain why the negative wins."
        "\nIMPORTANT: Output the 2NR speech transcript in HTML format, using <div>, <h2>, <b>, <ul>, <li>, <p>, and similar tags as appropriate. Do NOT use <pre>, <code>, or markdown formatting. The output should visually match the style of the input card HTML formats."
    )

    # Start the group chat
    chat_result = two_nr_coach.initiate_chat(
        group_chat_manager,
        message=context_message,
    )

    # Find the last 2NR speech draft in the chat history
    # (The drafter's last message)
    transcript = chat_result.chat_history[-1]["content"]
    # Append the 2NR transcript to the 2nr_case_html
    two_nr_case_html = "\n<div class='two-nr-section'>\n"
    two_nr_case_html += "<h1>2NR Speech</h1>\n"
    two_nr_case_html += f"<div class='twonr-transcript'>{transcript}</div>\n"
    two_nr_case_html += "</div>\n"

    return two_nr_case_html

In [ ]:
two_nr_html = add_2nr_to_case(debate_case, negative_case_html, new_ac_case, two_nc_html_full, onenr_html, one_ar_html)

In [ ]:
display(HTML(two_nr_html))

In [ ]:
two_nr_html

# 2AR

In [ ]:
def add_2ar_to_case(
    debate_case,
    negative_case_html,
    twoac_case_html,
    twonc_case_html,
    onenr_html,
    onear_case_html,
    two_nr_case_html
):
    """
    Generates a complete, high-quality 2AR debate speech transcript.
    The function takes as input:
        - debate_case (HTML, including 1AC)
        - negative_case_html (HTML, the 1NC)
        - twoac_case_html (HTML for the 2AC section)
        - twonc_case_html (HTML for the 2NC section)
        - onenr_html (HTML for the 1NR section)
        - onear_case_html (HTML for the 1AR section)
        - two_nr_case_html (HTML for the 2NR section)
    It returns a full, iteratively drafted 2AR transcript as 2ar_case_html.
    """

    llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        temperature=1.0,
        top_p=0.9,
        parallel_tool_calls=None
    )

    # 2AR Drafter: Writes the initial and revised 2AR speeches
    two_ar_drafter = ConversableAgent(
        name="two_ar_drafter",
        system_message=(
            "You are an expert affirmative policy debater preparing the 2AR speech. "
            "You have access to the full debate_case (including the 1AC), the negative_case_html (the 1NC, including all off-case positions: theory, disadvantages, counterplans, kritiks, etc), the 2AC, the 2NC, the 1NR, the 1AR, and the 2NR. "
            "Your job is to write a complete, high-quality, persuasive, and well-organized 2AR speech transcript. "
            "The speech should:\n"
            "- Extend the best affirmative arguments and evidence from the 1AC, 2AC, and 1AR\n"
            "- Directly answer and refute all of the 1NC, 2NC, 1NR, and 2NR arguments (off-case and on-case)\n"
            "- Clearly signpost and flow arguments (e.g., 'On the DA...', 'On the Kritik...', 'On Topicality...', etc.)\n"
            "- Explain why the affirmative is still winning the round\n"
            "- Use debate jargon and structure as in a real 2AR speech\n"
            "- Be extremely long, highly detailed, and complete—covering all major 2NR arguments and providing clear, specific, line-by-line answers\n"
            "- Be written as a transcript, as if the 2AR is being read aloud in a debate round\n"
            "- Do not simply list evidence or cards—write the full speech, integrating evidence and arguments as a debater would."
            "\nIMPORTANT: Output the 2AR speech transcript in HTML format, using <div>, <h2>, <b>, <ul>, <li>, <p>, and similar tags as appropriate. Do NOT use <pre>, <code>, or markdown formatting. The output should visually match the style of the input card HTML formats."
        ),
        llm_config=llm_config,
    )

    # 2AR Coach: Reviews and suggests improvements for the 2AR speech
    two_ar_coach = ConversableAgent(
        name="two_ar_coach",
        system_message=(
            "You are a highly experienced debate coach and judge. "
            "Your job is to review the 2AR speech draft and provide detailed, constructive feedback for improvement. "
            "Focus on:\n"
            "- Argument coverage: Did the 2AR answer all the most important 2NR and 1NR arguments?\n"
            "- Strategic focus: Did the 2AR focus on the arguments the affirmative is winning and answer the negative's best arguments?\n"
            "- Clarity and organization: Is the speech easy to flow and follow?\n"
            "- Persuasiveness and use of evidence: Are arguments well-supported and explained?\n"
            "- Realism: Does the speech sound like a real, high-level 2AR?\n"
            "Suggest specific improvements, then ask the debater to revise the speech accordingly."
        ),
        llm_config=llm_config,
    )

    from autogen import GroupChat

    # Speaker selection: alternate between drafter and coach for 2-3 rounds
    def speaker_selection_func(last_speaker, groupchat):
        # First message: drafter writes initial speech
        if len(groupchat.messages) == 0:
            return two_ar_drafter
        # Drafter just wrote: coach reviews
        if last_speaker is two_ar_drafter:
            return None
        if last_speaker is two_ar_coach:
            return two_ar_drafter
        return None

    group_chat = GroupChat(
        agents=[two_ar_drafter, two_ar_coach],
        messages=[],
        max_round=4,
        speaker_selection_method=speaker_selection_func
    )

    group_chat_manager = GroupChatManager(
        groupchat=group_chat,
        llm_config=llm_config,
    )

    # Compose the context for the 2AR drafter
    context_message = (
        f"{debate_case}\n\n"
        f"Negative Case (1NC):\n{negative_case_html}\n\n"
        f"2AC Speech and Cards:\n{twoac_case_html}\n\n"
        f"2NC Speech and Cards:\n{twonc_case_html}\n\n"
        f"1NR Speech and Cards:\n{onenr_html}\n\n"
        f"1AR Speech and Cards:\n{onear_case_html}\n\n"
        f"2NR Speech and Cards:\n{two_nr_case_html}\n\n"
        "Assume that the current year is 2022.\n"
        "Write a complete, high-quality, realistic 2AR speech transcript as if you are reading it aloud in a debate round. "
        "Cover all major 2NR and 1NR arguments, extend the affirmative, and answer all negative arguments. "
        "Use debate structure and jargon. Do not simply list evidence—write the full speech. "
        "When answering negative arguments or new arguments, focus your time on the arguments the affirmative is winning and explain why the affirmative wins."
        "\nIMPORTANT: Output the 2AR speech transcript in HTML format, using <div>, <h2>, <b>, <ul>, <li>, <p>, and similar tags as appropriate. Do NOT use <pre>, <code>, or markdown formatting. The output should visually match the style of the input card HTML formats."
    )

    # Start the group chat
    chat_result = two_ar_coach.initiate_chat(
        group_chat_manager,
        message=context_message,
    )

    # Find the last 2AR speech draft in the chat history
    # (The drafter's last message)
    transcript = chat_result.chat_history[-1]["content"]
    # Append the 2AR transcript to the 2ar_case_html
    two_ar_case_html = "\n<div class='two-ar-section'>\n"
    two_ar_case_html += "<h1>2AR Speech</h1>\n"
    two_ar_case_html += f"<div class='twoar-transcript'>{transcript}</div>\n"
    two_ar_case_html += "</div>\n"

    return two_ar_case_html

In [ ]:
two_ar_html = add_2ar_to_case(debate_case, negative_case_html, new_ac_case, two_nc_html_full, onenr_html, one_ar_html, two_nr_html)

In [ ]:
display(HTML(two_ar_html))

In [ ]:
two_ar_html

# Judge Vote!

In [ ]:
def judge_decision_on_round(
    debate_case,
    negative_case_html,
    twoac_case_html,
    twonc_case_html,
    onear_case_html,
    onenr_html,
    two_nr_case_html,
    two_ar_case_html
):
    """
    Simulates a debate judge voting on the round and writing a detailed RFD (Reason for Decision).
    The function takes as input:
        - debate_case (HTML, including 1AC)
        - negative_case_html (HTML, the 1NC)
        - twoac_case_html (HTML for the 2AC section)
        - twonc_case_html (HTML for the 2NC section)
        - onear_case_html (HTML for the 1AR section)
        - onenr_html (HTML for the 1NR section)
        - two_nr_case_html (HTML for the 2NR section)
        - two_ar_case_html (HTML for the 2AR section)
    It returns judge_decision_html, which includes the judge's vote and a long, detailed RFD.
    """

    llm_config = LLMConfig(
        api_type="openai",
        model="gpt-4.1-mini",
        api_key=OPENAI_API_KEY,
        temperature=2.0,
        top_p=0.9,
        parallel_tool_calls=None
    )

    # Judge: Reads the round and writes a detailed RFD and decision
    judge_agent = ConversableAgent(
        name="debate_judge",
        system_message=(
            "You are a highly experienced policy debate judge. "
            "You have just listened to a full debate round, including the 1AC, 1NC, 2AC, 2NC, 1AR, 1NR, 2NR, and 2AR. "
            "You must act as unbiasedly as possible, judging only the arguments and evidence presented in the round. "
            "Be open to voting for positions or arguments that you might personally disagree with, if the debaters for that side made more compelling arguments and won the relevant issues. "
            "Your job is to decide who won the round (Affirmative or Negative) and write a long, detailed Reason for Decision (RFD). "
            "Your RFD should:\n"
            "- Clearly state who you voted for (Affirmative or Negative) at the top\n"
            "- Explain, in detail, the most important issues in the round and how you resolved them\n"
            "- Reference specific arguments and speeches (including the 2AR, 2NR, and 1NR)\n"
            "- Flow the round and explain how you evaluated the arguments and evidence\n"
            "- Use debate jargon and structure as a real judge would\n"
            "- Be realistic, thorough, and educational for the debaters\n"
            "- Do not simply summarize the speeches—explain your decision process and why you voted the way you did"
            "\nIMPORTANT: Output your RFD in HTML format, using <div>, <h2>, <b>, <ul>, <li>, <p>, and similar tags as appropriate. Do NOT use <pre>, <code>, or markdown formatting. The output should visually match the style of the input card HTML formats."
        ),
        llm_config=llm_config,
    )

    # Judge Coach: Reviews the RFD and suggests improvements
    judge_coach_agent = ConversableAgent(
        name="judge_coach",
        system_message=(
            "You are a debate coach and judge trainer. "
            "Your job is to review the judge's RFD and provide detailed, constructive feedback for improvement. "
            "Focus on:\n"
            "- Clarity: Is the decision and RFD clear and easy to follow?\n"
            "- Argument evaluation: Did the judge accurately flow and resolve the key issues?\n"
            "- Educational value: Does the RFD help debaters understand what happened and how to improve?\n"
            "- Realism: Does the RFD sound like a real, high-level debate judge?\n"
            "Suggest specific improvements, then ask the judge to revise the RFD accordingly."
        ),
        llm_config=llm_config,
    )

    from autogen import GroupChat

    # Speaker selection: alternate between judge and coach for 2-3 rounds
    def speaker_selection_func(last_speaker, groupchat):
        # First message: judge writes initial RFD
        if last_speaker is judge_coach_agent:
            return judge_agent
        else:
            return None

    group_chat = GroupChat(
        agents=[judge_agent, judge_coach_agent],
        messages=[],
        max_round=2,
        speaker_selection_method=speaker_selection_func
    )

    group_chat_manager = GroupChatManager(
        groupchat=group_chat,
        llm_config=llm_config,
    )

    # Compose the context for the judge
    context_message = (
        f"{debate_case}\n\n"
        f"Negative Case (1NC):\n{negative_case_html}\n\n"
        f"2AC Speech and Cards:\n{twoac_case_html}\n\n"
        f"2NC Speech and Cards:\n{twonc_case_html}\n\n"
        f"1AR Speech and Cards:\n{onear_case_html}\n\n"
        f"1NR Speech and Cards:\n{onenr_html}\n\n"
        f"2NR Speech and Cards:\n{two_nr_case_html}\n\n"
        f"2AR Speech and Cards:\n{two_ar_case_html}\n\n"
        "Assume that the current year is 2022.\n"
        "You are the judge of this debate round. "
        "You must act as unbiasedly as possible, judging only the arguments and evidence presented in the round. "
        "Be open to voting for positions or arguments that you might personally disagree with, if the debaters for that side made more compelling arguments and won the relevant issues. "
        "Write your decision: clearly state who you voted for (Affirmative or Negative) and then write a long, detailed Reason for Decision (RFD) explaining how you evaluated the round, which arguments you found most important, and why you voted the way you did. "
        "Reference specific arguments and speeches, and use debate structure and jargon. "
        "Be realistic and thorough."
        "\nIMPORTANT: Output your RFD in HTML format, using <div>, <h2>, <b>, <ul>, <li>, <p>, and similar tags as appropriate. Do NOT use <pre>, <code>, or markdown formatting. The output should visually match the style of the input card HTML formats."
    )

    # Start the group chat
    chat_result = judge_coach_agent.initiate_chat(
        group_chat_manager,
        message=context_message,
    )

    # Find the last RFD in the chat history (judge's last message)
    rfd = chat_result.chat_history[-1]["content"]
    # Format the judge decision as HTML
    judge_decision_html = "\n<div class='judge-decision-section'>\n"
    judge_decision_html += "<h1>Judge Decision and RFD</h1>\n"
    judge_decision_html += f"<div class='judge-rfd'>{rfd}</div>\n"
    judge_decision_html += "</div>\n"

    return judge_decision_html

In [ ]:
judge_html = judge_decision_on_round(debate_case, negative_case_html, new_ac_case, two_nc_html_full, one_ar_html, onenr_html, two_nr_html, two_ar_html)

In [ ]:
display(HTML(judge_html))

In [ ]:
judge_html

In [ ]:
agentops.end_session("Success")